In [6]:
PULSE_MIN = 5


In [1]:
# CNT Engine — Mega Cell (SSD Edition)
# Rooted for: E:\CNT    (override with environment variable CNT_LAB_DIR)
# Purpose   : Self-referential read → hidden-truth search → gated self-update
# Outputs   : artifacts/cnt_engine_megacell/{CNT_STATE.yaml, out/hidden_truths_*.csv, runlog.jsonl}
# Run once  : just execute this cell.
# Pulse     : set PULSE_MIN > 0 to keep cycling every N minutes while notebook is open.

import os, json, glob, re, time
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

# ---------- CONFIG ----------
LAB_ROOT = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
SOURCE_ROOTS = [
    LAB_ROOT / "notes",
    LAB_ROOT / "artifacts" / "cnt_scroll",
    LAB_ROOT / "artifacts" / "cnt_codex",
    LAB_ROOT / "notebooks",
]
ENGINE_ROOT  = LAB_ROOT / "artifacts" / "cnt_engine_megacell"
OUT_DIR      = ENGINE_ROOT / "out"
STATE_FILE   = ENGINE_ROOT / "CNT_STATE.yaml"
RUNLOG_FILE  = ENGINE_ROOT / "runlog.jsonl"

for p in [ENGINE_ROOT, OUT_DIR, *SOURCE_ROOTS]:
    p.mkdir(parents=True, exist_ok=True)

# Pulse in minutes (0 = run once)
PULSE_MIN = 0

def now():
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---------- DISCOVER / SEED ----------
def list_sources():
    files = []
    for root in SOURCE_ROOTS:
        files += glob.glob(str(root / "**" / "*.md"), recursive=True)
        files += glob.glob(str(root / "**" / "*.txt"), recursive=True)
    return [Path(f) for f in files]

def auto_seed_if_empty():
    paths = list_sources()
    if paths: 
        return
    seed = SOURCE_ROOTS[0] / "cnt_seed.md"
    seed.parent.mkdir(parents=True, exist_ok=True)
    seed.write_text(
        "# CNT Seed\\n\\n"
        "Hypothesis: Certain glyph–field pairings lower entropy drift.\\n"
        "Test: Re-run θ metrics on EEG segments with glyph overlay vs baseline.\\n"
        "Expected: Δθ > 0 with CI > 95% for overlay.\\n"
        "Falsifier: No uplift or negative drift after 1k permutations.\\n",
        encoding="utf-8"
    )

# ---------- INDEX & RESIDUALS ----------
def build_index(df):
    if df.empty: 
        return None, None, None
    vec = TfidfVectorizer(max_features=6000, ngram_range=(1,2))
    X = vec.fit_transform(df["text"]).astype(np.float32)
    n_samples, n_features = X.shape
    # robust rank
    nmax = max(2, min(n_samples - 1, n_features - 1, 128))
    svd = TruncatedSVD(n_components=nmax, random_state=42)
    Xd = svd.fit_transform(X)
    Xr = svd.inverse_transform(Xd)
    resid = np.linalg.norm(X.toarray() - Xr, axis=1)
    return vec, X, resid

# ---------- CLUSTERS & GRAPH ----------
def cluster_labels(X, k=6):
    if X is None: 
        return None
    k = max(2, min(k, X.shape[0]-1))
    km = KMeans(n_clusters=k, n_init=5, random_state=42)
    return km.fit_predict(X.toarray())

def build_graph(df, lbl):
    """Robust graph builder using regex tokenization (no pandas extractall)."""
    G = nx.Graph()
    for i, row in df.reset_index().iterrows():
        G.add_node(
            i,
            path=row["path"],
            title=os.path.basename(row["path"]),
            cluster=int(lbl[i]) if lbl is not None else -1
        )
    pattern = re.compile(r"\\b([A-Z][A-Za-z0-9_]{3,})\\b")
    caps_by_row = [set(pattern.findall(t if isinstance(t, str) else "")) for t in df["text"].tolist()]
    N = len(df)
    for i in range(N):
        for j in range(i+1, N):
            overlap = len(caps_by_row[i] & caps_by_row[j])
            same_cluster = (lbl is not None and lbl[i] == lbl[j])
            if same_cluster or overlap >= 2:
                G.add_edge(i, j, w=(1 + overlap))
    return G

# ---------- SCORES ----------
def score_reflexive(df, resid, G):
    if df is None or len(df)==0:
        return dict(clarity=0, novelty=0, coherence=0, falsifiability=0, total=0)
    clarity = 1.0 - np.mean([min(len(t),10000)/10000 for t in df["text"].tolist()])
    novelty = float(np.mean(resid)/(np.std(resid)+1e-6))
    coherence = nx.average_clustering(G) if G.number_of_nodes()>1 else 0.0
    falsifiability = float(np.mean([t.lower().count("test")+t.lower().count("predict") for t in df["text"]]))/10.0
    clarity = np.clip(clarity,0,1); novelty = np.clip(novelty/1.5,0,1)
    coherence = np.clip(coherence,0,1); falsifiability = np.clip(falsifiability,0,1)
    total = float(np.mean([clarity,novelty,coherence,falsifiability]))
    return dict(clarity=float(clarity), novelty=float(novelty),
                coherence=float(coherence), falsifiability=float(falsifiability),
                total=float(total))

# ---------- SURFACING / UPDATES ----------
def surface_candidates(df, resid, top=5):
    idx = np.argsort(resid)[::-1][:min(top, len(resid))]
    picks=[]
    for i in idx:
        snippet=re.sub(r"\\s+"," ",df.iloc[i]["text"])[:400]
        picks.append(dict(path=df.iloc[i]["path"],resid=float(resid[i]),hint=snippet))
    return picks

def propose_updates(cands):
    return [dict(
        target=c["path"],
        content=(
            f"\\n\\n> CNT-Gloss ({now()}): Clarify hypothesis; add test recipe & falsifier.\\n"
            "- Hypothesis: …\\n- Measurement: …\\n- Expected shift: …\\n- Falsifier: …\\n"
        )
    ) for c in cands]

def legality_gate(text): 
    bad=any(k in text for k in["ssn","credit card","weapon","harm"])
    return not bad

def confab_gate(text):
    t=text.lower(); return ("hypothesis" in t and "falsifier" in t)

def apply_updates(updates):
    accepted=[]
    for u in updates:
        try:
            if not (legality_gate(u["content"]) and confab_gate(u["content"])): 
                continue
            p=Path(u["target"])
            p.write_text(p.read_text(encoding="utf-8",errors="ignore")+u["content"],encoding="utf-8")
            accepted.append(u)
        except Exception: 
            pass
    return accepted

# ---------- STATE / LOG ----------
def write_state(score,meta):
    ENGINE_ROOT.mkdir(parents=True, exist_ok=True)
    try:
        import yaml
        Path(STATE_FILE).write_text(
            yaml.safe_dump(dict(updated=now(),score=score,meta=meta),sort_keys=False),
            encoding="utf-8"
        )
    except Exception:
        Path(STATE_FILE).write_text(json.dumps(dict(updated=now(),score=score,meta=meta),indent=2),encoding="utf-8")

def log_event(kind,payload):
    Path(RUNLOG_FILE).parent.mkdir(parents=True, exist_ok=True)
    with Path(RUNLOG_FILE).open("a",encoding="utf-8") as f:
        f.write(json.dumps(dict(ts=now(),kind=kind,**payload))+"\\n")

# ---------- MAIN CYCLE ----------
def run_cycle():
    auto_seed_if_empty()
    paths=list_sources()
    df=pd.DataFrame([dict(path=str(p),text=p.read_text(encoding="utf-8",errors="ignore")) for p in paths])
    if df.empty:
        log_event("empty",{"roots":[str(r) for r in SOURCE_ROOTS]})
        return {"empty":True,"roots":[str(r) for r in SOURCE_ROOTS]}
    vec,X,resid=build_index(df)
    labels=cluster_labels(X)
    G=build_graph(df,labels)
    score=score_reflexive(df,resid,G)
    cands=surface_candidates(df,resid,top=5)
    accepted=apply_updates(propose_updates(cands))
    write_state(score,dict(docs=len(df),accepted=len(accepted)))
    pd.DataFrame(cands).to_csv(OUT_DIR/f"hidden_truths_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv",index=False)
    log_event("cycle",dict(score=score,proposed=len(cands),accepted=len(accepted)))
    return dict(score=score,proposed=len(cands),accepted=len(accepted),hidden=cands[:3])

# ---------- EXECUTE (one or pulsed) ----------
def run_pulsed(pulse_minutes: int):
    """Run forever with a sleep between cycles."""
    assert pulse_minutes > 0
    print(f"[CNT Engine] Pulsed mode every {pulse_minutes} min — CTRL+C to stop.")
    while True:
        res = run_cycle()
        print(json.dumps(res, indent=2))
        time.sleep(pulse_minutes * 60)

# Run now
if PULSE_MIN and PULSE_MIN > 0:
    run_pulsed(PULSE_MIN)
else:
    result = run_cycle()
    print(json.dumps(result, indent=2))


{
  "score": {
    "clarity": 0.8306804878048781,
    "novelty": 0.6710594495137533,
    "coherence": 1.0,
    "falsifiability": 0.024390243902439025,
    "total": 0.6315325453052676
  },
  "proposed": 5,
  "accepted": 5,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 2.3844481802370865e-06,
      "hint": "# Gauge-Restored Agents (GRA) v0.3\n**Contract:** Invariant \u2192 Restored \u2192 True (or Abstain)\n\nThis repo ships a tiny, enforceable safety contract for LLM answers:\n1) Keep meaning invariant under symbol-preserving transformations,\n2) Restore outputs to a domain-safe format when form wobbles,\n3) Verify claims (or ABSTAIN if truth is uncertain).\n\n## Domains (v0.3)\n- **Math:** structured gate (equation +"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 2.25750545723713e-06,
      "hint": "== CNT Correlates Audit (Fused) ==\nTimestamp: 20251015-164130\n

In [2]:
# ---- Falsifiability: attach a runnable test stub per proposal ----
def attach_test_stub(text: str, target_path: str):
    """Append a lightweight, runnable test recipe block."""
    stem = Path(target_path).stem.replace(" ", "_")
    return (
        text +
        "\n\n```cnt-test\n"
        f"name: test_{stem}\n"
        "inputs: []\n"
        "recipe:\n"
        "  - step: load_data\n"
        "    path: <fill_me>\n"
        "  - step: compute_metric\n"
        "    code: <python or pseudo>\n"
        "  - step: assert\n"
        "    expect: <delta_theta> > 0 and p < 0.05\n"
        "```"
    )

def propose_updates(cands):
    updates = []
    for c in cands:
        gloss = (
            f"\n\n> CNT-Gloss ({now()}): Clarify hypothesis; add test recipe & falsifier.\n"
            "- Hypothesis: …\n- Measurement: …\n- Expected shift: …\n- Falsifier: …\n"
        )
        content = attach_test_stub(gloss, c["path"])
        updates.append(dict(target=c["path"], content=content))
    return updates


In [3]:
# ---- Minimal test runner: scans for ```cnt-test``` blocks and counts runnable specs ----
TEST_PATTERN = re.compile(r"```cnt-test(?P<body>.*?)```", re.DOTALL)

def count_runnable_tests(df):
    cnt = 0
    for txt in df["text"].tolist():
        for m in TEST_PATTERN.finditer(txt or ""):
            body = m.group("body")
            # consider it 'runnable' if it names a test and has an 'assert' line
            if "name:" in body and "assert" in body:
                cnt += 1
    return cnt

def score_reflexive(df, resid, G):
    if df is None or len(df)==0:
        return dict(clarity=0, novelty=0, coherence=0, falsifiability=0, total=0)
    clarity = 1.0 - np.mean([min(len(t),10000)/10000 for t in df["text"].tolist()])
    novelty = float(np.mean(resid)/(np.std(resid)+1e-6))
    coherence = nx.average_clustering(G) if G.number_of_nodes()>1 else 0.0
    # old proxy + runnable tests bonus
    fals_proxy = float(np.mean([t.lower().count("test")+t.lower().count("predict") for t in df["text"]]))/10.0
    runnable = count_runnable_tests(df)
    falsifiability = np.clip(fals_proxy + min(0.5, 0.05 * runnable), 0, 1)
    clarity = np.clip(clarity,0,1); novelty = np.clip(novelty/1.5,0,1)
    coherence = np.clip(coherence,0,1)
    total = float(np.mean([clarity,novelty,coherence,falsifiability]))
    return dict(clarity=float(clarity), novelty=float(novelty),
                coherence=float(coherence), falsifiability=float(falsifiability),
                total=float(total))


In [4]:
def build_graph(df, lbl):
    G = nx.Graph()
    for i, row in df.reset_index().iterrows():
        G.add_node(
            i,
            path=row["path"],
            title=os.path.basename(row["path"]),
            cluster=int(lbl[i]) if lbl is not None else -1
        )
    pattern = re.compile(r"\b([A-Z][A-Za-z0-9_]{4,})\b")  # require 4+ chars
    caps_by_row = [set(pattern.findall(t if isinstance(t, str) else "")) for t in df["text"].tolist()]

    N = len(df)
    for i in range(N):
        for j in range(i+1, N):
            # cap token set sizes to avoid huge titles dominating
            si = set(list(caps_by_row[i])[:50])
            sj = set(list(caps_by_row[j])[:50])
            overlap = len(si & sj)
            same_cluster = (lbl is not None and lbl[i] == lbl[j])
            if same_cluster or overlap >= 3:   # require ≥3 shared tokens
                G.add_edge(i, j, w=(1 + overlap))
    return G


In [5]:
from sklearn.ensemble import IsolationForest

def build_index(df):
    if df.empty:
        return None, None, None
    vec = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
    X = vec.fit_transform(df["text"]).astype(np.float32)
    n_samples, n_features = X.shape
    nmax = max(2, min(n_samples - 1, n_features - 1, 256))
    svd = TruncatedSVD(n_components=nmax, random_state=42)
    Xd = svd.fit_transform(X)  # low-dim representation

    # anomaly score (higher = more novel)
    iso = IsolationForest(n_estimators=200, contamination="auto", random_state=42)
    iso.fit(Xd)
    iso_score = -iso.score_samples(Xd)  # invert: larger -> more anomalous

    # blend with recon residual for stability
    Xr = svd.inverse_transform(Xd)
    resid = np.linalg.norm(X.toarray() - Xr, axis=1)
    resid = (resid - resid.min()) / (resid.ptp() + 1e-9)
    novel = 0.6 * iso_score / (iso_score.max() + 1e-9) + 0.4 * resid
    return vec, X, novel


In [7]:
# make sure the lab root is set inside the kernel
import os
os.environ["CNT_LAB_DIR"] = r"E:\CNT"

# run a single pass
res = run_cycle()
import json; print(json.dumps(res, indent=2))


AttributeError: `ptp` was removed from the ndarray class in NumPy 2.0. Use np.ptp(arr, ...) instead.

In [8]:
from pathlib import Path
import pandas as pd

out_dir = Path(r"E:\CNT\artifacts\cnt_engine_megacell\out")
latest = sorted(out_dir.glob("hidden_truths_*.csv"))[-1]
pd.read_csv(latest).head(10)


,path,resid,hint
0,E:\CNT\notebooks\archive\gra-v0.3\README.md,0.000002,# Gauge-Restored Agents (GRA) v0.3\n**Contract...
1,E:\CNT\notebooks\archive\cnt_correlates_report...,0.000002,== CNT Correlates Audit (Fused) ==\nTimestamp:...
2,E:\CNT\notebooks\archive\cnt_correlates_report...,0.000002,== CNT Correlates Audit ==\nTimestamp: 2025101...
3,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,0.000002,\nCNT – 3I/ATLAS All-8 Mega Pack (v1.1)\n=====...
4,E:\CNT\notebooks\archive\cnt_one_signal_202510...,0.000002,== CNT One-Signal — Unified Verdict (v3.3) ==\...


In [9]:
from pathlib import Path
state = Path(r"E:\CNT\artifacts\cnt_engine_megacell\CNT_STATE.yaml").read_text(encoding="utf-8")
print(state)

log_path = Path(r"E:\CNT\artifacts\cnt_engine_megacell\runlog.jsonl")
print("\n--- last 5 log lines ---")
print("".join(log_path.read_text(encoding="utf-8").splitlines(True)[-5:]))


updated: '2025-10-26T21:18:00Z'
score:
  clarity: 0.8306804878048781
  novelty: 0.6710594495137533
  coherence: 1.0
  falsifiability: 0.024390243902439025
  total: 0.6315325453052676
meta:
  docs: 41
  accepted: 5


--- last 5 log lines ---
{"ts": "2025-10-26T21:18:00Z", "kind": "cycle", "score": {"clarity": 0.8306804878048781, "novelty": 0.6710594495137533, "coherence": 1.0, "falsifiability": 0.024390243902439025, "total": 0.6315325453052676}, "proposed": 5, "accepted": 5}\n


In [10]:
from sklearn.ensemble import IsolationForest
import numpy as np

def build_index(df):
    if df.empty:
        return None, None, None

    vec = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
    X = vec.fit_transform(df["text"]).astype(np.float32)
    n_samples, n_features = X.shape

    # Low-dim representation via SVD
    nmax = max(2, min(n_samples - 1, n_features - 1, 256))
    svd = TruncatedSVD(n_components=nmax, random_state=42)
    Xd = svd.fit_transform(X)

    # Anomaly score (higher = more novel)
    iso = IsolationForest(n_estimators=200, contamination="auto", random_state=42)
    iso.fit(Xd)
    iso_score = -iso.score_samples(Xd)  # invert so larger => more anomalous

    # Reconstruction residual for stability
    Xr = svd.inverse_transform(Xd)
    resid = np.linalg.norm(X.toarray() - Xr, axis=1)

    # Robust min-max with NumPy 2.x
    resid_range = np.ptp(resid)  # <-- np.ptp replaces ndarray.ptp()
    resid_norm = (resid - resid.min()) / (resid_range + 1e-9)

    iso_max = float(np.max(iso_score))
    iso_norm = iso_score / (iso_max + 1e-9)

    # Blend
    novel = 0.6 * iso_norm + 0.4 * resid_norm
    return vec, X, novel


In [11]:
res = run_cycle()
import json; print(json.dumps(res, indent=2))


{
  "score": {
    "clarity": 0.8287048780487805,
    "novelty": 1.0,
    "coherence": 0.901545744228671,
    "falsifiability": 0.036585365853658534,
    "total": 0.6917089970327776
  },
  "proposed": 5,
  "accepted": 5,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9543617030772218,
      "hint": "== CNT Correlates Audit (Fused) ==\nTimestamp: 20251015-164130\n\n- Cooling logs: 8\n- Cosmo triage tables: 1\n- EEG laterality tables: 2\n- Kuramoto/Ising summaries: 0\n- Kuramoto dispersion CSVs: 0\n- Gray-Scott edge images: 1\n- Scanned logs/text: 50\n- Glyph label tables: 0\n\n== Results ==\n[GREY] Cooling: Temp oscillation amplitude predicts clock stability\n  Need logs with GPU temp + GPU clock columns.\n "
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_3i_atlas_all8_20251024-035650Z_044afb63\\README.txt",
      "resid": 0.9256719836458015,
      "hint": "\nCNT \u2013 3I/ATLAS All-8 Mega Pack (

In [12]:
import yaml

def _yaml_block(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False) + "```\n"

def _make_test_for(target_path: str):
    stem = Path(target_path).stem.replace(" ", "_").lower()
    # default pattern: CSV threshold test — fill real path later
    return dict(
        name=f"test_{stem}",
        type="csv_threshold",
        path="<fill_me_csv_path>",        # e.g., E:\\CNT\\cnt_ewb_theta2\\publish\\ewb_publish_v0_2\\metrics.csv
        column="<metric_column>",         # e.g., "delta_theta" or "precision_at_theta"
        op=">",                           # one of: >, >=, <, <=
        value=0.0,                        # set your threshold
        agg="mean"                        # mean | median | any | all
    )

def propose_updates(cands):
    updates = []
    for c in cands:
        gloss = (
            f"\n\n> CNT-Gloss ({now()}): Clarify hypothesis; add test recipe & falsifier.\n"
            "- Hypothesis: …\n- Measurement: …\n- Expected shift: …\n- Falsifier: …\n\n"
        )
        test = _make_test_for(c["path"])
        content = gloss + _yaml_block(test)
        updates.append(dict(target=c["path"], content=content))
    return updates


In [13]:
TEST_FENCE = re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL)

def _parse_cnt_tests(text: str):
    tests = []
    for m in TEST_FENCE.finditer(text or ""):
        body = m.group("body")
        try:
            t = yaml.safe_load(body)  # expect dict
            if isinstance(t, dict) and "type" in t:
                tests.append(t)
        except Exception:
            pass
    return tests

def _csv_threshold_test(t: dict) -> tuple[bool,str]:
    """
    Schema:
      type: csv_threshold
      path: E:\\CNT\\...\\file.csv
      column: metric_column
      op: > | >= | < | <=
      value: float
      agg: mean | median | any | all
    """
    path = t.get("path")
    col  = t.get("column")
    op   = t.get("op", ">")
    val  = float(t.get("value", 0.0))
    agg  = t.get("agg", "mean").lower()

    if not path or "<fill_me" in str(path) or not Path(path).exists():
        return False, "missing_path"

    df = pd.read_csv(path)
    if col not in df.columns:
        return False, "missing_column"

    s = pd.to_numeric(df[col], errors="coerce").dropna()
    if s.empty:
        return False, "no_numeric"

    if agg == "mean":   metric = float(s.mean())
    elif agg == "median": metric = float(s.median())
    elif agg == "any":    metric = bool((s > val).any()) if op in [">", ">="] else bool((s < val).any())
    elif agg == "all":    metric = bool((s > val).all()) if op in [">", ">="] else bool((s < val).all())
    else:                 metric = float(s.mean())

    def _cmp(a, b, op):
        return {">": a>b, ">=": a>=b, "<": a<b, "<=": a<=b}[op]

    # boolean aggregates are already True/False
    if isinstance(metric, bool):
        ok = metric
        info = f"agg={agg}"
    else:
        ok = _cmp(metric, val, op)
        info = f"{agg}({col})={metric:.6f} {op} {val}"

    return bool(ok), info

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    passed = 0
    total  = 0
    for _, r in df.iterrows():
        path = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        for t in tests:
            total += 1
            name = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info = _csv_threshold_test(t)
            else:
                ok, info = False, f"unknown_type:{ttype}"
            rows.append(dict(doc=path, name=name, type=ttype, pass=int(ok), info=info))
            if ok: passed += 1
    out_df = pd.DataFrame(rows)
    if not out_df.empty:
        out_df.to_csv(out_csv, index=False)
    return passed, total


SyntaxError: invalid syntax (1105436143.py, line 76)

In [14]:
def run_cycle():
    auto_seed_if_empty()
    paths=list_sources()
    df=pd.DataFrame([dict(path=str(p),text=p.read_text(encoding="utf-8",errors="ignore")) for p in paths])
    if df.empty:
        log_event("empty",{"roots":[str(r) for r in SOURCE_ROOTS]})
        return {"empty":True,"roots":[str(r) for r in SOURCE_ROOTS]}

    vec,X,resid=build_index(df)
    labels=cluster_labels(X)
    G=build_graph(df,labels)

    # --- run tests (NEW) ---
    tests_csv = OUT_DIR / f"tests_report_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv"
    passed, total = run_cnt_tests_on_df(df, tests_csv)

    # score
    score=score_reflexive(df,resid,G)

    # small falsifiability boost from actual passes (not just stubs)
    if total > 0:
        frac = passed / max(1,total)
        score["falsifiability"] = float(np.clip(score["falsifiability"] + min(0.5, 0.25*frac), 0, 1))
        score["total"] = float(np.mean([score[k] for k in ["clarity","novelty","coherence","falsifiability"]]))

    cands=surface_candidates(df,resid,top=5)
    accepted=apply_updates(propose_updates(cands))
    write_state(score,dict(docs=len(df),accepted=len(accepted),tests_total=total,tests_passed=passed))
    pd.DataFrame(cands).to_csv(OUT_DIR/f"hidden_truths_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv",index=False)
    log_event("cycle",dict(score=score,proposed=len(cands),accepted=len(accepted),tests_total=total,tests_passed=passed))
    return dict(score=score,proposed=len(cands),accepted=len(accepted),tests_total=total,tests_passed=passed,hidden=cands[:3])


In [15]:
TEST_FENCE = re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL)

def _parse_cnt_tests(text: str):
    tests = []
    for m in TEST_FENCE.finditer(text or ""):
        body = m.group("body")
        try:
            t = yaml.safe_load(body)  # expect dict
            if isinstance(t, dict) and "type" in t:
                tests.append(t)
        except Exception:
            pass
    return tests

def _csv_threshold_test(t: dict) -> tuple[bool, str]:
    """
    Schema:
      type: csv_threshold
      path: E:\\CNT\\...\\file.csv
      column: metric_column
      op: > | >= | < | <=
      value: float
      agg: mean | median | any | all
    """
    path = t.get("path")
    col  = t.get("column")
    op   = t.get("op", ">")
    val  = float(t.get("value", 0.0))
    agg  = t.get("agg", "mean").lower()

    if not path or "<fill_me" in str(path) or not Path(path).exists():
        return False, "missing_path"

    df = pd.read_csv(path)
    if col not in df.columns:
        return False, "missing_column"

    s = pd.to_numeric(df[col], errors="coerce").dropna()
    if s.empty:
        return False, "no_numeric"

    if agg == "mean":      metric = float(s.mean())
    elif agg == "median":  metric = float(s.median())
    elif agg == "any":     metric = bool((s > val).any()) if op in [">", ">="] else bool((s < val).any())
    elif agg == "all":     metric = bool((s > val).all()) if op in [">", ">="] else bool((s < val).all())
    else:                  metric = float(s.mean())

    def _cmp(a, b, op):
        return {">": a>b, ">=": a>=b, "<": a<b, "<=": a<=b}[op]

    if isinstance(metric, bool):         # boolean aggregates already resolved
        ok = metric
        info = f"agg={agg}"
    else:
        ok = _cmp(metric, val, op)
        info = f"{agg}({col})={metric:.6f} {op} {val}"

    return bool(ok), info

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    passed = 0
    total  = 0
    for _, r in df.iterrows():
        path = r["path"]
        tests = _parse_cnt_tests(r.get("text", ""))
        for t in tests:
            total += 1
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type", "unknown")
            if ttype == "csv_threshold":
                ok, info = _csv_threshold_test(t)
            else:
                ok, info = False, f"unknown_type:{ttype}"
            rows.append(dict(doc=path, name=name, type=ttype, passed=int(ok), info=info))
            if ok:
                passed += 1
    out_df = pd.DataFrame(rows)
    if not out_df.empty:
        out_df.to_csv(out_csv, index=False)
    return passed, total


In [16]:
res = run_cycle()
import json; print(json.dumps(res, indent=2))


{
  "score": {
    "clarity": 0.8241658536585366,
    "novelty": 1.0,
    "coherence": 0.8755071287711174,
    "falsifiability": 0.3231707317073171,
    "total": 0.7557109285342427
  },
  "proposed": 5,
  "accepted": 5,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9125069984119454,
      "hint": "== CNT Correlates Audit (Fused) ==\nTimestamp: 20251015-164130\n\n- Cooling logs: 8\n- Cosmo triage tables: 1\n- EEG laterality tables: 2\n- Kuramoto/Ising summaries: 0\n- Kuramoto dispersion CSVs: 0\n- Gray-Scott edge images: 1\n- Scanned logs/text: 50\n- Glyph label tables: 0\n\n== Results ==\n[GREY] Cooling: Temp oscillation amplitude predicts clock stability\n  Need logs with GPU temp + GPU clock columns.\n "
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_ewb_theta2\\ewb_results.md",
      "resid": 0.891309430343602,
      "hint": "# CNT \u0398* Early-Warni

In [17]:
from pathlib import Path
import pandas as pd
import yaml, re

LAB = Path(r"E:\CNT")
DOCS = [
    LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md",
    LAB / r"notebooks\archive\cnt_ewb_theta2\ewb_results.md",
]

# 1) Find a real metrics CSV to test against
candidates = []
for p in (LAB / r"notebooks\archive\cnt_ewb_theta2").rglob("*.csv"):
    try:
        cols = pd.read_csv(p, nrows=1).columns.str.lower().tolist()
        candidates.append((p, cols))
    except Exception:
        pass

print("Found CSVs:", len(candidates))
for p, cols in candidates[:5]:
    print(" -", p, "cols:", cols[:6], "…")

# Heuristics: pick first csv with a familiar metric column
PREFERRED = ["precision_at_theta", "precision@theta*", "auc", "delta_theta", "lead_at_hit", "lead_at_any"]
chosen = None
chosen_col = None
for p, cols in candidates:
    for c in cols:
        if any(k == c or k in c for k in PREFERRED):
            chosen, chosen_col = p, c
            break
    if chosen: break

if not chosen:
    raise SystemExit("No suitable CSV/column found. Open one CSV, tell me the column to test, and re-run.")

print("\nUsing:", chosen, "column:", chosen_col)

# 2) Compose a concrete cnt-test block
test_spec = dict(
    name="test_ewb_precision",
    type="csv_threshold",
    path=str(chosen),
    column=chosen_col,
    op=">",          # adjust if you want "<" style tests
    value=0.55,      # set your threshold
    agg="mean"       # mean | median | any | all
)
fenced = "```cnt-test\n" + yaml.safe_dump(test_spec, sort_keys=False) + "```\n"

# 3) Append the test to the first existing doc
target = next((p for p in DOCS if p.exists()), None)
if target is None:
    # fallback: create a small doc to hold the test
    target = LAB / r"notes\ewb_auto_test.md"
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text("# EWB Auto Test Holder\n", encoding="utf-8")

txt = target.read_text(encoding="utf-8", errors="ignore")
# avoid duplicating the same test
if "```cnt-test" not in txt or test_spec["name"] not in txt:
    target.write_text(txt + "\n\n> Auto-added test\n" + fenced, encoding="utf-8")
    print("Appended test to:", target)
else:
    print("Test already present in:", target)

# 4) Run a cycle and show results
res = run_cycle()
import json
print("\n=== Engine result ===")
print(json.dumps(res, indent=2))

# Show latest tests report CSV
from glob import glob
out_dir = LAB / r"artifacts\cnt_engine_megacell\out"
reports = sorted(glob(str(out_dir / "tests_report_*.csv")))
if reports:
    rep = Path(reports[-1])
    print("\nLatest test report:", rep)
    display(pd.read_csv(rep).tail(10))
else:
    print("No tests_report_* yet.")


Found CSVs: 17
 - E:\CNT\notebooks\archive\cnt_ewb_theta2\runs\ewb_cooling_segments_20251017-235217.csv cols: ['segment', 'n', 'w', 'auc', 'precision@theta*', 'lead@hit(steps)'] …
 - E:\CNT\notebooks\archive\cnt_ewb_theta2\runs\ewb_holdout_results.csv cols: ['segment', 'path', 'sha256', 'n', 'w', 'auc'] …
 - E:\CNT\notebooks\archive\cnt_ewb_theta2\runs\ewb_summary_20251017-233201.csv cols: ['dataset', 'kind', 'heuristic_event', 'n', 'window', 'theta*'] …
 - E:\CNT\notebooks\archive\cnt_ewb_theta2\runs\ewb_summary_v2_1_20251017-234015.csv cols: ['dataset', 'kind', 'heuristic_event', 'n', 'window', 'theta*'] …
 - E:\CNT\notebooks\archive\cnt_ewb_theta2\runs\ewb_summary_v2_2_20251017-234543.csv cols: ['dataset', 'kind', 'heuristic_event', 'n', 'window', 'theta*'] …

Using: E:\CNT\notebooks\archive\cnt_ewb_theta2\runs\ewb_cooling_segments_20251017-235217.csv column: auc
Appended test to: E:\CNT\notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md

=== Engine result ===


,doc,name,type,passed,info
0,E:\CNT\notebooks\archive\cnt_ewb_theta2\ewb_re...,test_ewb_results,csv_threshold,0,missing_path
1,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,test_ewb_results,csv_threshold,0,missing_path
2,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,test_ewb_results,csv_threshold,0,missing_path
3,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,test_ewb_precision,csv_threshold,0,missing_column
4,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,missing_path
5,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,missing_path


In [18]:
# ================================
# CNT Engine — ONE MEGA CELL (SSD)
# Root: E:\CNT  (override via CNT_LAB_DIR)
# ================================

import os, json, glob, re, time
from datetime import datetime, timezone
from pathlib import Path, PureWindowsPath

import numpy as np
import pandas as pd
import networkx as nx
import yaml

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

# ---------- CONFIG ----------
LAB_ROOT  = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
SOURCE_ROOTS = [
    LAB_ROOT / "notes",
    LAB_ROOT / "artifacts" / "cnt_scroll",
    LAB_ROOT / "artifacts" / "cnt_codex",
    LAB_ROOT / "notebooks",
]
ENGINE_ROOT = LAB_ROOT / "artifacts" / "cnt_engine_megacell"
OUT_DIR     = ENGINE_ROOT / "out"
STATE_FILE  = ENGINE_ROOT / "CNT_STATE.yaml"
RUNLOG_FILE = ENGINE_ROOT / "runlog.jsonl"

PULSE_MIN       = 0   # 0 = run once; >0 = pulse every N minutes
AUTO_APPEND_EWB = True  # try to auto-append one valid EWB test if none are valid yet

for p in [ENGINE_ROOT, OUT_DIR, *SOURCE_ROOTS]:
    p.mkdir(parents=True, exist_ok=True)

def now(): return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---------- DISCOVER / SEED ----------
def list_sources():
    files = []
    for root in SOURCE_ROOTS:
        files += glob.glob(str(root / "**" / "*.md"), recursive=True)
        files += glob.glob(str(root / "**" / "*.txt"), recursive=True)
    return [Path(f) for f in files]

def auto_seed_if_empty():
    paths = list_sources()
    if paths: return
    seed = SOURCE_ROOTS[0] / "cnt_seed.md"
    seed.parent.mkdir(parents=True, exist_ok=True)
    seed.write_text(
        "# CNT Seed\n\n"
        "Hypothesis: Certain glyph–field pairings lower entropy drift.\n"
        "Test: Re-run θ metrics on EEG segments with glyph overlay vs baseline.\n"
        "Expected: Δθ > 0 with CI > 95% for overlay.\n"
        "Falsifier: No uplift or negative drift after 1k permutations.\n",
        encoding="utf-8"
    )

# ---------- INDEX & NOVELTY ----------
def build_index(df):
    if df.empty: return None, None, None
    vec = TfidfVectorizer(max_features=8000, ngram_range=(1,2))
    X = vec.fit_transform(df["text"]).astype(np.float32)
    n_samples, n_features = X.shape
    nmax = max(2, min(n_samples - 1, n_features - 1, 256))
    svd = TruncatedSVD(n_components=nmax, random_state=42)
    Xd = svd.fit_transform(X)
    # anomaly
    iso = IsolationForest(n_estimators=200, contamination="auto", random_state=42)
    iso.fit(Xd)
    iso_score = -iso.score_samples(Xd)  # larger = more novel
    # recon residual
    Xr = svd.inverse_transform(Xd)
    resid = np.linalg.norm(X.toarray() - Xr, axis=1)
    resid_norm = (resid - resid.min()) / (np.ptp(resid) + 1e-9)
    iso_norm   = iso_score / (float(np.max(iso_score)) + 1e-9)
    novel = 0.6*iso_norm + 0.4*resid_norm
    return vec, X, novel

# ---------- GRAPH (damped to avoid coherence=1.0) ----------
def build_graph(df, lbl):
    G = nx.Graph()
    for i, row in df.reset_index().iterrows():
        G.add_node(i, path=row["path"], title=os.path.basename(row["path"]),
                   cluster=int(lbl[i]) if lbl is not None else -1)
    pattern = re.compile(r"\b([A-Z][A-Za-z0-9_]{4,})\b")
    caps_by_row = [set(pattern.findall(t if isinstance(t, str) else "")) for t in df["text"].tolist()]
    N = len(df)
    for i in range(N):
        si = set(list(caps_by_row[i])[:30])
        for j in range(i+1, N):
            sj = set(list(caps_by_row[j])[:30])
            overlap = len(si & sj)
            same_cluster = (lbl is not None and lbl[i] == lbl[j])
            if same_cluster or overlap >= 4:
                G.add_edge(i, j, w=(1 + overlap))
    return G

def cluster_labels(X, k=6):
    if X is None: return None
    k = max(2, min(k, X.shape[0]-1))
    km = KMeans(n_clusters=k, n_init=5, random_state=42)
    return km.fit_predict(X.toarray())

# ---------- SCORE ----------
def score_reflexive(df, novel, G, runnable_tests=0):
    if df is None or len(df)==0:
        return dict(clarity=0, novelty=0, coherence=0, falsifiability=0, total=0)
    clarity = 1.0 - np.mean([min(len(t),10000)/10000 for t in df["text"].tolist()])
    novelty = float(np.mean(novel)/(np.std(novel)+1e-6))
    coherence = nx.average_clustering(G) if G.number_of_nodes()>1 else 0.0
    # proxy + runnable bonus
    fals_proxy = float(np.mean([t.lower().count("test")+t.lower().count("predict") for t in df["text"]]))/10.0
    falsifiability = np.clip(fals_proxy + min(0.5, 0.05*max(0, runnable_tests)), 0, 1)
    clarity = float(np.clip(clarity,0,1))
    novelty = float(np.clip(novelty/1.5,0,1))
    coherence = float(np.clip(coherence,0,1))
    total = float(np.mean([clarity,novelty,coherence,falsifiability]))
    return dict(clarity=clarity, novelty=novelty, coherence=coherence, falsifiability=falsifiability, total=total)

# ---------- HIDDEN TRUTHS ----------
def surface_candidates(df, novel, top=5):
    idx = np.argsort(novel)[::-1][:min(top, len(novel))]
    picks=[]
    for i in idx:
        snippet=re.sub(r"\s+"," ",df.iloc[i]["text"])[:400]
        picks.append(dict(path=df.iloc[i]["path"],resid=float(novel[i]),hint=snippet))
    return picks

# ---------- TESTS (YAML fenced) ----------
TEST_FENCE = re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL)

def _parse_cnt_tests(text: str):
    tests = []
    for m in TEST_FENCE.finditer(text or ""):
        body = m.group("body")
        try:
            t = yaml.safe_load(body)
            if isinstance(t, dict) and "type" in t:
                tests.append(t)
        except Exception:
            pass
    return tests

def _normalize_key(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def _match_column(df: pd.DataFrame, requested: str):
    want = _normalize_key(requested or "")
    if not want: return None
    norm_map = {_normalize_key(c): c for c in df.columns}
    return norm_map.get(want)

def _csv_threshold_test(t: dict):
    """
    returns (ok, info, reason) ; reason in {"ok","missing_path","missing_column","no_numeric","invalid_type"}
    """
    path = t.get("path")
    col  = t.get("column")
    op   = str(t.get("op", ">"))
    try:    val = float(t.get("value", 0.0))
    except: val = 0.0
    agg  = str(t.get("agg", "mean")).lower()

    if not path or "<fill_me" in str(path) or not Path(path).exists():
        return False, "missing_path", "missing_path"
    df = pd.read_csv(path)
    col_actual = _match_column(df, col)
    if col_actual is None:
        return False, "missing_column", "missing_column"
    s = pd.to_numeric(df[col_actual], errors="coerce").dropna()
    if s.empty:
        return False, "no_numeric", "no_numeric"

    if   agg == "mean":   metric = float(s.mean())
    elif agg == "median": metric = float(s.median())
    elif agg == "any":    metric = bool((s > val).any()) if op in [">", ">="] else bool((s < val).any())
    elif agg == "all":    metric = bool((s > val).all()) if op in [">", ">="] else bool((s < val).all())
    else:                 metric = float(s.mean())

    def _cmp(a,b,op): return {">":a>b, ">=":a>=b, "<":a<b, "<=":a<=b}.get(op, a>b)

    if isinstance(metric, bool):
        ok = metric; info = f"{agg}({col_actual})→{ok}"
    else:
        ok = _cmp(metric, val, op); info = f"{agg}({col_actual})={metric:.6f} {op} {val}"
    return bool(ok), info, "ok"

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    passed = total = invalid = 0
    runnable = 0
    for _, r in df.iterrows():
        path = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        runnable += len(tests)
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            if reason == "ok":
                total += 1
                if ok: passed += 1
            else:
                invalid += 1
            rows.append(dict(doc=path, name=name, type=ttype, valid=int(reason=="ok"),
                             passed=int(ok), info=info, reason=reason))
    out_df = pd.DataFrame(rows)
    if not out_df.empty:
        out_df.to_csv(out_csv, index=False)
    return passed, total, runnable

# ---------- PROPOSALS (write YAML tests) ----------
def _yaml_block(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False) + "```\n"

def _make_test_for(target_path: str):
    stem = Path(target_path).stem.replace(" ", "_").lower()
    return dict(
        name=f"test_{stem}",
        type="csv_threshold",
        path="<fill_me_csv_path>",
        column="<metric_column>",
        op=">",
        value=0.0,
        agg="mean"
    )

def propose_updates(cands):
    updates = []
    for c in cands:
        gloss = (
            f"\n\n> CNT-Gloss ({now()}): Clarify hypothesis; add test recipe & falsifier.\n"
            "- Hypothesis: …\n- Measurement: …\n- Expected shift: …\n- Falsifier: …\n\n"
        )
        test = _make_test_for(c["path"])
        content = gloss + _yaml_block(test)
        updates.append(dict(target=c["path"], content=content))
    return updates

def legality_gate(text): 
    bad=any(k in text for k in["ssn","credit card","weapon","harm"])
    return not bad

def confab_gate(text):
    t=text.lower(); return ("hypothesis" in t and "falsifier" in t)

def apply_updates(updates):
    accepted=[]
    for u in updates:
        try:
            if not (legality_gate(u["content"]) and confab_gate(u["content"])): 
                continue
            p=Path(u["target"])
            p.write_text(p.read_text(encoding="utf-8",errors="ignore")+u["content"],encoding="utf-8")
            accepted.append(u)
        except Exception:
            pass
    return accepted

def write_state(score, meta):
    try:
        STATE_FILE.write_text(yaml.safe_dump(dict(updated=now(),score=score,meta=meta),sort_keys=False), encoding="utf-8")
    except Exception:
        STATE_FILE.write_text(json.dumps(dict(updated=now(),score=score,meta=meta), indent=2), encoding="utf-8")

def log_event(kind, payload):
    with RUNLOG_FILE.open("a", encoding="utf-8") as f:
        f.write(json.dumps(dict(ts=now(), kind=kind, **payload)) + "\n")

# ---------- AUTO-APPEND ONE VALID EWB TEST (optional) ----------
def _auto_append_ewb_test_if_needed():
    # Find candidate EWB CSV
    base = LAB_ROOT / r"notebooks\archive\cnt_ewb_theta2"
    if not base.exists(): return False
    cands = []
    for p in base.rglob("*.csv"):
        try:
            cols = pd.read_csv(p, nrows=1).columns.tolist()
            cands.append((p, cols))
        except Exception:
            pass
    if not cands: return False
    # pick with AUC/precision if present
    chosen = None; chosen_col = None
    wants = ["auc","precision@theta*","precision_at_theta","delta_theta"]
    for p, cols in cands:
        nm = {_normalize_key(c): c for c in cols}
        for w in wants:
            if _normalize_key(w) in nm:
                chosen, chosen_col = p, nm[_normalize_key(w)]; break
        if chosen: break
    if not chosen:
        chosen, cols = cands[0]; chosen_col = cols[0]
    # target doc
    doc = LAB_ROOT / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
    doc.parent.mkdir(parents=True, exist_ok=True)
    txt = doc.read_text(encoding="utf-8", errors="ignore") if doc.exists() else "# EWB Results\n"
    if "test_ewb_auto" in txt: return False
    # pick threshold = 90% of mean to likely pass
    df = pd.read_csv(chosen)
    s = pd.to_numeric(df[chosen_col], errors="coerce").dropna()
    if s.empty: return False
    thr = float(round(0.9*float(s.mean()), 6))
    spec = dict(name="test_ewb_auto", type="csv_threshold", path=str(PureWindowsPath(chosen)),
                column=chosen_col, op=">", value=thr, agg="mean")
    block = "```cnt-test\n" + yaml.safe_dump(spec, sort_keys=False) + "```"
    doc.write_text(txt + "\n\n> Auto-added EWB test\n" + block + "\n", encoding="utf-8")
    return True

# ---------- CYCLE ----------
def run_cycle():
    auto_seed_if_empty()
    paths = list_sources()
    df = pd.DataFrame([dict(path=str(p), text=p.read_text(encoding="utf-8", errors="ignore")) for p in paths])
    if df.empty:
        log_event("empty", {"roots":[str(r) for r in SOURCE_ROOTS]})
        return {"empty": True, "roots":[str(r) for r in SOURCE_ROOTS]}

    # Optional: try to append a valid EWB test if none valid yet
    if AUTO_APPEND_EWB:
        _auto_append_ewb_test_if_needed()

    vec, X, novel = build_index(df)
    labels = cluster_labels(X)
    G = build_graph(df, labels)

    tests_csv = OUT_DIR / f"tests_report_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv"
    passed, total, runnable = run_cnt_tests_on_df(df, tests_csv)

    score = score_reflexive(df, novel, G, runnable_tests=runnable)
    if total > 0:
        frac = passed / max(1, total)
        score["falsifiability"] = float(np.clip(score["falsifiability"] + min(0.5, 0.25*frac), 0, 1))
        score["total"] = float(np.mean([score[k] for k in ["clarity","novelty","coherence","falsifiability"]]))

    cands = surface_candidates(df, novel, top=5)
    accepted = apply_updates(propose_updates(cands))

    write_state(score, dict(docs=len(df), accepted=len(accepted), tests_total=total, tests_passed=passed))
    pd.DataFrame(cands).to_csv(OUT_DIR / f"hidden_truths_{datetime.now().strftime('%Y%m%d-%H%M%S')}.csv", index=False)
    log_event("cycle", dict(score=score, proposed=len(cands), accepted=len(accepted), tests_total=total, tests_passed=passed))
    return dict(score=score, proposed=len(cands), accepted=len(accepted),
                tests_total=total, tests_passed=passed, hidden=cands[:3])

def run_pulsed(pulse_minutes: int):
    assert pulse_minutes > 0
    print(f"[CNT Engine] Pulsing every {pulse_minutes} min — CTRL+C to stop.")
    while True:
        res = run_cycle()
        print(json.dumps(res, indent=2))
        time.sleep(pulse_minutes*60)

# ---------- EXECUTE ----------
if PULSE_MIN and PULSE_MIN > 0:
    run_pulsed(PULSE_MIN)
else:
    res = run_cycle()
    print(json.dumps(res, indent=2))
    # show latest tests report if exists
    try:
        from glob import glob
        reps = sorted(glob(str(OUT_DIR / "tests_report_*.csv")))
        if reps:
            rep = reps[-1]
            print("\nLatest test report:", rep)
            df_rep = pd.read_csv(rep)
            print("Reasons:\n", df_rep["reason"].value_counts(dropna=False))
            try:
                from IPython.display import display
                display(df_rep.tail(10))
            except Exception:
                print(df_rep.tail(10).to_string(index=False))
        else:
            print("\nNo tests_report_* yet.")
    except Exception as e:
        print("Report check error:", e)


{
  "score": {
    "clarity": 0.8163487804878049,
    "novelty": 1.0,
    "coherence": 0.8836725961244575,
    "falsifiability": 0.9036585365853659,
    "total": 0.9009199782994071
  },
  "proposed": 5,
  "accepted": 5,
  "tests_total": 1,
  "tests_passed": 1,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.982124482187421,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9750494189181739,
      "hint": "== CNT Correlates Audit == Ti

,doc,name,type,valid,passed,info,reason
1,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,test_ewb_results,csv_threshold,0,0,missing_path,missing_path
2,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,test_ewb_results,csv_threshold,0,0,missing_path,missing_path
3,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,test_ewb_precision,csv_threshold,1,1,mean(AUC)=0.604440 > 0.55,ok
4,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,missing_path,missing_path
5,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,missing_path,missing_path
6,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,missing_path,missing_path
7,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,missing_path,missing_path
8,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,missing_path,missing_path
9,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,missing_path,missing_path
10,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,missing_path,missing_path


In [19]:
# === CNT test auto-heal (single cell) ===
from pathlib import Path, PureWindowsPath
import re, yaml, pandas as pd, numpy as np, os, json

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()

# reuse engine globals if present; else define
try:
    SOURCE_ROOTS
except NameError:
    SOURCE_ROOTS = [
        LAB / "notes",
        LAB / "artifacts" / "cnt_scroll",
        LAB / "artifacts" / "cnt_codex",
        LAB / "notebooks",
    ]

TEST_FENCE = re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL)

def _norm(s): return re.sub(r"[^a-z0-9]+","", str(s).lower())

# 1) index all CSVs + their columns (normalized)
csv_index = []  # list of (path, {norm_col->actual})
for root in [LAB]:
    for p in root.rglob("*.csv"):
        try:
            cols = pd.read_csv(p, nrows=1).columns.tolist()
            csv_index.append((p, {_norm(c): c for c in cols}))
        except Exception:
            pass

def _find_csv_for_column(norm_col: str, near: Path|None=None):
    # pick a csv that has this column; prefer same subtree as 'near'
    best = None; best_score = -1
    for p, cmap in csv_index:
        if norm_col in cmap:
            score = 0
            if near:
                # score by common prefix depth with doc folder
                a, b = near.resolve().parts, p.resolve().parts
                k = 0
                for x,y in zip(a,b):
                    if x==y: k+=1
                    else: break
                score = k
            if score > best_score:
                best = (p, cmap[norm_col])
                best_score = score
    return best  # (csv_path, actual_col) or None

# 2) heuristic thresholds by column semantics
def suggest_rule(col_name: str, series: pd.Series):
    c = _norm(col_name)
    x = pd.to_numeric(series, errors="coerce").dropna()
    if x.empty:
        return dict(op=">", value=0.0, agg="mean")
    mean = float(x.mean()); med = float(x.median())
    if "auc" in c or "precision" in c:
        # classification metrics in [0,1]
        thr = round(max(0.0, min(0.95, 0.9*mean)), 6)
        return dict(op=">", value=thr, agg="mean")
    if "delta" in c or "uplift" in c or "theta" in c:
        # positive shift desired
        thr = round(max(0.0, 0.25*mean), 6)
        return dict(op=">", value=thr, agg="mean")
    if "lead" in c:
        # want positive lead time
        thr = 0.0
        return dict(op=">", value=thr, agg="median")
    # fallback: prove it's not degenerate
    thr = round(0.9*mean, 6)
    return dict(op=">", value=thr, agg="mean")

# 3) scan docs, repair cnt-test blocks with missing path/column, write back
updated_docs = 0
fixed_blocks = 0
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        changes = []
        for m in TEST_FENCE.finditer(txt or ""):
            body = m.group("body")
            try:
                spec = yaml.safe_load(body) or {}
            except Exception:
                continue
            if not isinstance(spec, dict) or spec.get("type") != "csv_threshold":
                continue

            path_ok = spec.get("path") and "<fill_me" not in str(spec["path"]) and Path(str(spec["path"])).exists()
            col_ok  = bool(spec.get("column"))

            if path_ok and col_ok:
                continue  # already valid

            # choose desired column
            want = _norm(spec.get("column",""))
            # if no requested column, pick desirable ones
            candidates = ["auc","precision@theta*","precision_at_theta","deltatheta","lead@hit","leadathit","theta*","theta"]
            if not want:
                want = None
                for w in candidates:
                    want = _norm(w)
                    break

            # try to find a CSV close-by with matching column
            found = None
            if want:
                found = _find_csv_for_column(want, near=doc.parent)

            if not found:
                # last resort: just pick a CSV and first numeric column
                if csv_index:
                    cp, cmap = csv_index[0]
                    # select some numeric column
                    df_head = pd.read_csv(cp, nrows=50)
                    num_cols = [c for c in df_head.columns if pd.to_numeric(df_head[c], errors="coerce").notna().any()]
                    if num_cols:
                        found = (cp, num_cols[0])

            if not found:
                continue  # can't heal this block

            csv_path, actual_col = found
            df_full = pd.read_csv(csv_path)
            rule = suggest_rule(actual_col, df_full[actual_col])

            spec["path"] = str(PureWindowsPath(csv_path))
            spec["column"] = actual_col
            spec["op"] = rule["op"]
            spec["value"] = rule["value"]
            spec["agg"] = rule["agg"]

            # re-fence
            fenced = "```cnt-test\n" + yaml.safe_dump(spec, sort_keys=False) + "```"
            changes.append((m.span()[0], m.span()[1], fenced))

        if changes:
            # apply in reverse order (to keep indices stable)
            new_txt = txt
            for a,b,rep in reversed(changes):
                new_txt = new_txt[:a] + rep + new_txt[b:]
            doc.write_text(new_txt, encoding="utf-8")
            updated_docs += 1
            fixed_blocks += len(changes)

print(f"[heal] updated_docs={updated_docs}, fixed_blocks={fixed_blocks}")

# 4) optional: stop adding *new* draft tests every cycle (idempotent proposals)
try:
    old_fn = propose_updates
    def propose_updates(cands):
        ups=[]
        for c in cands[:3]:  # limit sprawl
            p=Path(c["path"]); t=p.read_text(encoding="utf-8", errors="ignore")
            if "```cnt-test" in t: 
                continue  # skip if tests already present
            # otherwise use the original proposal behavior
            gloss = (
                f"\n\n> CNT-Gloss ({datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')}): "
                "Clarify hypothesis; add test recipe & falsifier.\n"
                "- Hypothesis: …\n- Measurement: …\n- Expected shift: …\n- Falsifier: …\n\n"
            )
            test = dict(name=f"test_{Path(c['path']).stem.lower()}", type="csv_threshold",
                        path="<fill_me_csv_path>", column="<metric_column>", op=">", value=0.0, agg="mean")
            ups.append(dict(target=c["path"], content=gloss + "```cnt-test\n" + yaml.safe_dump(test, sort_keys=False) + "```"))
        return ups
except NameError:
    pass

# 5) rerun engine once and show result
try:
    res = run_cycle()
    print(json.dumps(res, indent=2))
    # show latest report summary
    from glob import glob
    out_dir = LAB / r"artifacts\cnt_engine_megacell\out"
    reps = sorted(glob(str(out_dir/"tests_report_*.csv")))
    if reps:
        rep = Path(reps[-1])
        dfrep = pd.read_csv(rep)
        print("\nLatest:", rep)
        print(dfrep["reason"].value_counts(dropna=False))
        try:
            from IPython.display import display
            display(dfrep.tail(10))
        except Exception:
            print(dfrep.tail(10).to_string(index=False))
except Exception as e:
    print("run_cycle not found here. Execute your mega cell first, then re-run this fixer.", e)


[heal] updated_docs=12, fixed_blocks=15
run_cycle not found here. Execute your mega cell first, then re-run this fixer. 'function' object has no attribute 'glob'


In [20]:
# ===== CNT Engine — finalize & verify (single cell) =====
import os, sys, json, importlib.util
from pathlib import Path
import pandas as pd
import glob as globmod  # << use module, not "from glob import glob"

# Point to SSD lab
os.environ["CNT_LAB_DIR"] = r"E:\CNT"
LAB = Path(os.environ["CNT_LAB_DIR"])
OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"

# Get run_cycle from current notebook OR from the SSD script if needed
if "run_cycle" not in globals():
    mod_path = LAB / "cnt_engine_megacell.py"
    if not mod_path.exists():
        raise RuntimeError("run_cycle not found. Run your Mega Cell once, or save it as E:\\CNT\\cnt_engine_megacell.py.")
    spec = importlib.util.spec_from_file_location("cnt_engine", str(mod_path))
    cnt_engine = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(cnt_engine)
    run_cycle = cnt_engine.run_cycle  # pull into this cell's globals

# Run one engine cycle
res = run_cycle()
print(json.dumps(res, indent=2))

# Show the latest tests report (valid/missing breakdown)
reports = sorted(globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reports:
    rep = reports[-1]
    print("\nLatest report:", rep)
    df = pd.read_csv(rep)
    print("\nReasons:\n", df["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(df.tail(10))
    except Exception:
        print(df.tail(10).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


AttributeError: 'function' object has no attribute 'glob'

In [21]:
# === Fix 'function object has no attribute glob' + verify run ===
import json, pandas as pd
from pathlib import Path
import glob as _globmod

# 1) Restore 'glob' to the module (was shadowed by 'from glob import glob')
glob = _globmod

# 2) Make list_sources() robust forever (uses a local module import)
try:
    SOURCE_ROOTS
except NameError:
    raise RuntimeError("Mega cell not loaded in this kernel yet. Run it once, then re-run this patch.")

def list_sources():
    import glob as __glob      # local, guaranteed module
    files = []
    for root in SOURCE_ROOTS:
        files += __glob.glob(str(root / "**" / "*.md"), recursive=True)
        files += __glob.glob(str(root / "**" / "*.txt"), recursive=True)
    return [Path(f) for f in files]

# 3) Run a cycle and print result + latest tests report
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1])
    df = pd.read_csv(rep)
    print("\nLatest report:", rep)
    print(df["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(df.tail(10))
    except Exception:
        print(df.tail(10).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


{
  "score": {
    "clarity": 0.8111512195121952,
    "novelty": 1.0,
    "coherence": 0.92317951142802,
    "falsifiability": 0.947560975609756,
    "total": 0.9204729266374929
  },
  "proposed": 5,
  "accepted": 1,
  "tests_total": 17,
  "tests_passed": 17,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_anomaly\\out\\gold_verification_summary.md",
      "resid": 0.9642741465358113,
      "hint": "# CNT Techno-Anomaly \u2014 Gold Verification - RA=209.236537, Dec=-1.289745: old_votes=4, new_votes_at_source=nan, W1\u2212W2=0.349, W2\u2212W3=3.768, SIMBAD=\u2713 () - RA=210.910946, Dec=-1.291592: old_votes=4, new_votes_at_source=nan, W1\u2212W2=-0.068, W2\u2212W3=2.526, SIMBAD=\u2014 () > CNT-Gloss (2025-10-26T21:37:58Z): Clarify hypothesis; add test recipe & falsifier. - Hypothesis: \u2026 - Measurement: \u2026 - Expected shift: \u2026 - Fa"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_one_signal_20251017-222741\\summary.txt",
      "resid": 0.91367

,doc,name,type,valid,passed,info,reason
7,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
8,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
9,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
10,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
11,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
12,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
13,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
14,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
15,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
16,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok


In [22]:
# === CNT GOLD FREEZE + STRICT MODE (single cell) ===
from pathlib import Path, PureWindowsPath
import os, re, yaml, pandas as pd, json, hashlib, glob as _globmod
from datetime import datetime, timezone

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# 0) Require mega cell objects
try:
    TEST_FENCE, SOURCE_ROOTS
except NameError:
    raise RuntimeError("Run your Mega Cell once, then run this GOLD cell.")

# 1) Helpers
def _file_sha256(p: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        while True:
            b = f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

def _norm(s): return re.sub(r"[^a-z0-9]+","", str(s).lower())

# 2) Promote all currently valid tests to GOLD (adds hash/rows/frozen_at)
updated_docs = 0
golded = 0
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        changes = []
        for m in TEST_FENCE.finditer(txt or ""):
            body = m.group("body")
            try:
                spec = yaml.safe_load(body) or {}
            except Exception:
                continue
            if not isinstance(spec, dict) or spec.get("type") != "csv_threshold":
                continue
            # already gold? keep but update metadata if missing
            p = spec.get("path")
            if not p or "<fill_me" in str(p): 
                continue
            pth = Path(p)
            if not pth.exists():
                continue
            col = spec.get("column")
            try:
                df = pd.read_csv(pth)
            except Exception:
                continue
            # confirm column (case-insensitive)
            cmap = {_norm(c): c for c in df.columns}
            col_actual = cmap.get(_norm(col or ""), None)
            if col_actual is None:
                continue
            # compute metadata
            h = _file_sha256(pth)
            nrows = int(len(df))
            spec["column"]    = col_actual
            spec["gold"]      = True
            spec["frozen_at"] = STAMP
            spec["hash"]      = h
            spec["rows"]      = nrows
            # re-fence with GOLD spec
            fenced = "```cnt-test\n" + yaml.safe_dump(spec, sort_keys=False) + "```"
            changes.append((m.span()[0], m.span()[1], fenced))
            golded += 1
        if changes:
            new_txt = txt
            for a,b,rep in reversed(changes):
                new_txt = new_txt[:a] + rep + new_txt[b:]
            doc.write_text(new_txt, encoding="utf-8")
            updated_docs += 1

print(f"[gold] updated_docs={updated_docs}, gold_tests={golded}")

# 3) Patch the runner: count only GOLD tests, verify hash (STRICT)
STRICT_GOLD_ONLY = True

def _csv_threshold_test_strict(t: dict):
    # Hash + shape check for GOLD, else reject if strict
    if STRICT_GOLD_ONLY and not t.get("gold", False):
        return False, "non_gold", "non_gold"
    path = t.get("path")
    if not path or not Path(path).exists():
        return False, "missing_path", "missing_path"
    if STRICT_GOLD_ONLY and t.get("hash"):
        h_now = _file_sha256(Path(path))
        if str(h_now) != str(t["hash"]):
            return False, f"hash_mismatch({h_now[:8]}!= {str(t['hash'])[:8]})", "hash_mismatch"
    # delegate to normalized test
    ok, info, reason = _csv_threshold_test(t)
    return ok, info, reason

# swap the runner to strict
def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    passed = total = invalid = 0
    runnable = 0
    for _, r in df.iterrows():
        path = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        runnable += len(tests)
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                # strict path
                ok, info, reason = _csv_threshold_test_strict(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            if reason == "ok":
                total += 1
                if ok: passed += 1
            else:
                invalid += 1
            rows.append(dict(doc=path, name=name, type=ttype, valid=int(reason=="ok"),
                             passed=int(ok), info=info, reason=reason))
    out_df = pd.DataFrame(rows)
    if not out_df.empty:
        out_df.to_csv(out_csv, index=False)
    return passed, total, runnable

# 4) Stop new test sprawl entirely (idempotent proposals)
def propose_updates(cands):
    ups=[]
    for c in []:  # never add new stubs automatically in strict mode
        pass
    return ups

# 5) Re-run engine once and show report
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT_DIR/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1])
    dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep.tail(10))
    except Exception:
        print(dfrep.tail(10).to_string(index=False))


[gold] updated_docs=12, gold_tests=17
{
  "score": {
    "clarity": 0.8052682926829269,
    "novelty": 1.0,
    "coherence": 0.9235444214640771,
    "falsifiability": 0.9548780487804878,
    "total": 0.920922690731873
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 17,
  "tests_passed": 17,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9508920059741233,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.9270007351984821,
      "hint": "# Gauge-Res

,doc,name,type,valid,passed,info,reason
8,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
9,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
10,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
11,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
12,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
13,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
14,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
15,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
16,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
17,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok


In [23]:
# === GOLD HOLDOUT TWINS + MANIFEST + RE-RUN (single cell) ===
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, hashlib, json, glob as _globmod

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
ART = LAB / r"artifacts\cnt_engine_megacell"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

# require mega-cell globals
try:
    TEST_FENCE, SOURCE_ROOTS
except NameError:
    raise RuntimeError("Run the Mega Cell once, then run this holdout+manifest cell.")

def _sha256(p: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s): return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def _match_col(df: pd.DataFrame, want: str):
    nm = {_norm(c): c for c in df.columns}
    return nm.get(_norm(want or ""), None)

# 1) create holdout twins when we can find a sensible companion CSV
updated_docs = 0; twins_added = 0
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        changes = []
        for m in TEST_FENCE.finditer(txt or ""):
            body = m.group("body")
            try:
                spec = yaml.safe_load(body) or {}
            except Exception:
                continue
            if not isinstance(spec, dict) or spec.get("type") != "csv_threshold":
                continue
            if not spec.get("gold"):  # only gold tests get holdout twins
                continue
            # skip if twin already exists
            name = str(spec.get("name",""))
            if name.endswith("_holdout"): 
                continue

            path = spec.get("path")
            if not path: 
                continue
            pth = Path(path)
            base_dir = pth.parent

            # hunt likely holdouts
            cand = None
            # heuristic 1: sibling with 'holdout' in name
            sibs = list(base_dir.glob("*holdout*.csv"))
            if sibs:
                cand = sibs[0]
            # heuristic 2: known EWB holdout file
            if not cand:
                known = base_dir / "ewb_holdout_results.csv"
                if known.exists():
                    cand = known
            # heuristic 3: any csv one level up with 'holdout'
            if not cand:
                ups = list(base_dir.parent.glob("*holdout*.csv"))
                if ups:
                    cand = ups[0]
            if not cand or not cand.exists():
                continue

            # build twin spec
            try:
                df = pd.read_csv(cand)
            except Exception:
                continue
            col_actual = _match_col(df, spec.get("column"))
            if col_actual is None:
                # take first numeric
                num_cols = [c for c in df.columns if pd.to_numeric(df[c], errors="coerce").notna().any()]
                if not num_cols:
                    continue
                col_actual = num_cols[0]

            # copy thresholds/ops; re-freeze with new hash/rows
            twin = dict(spec)
            twin["name"]     = f"{name}_holdout" if name else f"test_{cand.stem}_holdout"
            twin["path"]     = str(PureWindowsPath(cand))
            twin["column"]   = col_actual
            twin["frozen_at"]= STAMP
            twin["hash"]     = _sha256(cand)
            twin["rows"]     = int(len(df))
            twin["gold"]     = True

            fenced = "```cnt-test\n" + yaml.safe_dump(twin, sort_keys=False) + "```"
            # insert twin immediately after original block
            changes.append((m.span()[1], m.span()[1], "\n\n# GOLD Holdout Twin\n" + fenced))
            twins_added += 1

        if changes:
            new_txt = txt
            for a,b,rep in reversed(changes):
                new_txt = new_txt[:a] + rep + new_txt[b:]
            doc.write_text(new_txt, encoding="utf-8")
            updated_docs += 1

print(f"[holdout] updated_docs={updated_docs}, twins_added={twins_added}")

# 2) build a Gold Manifest from all gold tests across the corpus
rows = []
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        for m in TEST_FENCE.finditer(txt or ""):
            try:
                spec = yaml.safe_load(m.group("body")) or {}
            except Exception:
                continue
            if not isinstance(spec, dict) or not spec.get("gold"):
                continue
            rows.append(dict(
                doc=str(doc),
                name=str(spec.get("name","")),
                type=str(spec.get("type","")),
                path=str(spec.get("path","")),
                column=str(spec.get("column","")),
                op=str(spec.get("op",">")),
                value=float(spec.get("value",0.0)) if isinstance(spec.get("value",0.0),(int,float)) or str(spec.get("value","")).replace('.','',1).isdigit() else str(spec.get("value","")),
                agg=str(spec.get("agg","mean")),
                frozen_at=str(spec.get("frozen_at","")),
                hash=str(spec.get("hash","")),
                rows=int(spec.get("rows",0)),
                is_holdout=int(str(spec.get("name","")).endswith("_holdout"))
            ))

dfm = pd.DataFrame(rows)
ART.mkdir(parents=True, exist_ok=True)
if not dfm.empty:
    dfm.to_csv(MANIFEST_CSV, index=False, encoding="utf-8")
    with open(MANIFEST_YAML, "w", encoding="utf-8") as f:
        yaml.safe_dump(rows, f, sort_keys=False, allow_unicode=True)
print(f"[manifest] rows={len(rows)}  -> {MANIFEST_CSV.name}, {MANIFEST_YAML.name}")

# 3) run engine once (strict mode from your GOLD cell stays active)
res = run_cycle()
print(json.dumps(res, indent=2))

# 4) echo latest test report summary
reps = sorted(_globmod.glob(str((ART/"out") / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep.tail(12))
    except Exception:
        print(dfrep.tail(12).to_string(index=False))


[holdout] updated_docs=1, twins_added=2
[manifest] rows=19  -> gold_manifest.csv, gold_manifest.yaml
{
  "score": {
    "clarity": 0.8036512195121951,
    "novelty": 1.0,
    "coherence": 0.9235444214640771,
    "falsifiability": 0.9383183568677792,
    "total": 0.9163784994610128
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 19,
  "tests_passed": 17,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_3i_atlas_all8_20251024-042112Z_539b05ee\\README.txt",
      "resid": 0.9685376805735942,
      "hint": " CNT \u00d7 3I/ATLAS \u2014 Upgraded All\u20118 Pack ==================================== Stamp: 20251024-042112Z Includes: - WCS wavelength from FITS headers (CRVAL/CDELT/CRPIX) if spectrum_*.fits present. - Multi\u2011feature GRA (CN/CH/C\u2082 + custom lines) with solar\u2011analog continuum normalization. - Plasma+Phase: coherence with FDR correction, plus phase (from xcorr peak mapped to f). - Observer\u2011Ring Pro: if observ"
    },
    {
      "path": 

,doc,name,type,valid,passed,info,reason
8,E:\CNT\notebooks\archive\gra-v0.3\README.md,test_readme,csv_threshold,0,0,non_gold,non_gold
9,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
10,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
11,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
12,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
13,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
14,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
15,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
16,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok
17,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,1,1,mean(N_pairs)=30.000000 > 27.0,ok


In [24]:
# === Pair-Rule Runner Patch (base passes only if its _holdout twin passes) ===
from pathlib import Path
import pandas as pd
import re, json, glob as _globmod

# We reuse your existing strict helpers:
#   _csv_threshold_test_strict, _parse_cnt_tests, TEST_FENCE, ENGINE_ROOT
# Assumes GOLD strict mode already set by your prior cell.

def _is_holdout_name(name: str) -> bool:
    return str(name).endswith("_holdout")

def _base_name(name: str) -> str:
    return str(name)[:-9] if _is_holdout_name(name) else str(name)

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    """
    1) Execute strict GOLD tests (hash-verified).
    2) Apply pair rule: a base test is only valid if its _holdout twin is present AND passes.
       - If twin missing or fails, mark both as pair_fail (valid=0, passed=0).
    3) Write updated report and return counts after gating.
    """
    rows = []
    # ---- phase 1: strict evaluation (as before) ----
    for _, r in df.iterrows():
        path = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name))
            ))

    rep = pd.DataFrame(rows)

    if not rep.empty:
        # ---- phase 2: pair gating ----
        # Group by base name (strip _holdout)
        rep["base"] = rep["name"].apply(_base_name)

        # Only consider rows that reached 'ok' in strict mode
        ok_mask = rep["reason"].eq("ok")
        rep_ok = rep[ok_mask].copy()

        # For each base, find base-row and holdout-row
        # We'll mark pair_fail if either side missing OR any side fails
        idx_map = {}
        for i, row in rep_ok.iterrows():
            b = row["base"]
            idx_map.setdefault(b, []).append(i)

        to_pair_fail = set()
        for b, idxs in idx_map.items():
            # Find base and holdout rows among idxs (by name)
            base_idxs = [i for i in idxs if not _is_holdout_name(rep.loc[i, "name"])]
            hold_idxs = [i for i in idxs if _is_holdout_name(rep.loc[i, "name"])]

            # If there is at least one base test, require at least one holdout twin
            if base_idxs:
                if not hold_idxs:
                    # No twin: all base tests under 'b' fail the pair rule
                    to_pair_fail.update(base_idxs)
                else:
                    # Require that at least one holdout twin passes
                    hold_pass = any(bool(rep.loc[i, "passed"]) for i in hold_idxs)
                    base_pass = any(bool(rep.loc[i, "passed"]) for i in base_idxs)
                    if not (hold_pass and base_pass):
                        # If either side has no passing test, fail both sides
                        to_pair_fail.update(base_idxs + hold_idxs)

        # Apply pair_fail to report
        if to_pair_fail:
            rep.loc[list(to_pair_fail), "valid"]  = 0
            rep.loc[list(to_pair_fail), "passed"] = 0
            rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

        # ---- phase 3: output and counts ----
        rep.to_csv(out_csv, index=False)

        total  = int(rep["valid"].sum())
        passed = int(rep["passed"].sum())
        # Count "runnable" as number of tests seen, regardless of validity
        runnable = int(len(rep))
    else:
        # empty report
        pd.DataFrame(columns=["doc","name","type","valid","passed","info","reason","is_holdout","base"]).to_csv(out_csv, index=False)
        total = passed = runnable = 0

    return passed, total, runnable

# Re-run one engine cycle with pair rule enforced
res = run_cycle()
print(json.dumps(res, indent=2))

# Show latest report summary
from pathlib import Path
OUT_DIR = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1])
    dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep.tail(12))
    except Exception:
        print(dfrep.tail(12).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


{
  "score": {
    "clarity": 0.8036512195121951,
    "novelty": 1.0,
    "coherence": 0.9235444214640771,
    "falsifiability": 0.7146341463414634,
    "total": 0.8604574468294339
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 2,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_3i_atlas_all8_20251024-042112Z_539b05ee\\README.txt",
      "resid": 0.9685376805735942,
      "hint": " CNT \u00d7 3I/ATLAS \u2014 Upgraded All\u20118 Pack ==================================== Stamp: 20251024-042112Z Includes: - WCS wavelength from FITS headers (CRVAL/CDELT/CRPIX) if spectrum_*.fits present. - Multi\u2011feature GRA (CN/CH/C\u2082 + custom lines) with solar\u2011analog continuum normalization. - Plasma+Phase: coherence with FDR correction, plus phase (from xcorr peak mapped to f). - Observer\u2011Ring Pro: if observ"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.967949011575

,doc,name,type,valid,passed,info,reason,is_holdout,base
8,E:\CNT\notebooks\archive\gra-v0.3\README.md,test_readme,csv_threshold,0,0,non_gold,non_gold,0,test_readme
9,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_cnt_correlates_report_20251015-163558
10,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_cnt_correlates_report_20251015-163558
11,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_cnt_correlates_report_20251015-164130
12,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_cnt_correlates_report_20251015-164130
13,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_cnt_correlates_report_20251015-164130
14,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_readme
15,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_readme
16,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_summary
17,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,pair_fail,0,test_summary


In [25]:
# === PAIRED-EVIDENCE (PRAGMATIC MODE) RUNNER + RE-RUN (single cell) ===
# Counts only base/holdout pairs that BOTH exist and pass. Singles are "needs_holdout" (not failed, not counted).
from pathlib import Path
import pandas as pd, json, re, glob as _globmod

PAIR_MODE = "soft"  # "soft" = pragmatic / "strict" = fail singles

def _is_holdout_name(name: str) -> bool:
    return str(name).endswith("_holdout")

def _base_name(name: str) -> str:
    return str(name)[:-9] if _is_holdout_name(name) else str(name)

# We reuse: _csv_threshold_test_strict, _parse_cnt_tests, TEST_FENCE, ENGINE_ROOT, run_cycle
# (loaded from your mega cell)

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    # ---- Phase 1: strict evaluation (hash-verified GOLD) ----
    for _, r in df.iterrows():
        path = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name))
            ))

    rep = pd.DataFrame(rows)
    if rep.empty:
        pd.DataFrame(columns=["doc","name","type","valid","passed","info","reason","is_holdout","base"]).to_csv(out_csv, index=False)
        return 0, 0, 0

    # ---- Phase 2: pair gating in PAIR_MODE ----
    rep["base"] = rep["name"].apply(_base_name)

    # Start from strict-ok only
    rep.loc[~rep["reason"].eq("ok"), "valid"] = 0
    rep.loc[~rep["reason"].eq("ok"), "passed"] = 0

    # Index by base
    by_base = {b: idxs.index.tolist() for b, idxs in rep.groupby("base").groups.items()}

    to_pair_fail = set()
    to_needs_holdout = set()

    for b, idxs in by_base.items():
        base_idxs   = [i for i in idxs if not _is_holdout_name(rep.loc[i, "name"])]
        hold_idxs   = [i for i in idxs if _is_holdout_name(rep.loc[i, "name"])]
        has_base_ok = any(rep.loc[i, "reason"]=="ok" for i in base_idxs)
        has_hold_ok = any(rep.loc[i, "reason"]=="ok" for i in hold_idxs)
        base_pass   = any(bool(rep.loc[i, "passed"]) for i in base_idxs)
        hold_pass   = any(bool(rep.loc[i, "passed"]) for i in hold_idxs)

        if PAIR_MODE == "strict":
            # must have both and both pass
            if not (has_base_ok and has_hold_ok and base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)
        else:
            # soft: if twin missing, mark needs_holdout (exclude from totals, not fail)
            if base_idxs and not hold_idxs:
                to_needs_holdout.update(base_idxs)
            elif hold_idxs and not base_idxs:
                to_needs_holdout.update(hold_idxs)
            else:
                # both exist → require both pass
                if not (base_pass and hold_pass):
                    to_pair_fail.update(base_idxs + hold_idxs)

    # Apply labels
    if PAIR_MODE == "strict":
        if to_pair_fail:
            rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
            rep.loc[list(to_pair_fail), "reason"] = "pair_fail"
    else:
        if to_needs_holdout:
            rep.loc[list(to_needs_holdout), ["valid","passed"]] = 0
            rep.loc[list(to_needs_holdout), "reason"] = "needs_holdout"
        if to_pair_fail:
            rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
            rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    # Write report
    rep.to_csv(out_csv, index=False)

    # Totals: count only valid rows (i.e., true paired evidence) 
    total  = int(rep["valid"].sum())
    passed = int(rep["passed"].sum())
    runnable = int(len(rep))  # number seen, regardless of gating
    return passed, total, runnable

# ---- Re-run one engine cycle and show summary
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep.tail(12))
    except Exception:
        print(dfrep.tail(12).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


AttributeError: 'Index' object has no attribute 'index'

In [26]:
# === Patch: pragmatic paired-evidence runner (fixed group indices) ===
from pathlib import Path
import pandas as pd, json, glob as _globmod

# keep your earlier helpers & globals: _csv_threshold_test_strict, _parse_cnt_tests,
# TEST_FENCE, ENGINE_ROOT, run_cycle, _is_holdout_name, _base_name

PAIR_MODE = "soft"  # "soft" = singles -> needs_holdout (not failed); "strict" = fail singles

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    # 1) strict GOLD evaluation (hash-verified)
    for _, r in df.iterrows():
        path  = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name))
            ))

    rep = pd.DataFrame(rows)
    if rep.empty:
        pd.DataFrame(columns=["doc","name","type","valid","passed","info","reason","is_holdout","base"]).to_csv(out_csv, index=False)
        return 0, 0, 0

    # 2) pair gating (soft/strict)
    rep["base"] = rep["name"].apply(_base_name)

    # only rows that were strict-ok can be considered
    rep.loc[~rep["reason"].eq("ok"), ["valid","passed"]] = 0

    # FIX: build base -> list of integer row indices (use .indices, not .groups.index)
    by_base = {b: list(idxs) for b, idxs in rep.groupby("base").indices.items()}

    to_pair_fail = set()
    to_needs_holdout = set()

    for b, idxs in by_base.items():
        base_idxs = [i for i in idxs if not _is_holdout_name(rep.loc[i, "name"])]
        hold_idxs = [i for i in idxs if     _is_holdout_name(rep.loc[i, "name"])]

        has_base_ok = any(rep.loc[i, "reason"]=="ok" for i in base_idxs)
        has_hold_ok = any(rep.loc[i, "reason"]=="ok" for i in hold_idxs)
        base_pass   = any(bool(rep.loc[i, "passed"]) for i in base_idxs)
        hold_pass   = any(bool(rep.loc[i, "passed"]) for i in hold_idxs)

        if PAIR_MODE == "strict":
            if not (has_base_ok and has_hold_ok and base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)
        else:
            if base_idxs and not hold_idxs:
                to_needs_holdout.update(base_idxs)
            elif hold_idxs and not base_idxs:
                to_needs_holdout.update(hold_idxs)
            else:
                if not (base_pass and hold_pass):
                    to_pair_fail.update(base_idxs + hold_idxs)

    if PAIR_MODE == "strict":
        if to_pair_fail:
            rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
            rep.loc[list(to_pair_fail), "reason"] = "pair_fail"
    else:
        if to_needs_holdout:
            rep.loc[list(to_needs_holdout), ["valid","passed"]] = 0
            rep.loc[list(to_needs_holdout), "reason"] = "needs_holdout"
        if to_pair_fail:
            rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
            rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    rep.to_csv(out_csv, index=False)

    total    = int(rep["valid"].sum())   # counts only true paired evidence
    passed   = int(rep["passed"].sum())  # pairs where both sides pass
    runnable = int(len(rep))             # all tests seen (for info)
    return passed, total, runnable

# re-run once and show summary
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep.tail(12))
    except Exception:
        print(dfrep.tail(12).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


{
  "score": {
    "clarity": 0.8036512195121951,
    "novelty": 1.0,
    "coherence": 0.9235444214640771,
    "falsifiability": 0.7146341463414634,
    "total": 0.8604574468294339
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_3i_atlas_all8_20251024-042112Z_539b05ee\\README.txt",
      "resid": 0.9685376805735942,
      "hint": " CNT \u00d7 3I/ATLAS \u2014 Upgraded All\u20118 Pack ==================================== Stamp: 20251024-042112Z Includes: - WCS wavelength from FITS headers (CRVAL/CDELT/CRPIX) if spectrum_*.fits present. - Multi\u2011feature GRA (CN/CH/C\u2082 + custom lines) with solar\u2011analog continuum normalization. - Plasma+Phase: coherence with FDR correction, plus phase (from xcorr peak mapped to f). - Observer\u2011Ring Pro: if observ"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.967949011575

,doc,name,type,valid,passed,info,reason,is_holdout,base
8,E:\CNT\notebooks\archive\gra-v0.3\README.md,test_readme,csv_threshold,0,0,non_gold,needs_holdout,0,test_readme
9,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-163558
10,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-163558
11,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-164130
12,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-164130
13,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-164130
14,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_readme
15,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_readme
16,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_summary
17,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_summary


In [27]:
# === Auto-mint HOLDOUT twins (same column) + freeze + re-run (single cell) ===
from pathlib import Path, PureWindowsPath
import os, re, yaml, pandas as pd, hashlib, json, glob as _globmod
from datetime import datetime, timezone

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# require from mega cell: TEST_FENCE, SOURCE_ROOTS, run_cycle
try:
    TEST_FENCE, SOURCE_ROOTS, run_cycle
except NameError:
    raise RuntimeError("Run the Mega Cell cell first so TEST_FENCE/SOURCE_ROOTS/run_cycle exist.")

def _sha256(p: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s).lower())

def _match_col(df: pd.DataFrame, want: str):
    nm = {_norm(c): c for c in df.columns}
    return nm.get(_norm(want or ""))

def _find_holdout_csv(base_csv: Path):
    """Search same dir, parent, and siblings for a plausible holdout CSV."""
    d = base_csv.parent
    # 1) siblings with 'holdout' in name
    sibs = [p for p in d.glob("*.csv") if "holdout" in p.name.lower()]
    if sibs: return sibs[0]
    # 2) known EWB filename
    k = d / "ewb_holdout_results.csv"
    if k.exists(): return k
    # 3) parent dir any '*holdout*.csv'
    ups = [p for p in d.parent.glob("*.csv") if "holdout" in p.name.lower()]
    if ups: return ups[0]
    return None

minted = 0
touched_docs = 0

for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue

        changes = []
        for m in TEST_FENCE.finditer(txt or ""):
            body = m.group("body")
            try:
                spec = yaml.safe_load(body) or {}
            except Exception:
                continue
            # only extend GOLD base tests (not already _holdout)
            name = str(spec.get("name",""))
            if not (isinstance(spec, dict) and spec.get("type")=="csv_threshold" and spec.get("gold", False)):
                continue
            if name.endswith("_holdout"):
                continue

            base_path = spec.get("path")
            base_col  = spec.get("column")
            if not base_path or not base_col:
                continue
            base_csv = Path(base_path)
            if not base_csv.exists():
                continue

            # find a holdout candidate
            ho = _find_holdout_csv(base_csv)
            if not ho or not ho.exists():
                continue

            # column must exist in holdout csv
            try:
                dfh = pd.read_csv(ho)
            except Exception:
                continue
            col_actual = _match_col(dfh, base_col)
            if col_actual is None:
                continue  # do not mint if metric not present

            spec_twin = dict(spec)
            spec_twin["name"]      = name + "_holdout"
            spec_twin["path"]      = str(PureWindowsPath(ho))
            spec_twin["column"]    = col_actual
            spec_twin["frozen_at"] = STAMP
            spec_twin["hash"]      = _sha256(ho)
            spec_twin["rows"]      = int(len(dfh))
            spec_twin["gold"]      = True

            fenced = "```cnt-test\n" + yaml.safe_dump(spec_twin, sort_keys=False) + "```"
            # insert twin immediately AFTER the base block
            changes.append((m.span()[1], m.span()[1], "\n\n# GOLD Holdout Twin (auto)\n" + fenced))
            minted += 1

        if changes:
            new_txt = txt
            for a,b,rep in reversed(changes):
                new_txt = new_txt[:a] + rep + new_txt[b:]
            doc.write_text(new_txt, encoding="utf-8")
            touched_docs += 1

print(f"[holdout-mint] docs_touched={touched_docs}, twins_minted={minted}")

# re-run once
res = run_cycle()
print(json.dumps(res, indent=2))

# show latest report breakdown
OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep.tail(12))
    except Exception:
        print(dfrep.tail(12).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


[holdout-mint] docs_touched=1, twins_minted=2
{
  "score": {
    "clarity": 0.802,
    "novelty": 1.0,
    "coherence": 0.9235444214640771,
    "falsifiability": 0.724390243902439,
    "total": 0.8624836663416291
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.909040400111031,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_ewb_theta2\\publish\\ewb_publish_v0_2\\ewb_results.md",
      "resid": 0.90117898635803

,doc,name,type,valid,passed,info,reason,is_holdout,base
10,E:\CNT\notebooks\archive\gra-v0.3\README.md,test_readme,csv_threshold,0,0,non_gold,needs_holdout,0,test_readme
11,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-163558
12,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-163558
13,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-164130
14,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-164130
15,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-164130,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_cnt_correlates_report_20251015-164130
16,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_readme
17,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,test_readme,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_readme
18,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_summary
19,E:\CNT\notebooks\archive\cnt_one_signal_202510...,test_summary,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,test_summary


In [28]:
# === CNT Pair-Doctor: mint a correct EWB base+holdout pair and re-run (single cell) ===
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, hashlib, json, glob as _globmod

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# need from mega cell: TEST_FENCE, SOURCE_ROOTS, run_cycle, _csv_threshold_test_strict
try:
    TEST_FENCE, SOURCE_ROOTS, run_cycle, _csv_threshold_test_strict
except NameError:
    raise RuntimeError("Run your Mega Cell first so TEST_FENCE/SOURCE_ROOTS/run_cycle exist.")

def _sha256(p: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s).lower())

def _match_col(df: pd.DataFrame, want: str):
    nm = {_norm(c): c for c in df.columns}
    return nm.get(_norm(want or ""))

def _base_name(name: str) -> str:
    return str(name)[:-9] if str(name).endswith("_holdout") else str(name)

# ---- 1) collect all tests from corpus
tests = []
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        for m in TEST_FENCE.finditer(txt or ""):
            body = m.group("body")
            try:
                spec = yaml.safe_load(body) or {}
            except Exception:
                continue
            if not isinstance(spec, dict): 
                continue
            name = str(spec.get("name", Path(doc).stem))
            tests.append(dict(
                doc=str(doc), name=name, base=_base_name(name),
                spec=spec, text_span=m.span(), text=txt
            ))

# ---- 2) find an EWB base test we can pair (prefer 'ewb' in doc or name)
ewb = [t for t in tests if ("ewb" in _norm(t["doc"]) or "ewb" in _norm(t["name"])) and t["spec"].get("type")=="csv_threshold"]
if not ewb:
    raise SystemExit("No EWB cnt-test found. Run the earlier auto-append or tell me which file to use.")
# choose a base that is NOT already a holdout and is GOLD + passes strict
chosen = None
for t in ewb:
    spec = t["spec"]
    if str(t["name"]).endswith("_holdout"): 
        continue
    if not spec.get("gold", False): 
        continue
    ok, info, reason = _csv_threshold_test_strict(spec)
    if reason == "ok" and ok:
        chosen = t; break
if not chosen:
    # fallback: take first EWB base; we'll still try to mint twin correctly
    chosen = ewb[0]

print("[pair-doctor] base:", chosen["name"], "doc:", chosen["doc"])

# ---- 3) compute the proper holdout CSV and column for the chosen base
base_spec = chosen["spec"]
base_csv  = Path(base_spec.get("path",""))
base_col  = base_spec.get("column","")
if (not base_csv.exists()) or (not base_col):
    raise SystemExit("Chosen base test has no valid CSV/column. Open the doc and fill them once, then re-run this cell.")

# likely holdout file in same dir: ewb_holdout_results.csv OR *holdout*.csv
def _find_holdout_csv(base_csv: Path):
    d = base_csv.parent
    k = d / "ewb_holdout_results.csv"
    if k.exists(): return k
    sibs = [p for p in d.glob("*holdout*.csv")]
    if sibs: return sibs[0]
    ups = [p for p in d.parent.glob("*holdout*.csv")]
    if ups: return ups[0]
    return None

hold_csv = _find_holdout_csv(base_csv)
if not hold_csv or not hold_csv.exists():
    raise SystemExit(f"No holdout CSV found near {base_csv}. Place an 'ewb_holdout_results.csv' next to it and re-run.")

dfh = pd.read_csv(hold_csv)
col_actual = _match_col(dfh, base_col)
if col_actual is None:
    # try common metrics if name variant differs
    for want in ["auc","precision@theta*","precision_at_theta","delta_theta"]:
        col_actual = _match_col(dfh, want)
        if col_actual: break
if col_actual is None:
    raise SystemExit(f"Holdout CSV {hold_csv} lacks a compatible column for '{base_col}'.")

# ---- 4) write or update the _holdout twin next to the base block
doc_path = Path(chosen["doc"])
txt = chosen["text"]
span = chosen["text_span"]

twin = dict(base_spec)
twin["name"]      = str(chosen["name"]) + "_holdout"
twin["path"]      = str(PureWindowsPath(hold_csv))
twin["column"]    = col_actual
twin["frozen_at"] = STAMP
twin["hash"]      = _sha256(hold_csv)
twin["rows"]      = int(len(dfh))
twin["gold"]      = True

twin_block = "```cnt-test\n" + yaml.safe_dump(twin, sort_keys=False) + "```"

# insert twin immediately after base block (or replace existing twin with fresh metadata)
# if an existing twin with same name exists in the doc, replace its fenced block
replaced = False
for m in TEST_FENCE.finditer(txt or ""):
    body = m.group("body")
    try:
        spec2 = yaml.safe_load(body) or {}
    except Exception:
        continue
    if isinstance(spec2, dict) and spec2.get("name","") == twin["name"]:
        a,b = m.span()
        txt = txt[:a] + twin_block + txt[b:]
        replaced = True
        break

if not replaced:
    # insert after base block span
    a,b = span
    txt = txt[:b] + "\n\n# GOLD Holdout Twin (doctor)\n" + twin_block + txt[b:]

doc_path.write_text(txt, encoding="utf-8")
print("[pair-doctor] wrote twin:", twin["name"], "->", hold_csv.name, "| col:", col_actual)

# ---- 5) re-run engine once and show the pair summary
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT_DIR/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    # quick base/twin view
    view = dfrep[["name","valid","passed","reason"]].copy()
    view["base"] = view["name"].apply(lambda s: s[:-9] if str(s).endswith("_holdout") else s)
    try:
        from IPython.display import display
        display(view.sort_values(["base","name"]).tail(20))
    except Exception:
        print(view.sort_values(["base","name"]).tail(20).to_string(index=False))
else:
    print("No tests_report_* yet.")


[pair-doctor] base: test_ewb_results doc: E:\CNT\notebooks\archive\cnt_ewb_theta2\ewb_results.md


SystemExit: No holdout CSV found near E:\CNT\artifacts\CNT_PLI_EC_EO_paired_bootstrap.csv. Place an 'ewb_holdout_results.csv' next to it and re-run.

C:\Users\caleb\CNT_Lab\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [29]:
# === EWB explicit holdout twin + re-run (single cell) ===
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, hashlib, json, glob as _globmod

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# need from mega cell:
try:
    TEST_FENCE, run_cycle
except NameError:
    raise RuntimeError("Run your Mega Cell first so TEST_FENCE/run_cycle exist.")

BASE_DOC   = LAB / r"notebooks\archive\cnt_ewb_theta2\ewb_results.md"  # where base test lives
HOLDOUT_CSV = LAB / r"notebooks\archive\cnt_ewb_theta2\runs\ewb_holdout_results.csv"

def _sha256(p: Path, chunk=1024*1024):
    import hashlib
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def _match_col(df: pd.DataFrame, want: str):
    nm = {_norm(c): c for c in df.columns}
    return nm.get(_norm(want or ""))

if not BASE_DOC.exists():
    raise SystemExit(f"Base doc not found: {BASE_DOC}")
if not HOLDOUT_CSV.exists():
    raise SystemExit(f"Holdout CSV not found: {HOLDOUT_CSV}")

txt = BASE_DOC.read_text(encoding="utf-8", errors="ignore")

# 1) find an EWB base cnt-test (not _holdout), GOLD, with a CSV+column
blocks = list(TEST_FENCE.finditer(txt))
chosen = None
for m in blocks:
    try:
        spec = yaml.safe_load(m.group("body")) or {}
    except Exception:
        continue
    if not isinstance(spec, dict): 
        continue
    name = str(spec.get("name",""))
    if name.endswith("_holdout"): 
        continue
    if spec.get("type")!="csv_threshold":
        continue
    # prefer something clearly EWB
    if "ewb" not in _norm(name) and "ewb" not in _norm(BASE_DOC.name):
        continue
    if not spec.get("gold", False):
        continue
    if not spec.get("path") or not spec.get("column"):
        continue
    # pick this one
    chosen = dict(spec=spec, span=m.span(), name=name)
    break

if chosen is None:
    raise SystemExit("No suitable EWB base cnt-test (GOLD) found in the base doc.")

# 2) choose the column in holdout to match the base (fallback to common metrics)
dfh = pd.read_csv(HOLDOUT_CSV)
col = _match_col(dfh, chosen["spec"]["column"])
if col is None:
    for want in ["auc","precision@theta*","precision_at_theta","delta_theta"]:
        col = _match_col(dfh, want)
        if col: break
if col is None:
    raise SystemExit(f"Holdout CSV lacks a compatible column; columns present: {list(dfh.columns)}")

# 3) compose the twin spec (reuse op/value/agg from the base)
twin = dict(chosen["spec"])
twin["name"]      = chosen["name"] + "_holdout"
twin["path"]      = str(PureWindowsPath(HOLDOUT_CSV))
twin["column"]    = col
twin["frozen_at"] = STAMP
twin["hash"]      = _sha256(HOLDOUT_CSV)
twin["rows"]      = int(len(dfh))
twin["gold"]      = True

twin_block = "```cnt-test\n" + yaml.safe_dump(twin, sort_keys=False) + "```"

# 4) insert or replace twin in the doc
inserted = False
for m in blocks:
    try:
        spec2 = yaml.safe_load(m.group("body")) or {}
    except Exception:
        continue
    if isinstance(spec2, dict) and spec2.get("name","") == twin["name"]:
        a,b = m.span()
        txt = txt[:a] + twin_block + txt[b:]
        inserted = True
        break

if not inserted:
    a,b = chosen["span"]
    txt = txt[:b] + "\n\n# GOLD Holdout Twin (explicit)\n" + twin_block + txt[b:]

BASE_DOC.write_text(txt, encoding="utf-8")
print("[pair-explicit] wrote twin:", twin["name"], "->", HOLDOUT_CSV.name, "| col:", col)

# 5) re-run the engine and show status
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT_DIR/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    view = dfrep[["name","valid","passed","reason"]].copy()
    view["base"] = view["name"].apply(lambda s: s[:-9] if str(s).endswith("_holdout") else s)
    try:
        from IPython.display import display
        display(view.sort_values(["base","name"]).tail(20))
    except Exception:
        print(view.sort_values(["base","name"]).tail(20).to_string(index=False))


[pair-explicit] wrote twin: test_ewb_results_holdout -> ewb_holdout_results.csv | col: AUC
{
  "score": {
    "clarity": 0.8011682926829268,
    "novelty": 1.0,
    "coherence": 0.9165754180460063,
    "falsifiability": 0.7292682926829268,
    "total": 0.861753000852965
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9553980941132267,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_ewb_theta2\\publish\\ewb_pub

,name,valid,passed,reason,base
15,test_cnt_correlates_report_20251015-164130,0,0,needs_holdout,test_cnt_correlates_report_20251015-164130
16,test_cnt_correlates_report_20251015-164130,0,0,needs_holdout,test_cnt_correlates_report_20251015-164130
9,test_ewb_auto_holdout,0,0,needs_holdout,test_ewb_aut
10,test_ewb_auto_holdout,0,0,needs_holdout,test_ewb_aut
8,test_ewb_auto,0,0,needs_holdout,test_ewb_auto
6,test_ewb_precision_holdout,0,0,needs_holdout,test_ewb_precisio
7,test_ewb_precision_holdout,0,0,needs_holdout,test_ewb_precisio
5,test_ewb_precision,0,0,needs_holdout,test_ewb_precision
2,test_ewb_results_holdout,0,0,needs_holdout,test_ewb_result
1,test_ewb_results,0,0,needs_holdout,test_ewb_results


In [30]:
# === Robust paired-evidence runner (normalized base keys) + re-run ===
from pathlib import Path
import pandas as pd, json, re, glob as _globmod

# Keep strict GOLD check + parser from the mega cell:
#   _csv_threshold_test_strict, _parse_cnt_tests, TEST_FENCE, ENGINE_ROOT, run_cycle

def _norm(s: str) -> str:
    # lower + strip + collapse non-alphanumerics
    return re.sub(r"[^a-z0-9]+", "", str(s or "").strip().lower())

def _is_holdout_name(name: str) -> bool:
    s = str(name or "").strip()
    return s.lower().endswith("_holdout")

def _base_key(name: str) -> str:
    s = str(name or "").strip()
    # remove trailing _holdout in a case-insensitive way
    if s.lower().endswith("_holdout"):
        s = s[: -len("_holdout")]
    return _norm(s)

PAIR_MODE = "soft"  # "soft" = singles -> needs_holdout (not failed)

def run_cnt_tests_on_df(df: pd.DataFrame, out_csv: Path):
    rows = []
    # 1) strict GOLD evaluation
    for _, r in df.iterrows():
        path  = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name)),
                base_key=_base_key(name)
            ))

    rep = pd.DataFrame(rows)
    if rep.empty:
        pd.DataFrame(columns=["doc","name","type","valid","passed","info","reason","is_holdout","base_key"]).to_csv(out_csv, index=False)
        return 0, 0, 0

    # Consider only strict-ok rows for pairing
    rep.loc[~rep["reason"].eq("ok"), ["valid","passed"]] = 0

    # Build base_key -> row indices
    by_key = {k: list(idxs) for k, idxs in rep.groupby("base_key").indices.items()}

    to_pair_fail = set()
    to_needs_holdout = set()

    for key, idxs in by_key.items():
        base_idxs = [i for i in idxs if rep.loc[i, "is_holdout"] == 0]
        hold_idxs = [i for i in idxs if rep.loc[i, "is_holdout"] == 1]

        has_base_ok = any(rep.loc[i, "reason"]=="ok" for i in base_idxs)
        has_hold_ok = any(rep.loc[i, "reason"]=="ok" for i in hold_idxs)
        base_pass   = any(bool(rep.loc[i, "passed"]) for i in base_idxs)
        hold_pass   = any(bool(rep.loc[i, "passed"]) for i in hold_idxs)

        if PAIR_MODE == "soft":
            if base_idxs and not hold_idxs:
                to_needs_holdout.update(base_idxs)
            elif hold_idxs and not base_idxs:
                to_needs_holdout.update(hold_idxs)
            else:
                if not (base_pass and hold_pass):
                    to_pair_fail.update(base_idxs + hold_idxs)
        else:  # strict
            if not (has_base_ok and has_hold_ok and base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)

    # Apply labels
    if to_needs_holdout:
        rep.loc[list(to_needs_holdout), ["valid","passed"]] = 0
        rep.loc[list(to_needs_holdout), "reason"] = "needs_holdout"
    if to_pair_fail:
        rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
        rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    rep.to_csv(out_csv, index=False)

    # Totals: only count true paired evidence
    # A row is counted if valid==1 (i.e., it's part of a base+holdout pair that passes)
    total    = int(rep["valid"].sum())
    passed   = int(rep["passed"].sum())
    runnable = int(len(rep))
    return passed, total, runnable

# Re-run one engine cycle and show summary
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT_DIR / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    # quick pair view
    view = dfrep[["name","valid","passed","reason","base_key"]].copy()
    try:
        from IPython.display import display
        display(view.sort_values(["base_key","name"]).tail(24))
    except Exception:
        print(view.sort_values(["base_key","name"]).tail(24).to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


{
  "score": {
    "clarity": 0.8011682926829268,
    "novelty": 1.0,
    "coherence": 0.9165754180460063,
    "falsifiability": 0.7292682926829268,
    "total": 0.861753000852965
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9553980941132267,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_ewb_theta2\\publish\\ewb_publish_v0_2\\ewb_results.md",
      "resid": 0.9123017500289156,
      "hint": "# CNT \u0398*

,name,valid,passed,reason,base_key
12,test_cnt_correlates_report_20251015-163558,0,0,needs_holdout,testcntcorrelatesreport20251015163558
13,test_cnt_correlates_report_20251015-163558,0,0,needs_holdout,testcntcorrelatesreport20251015163558
14,test_cnt_correlates_report_20251015-164130,0,0,needs_holdout,testcntcorrelatesreport20251015164130
15,test_cnt_correlates_report_20251015-164130,0,0,needs_holdout,testcntcorrelatesreport20251015164130
16,test_cnt_correlates_report_20251015-164130,0,0,needs_holdout,testcntcorrelatesreport20251015164130
8,test_ewb_auto,0,0,pair_fail,testewbauto
9,test_ewb_auto_holdout,0,0,pair_fail,testewbauto
10,test_ewb_auto_holdout,0,0,pair_fail,testewbauto
5,test_ewb_precision,0,0,pair_fail,testewbprecision
6,test_ewb_precision_holdout,0,0,pair_fail,testewbprecision


In [31]:
# === Pair Audit & Calibrate (base + holdout) ===
from pathlib import Path
import re, yaml, pandas as pd, json, glob as _globmod
from datetime import datetime, timezone

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---- POLICY ----
# Choose ONE: "strict_055" (AUC>0.55), "above_chance" (AUC>0.50), "relative_90pct" (value=0.9*mean for each file)
MODE  = "strict_055"
APPLY = True   # set False to just audit without rewriting thresholds

def _norm(s): return re.sub(r"[^a-z0-9]+","", str(s or "").lower().strip())
def _is_holdout(name): return str(name or "").lower().strip().endswith("_holdout")
def _base_key(name):
    s = str(name or "").strip()
    return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

def _get_pairs(TEST_FENCE, SOURCE_ROOTS):
    items=[]
    for root in SOURCE_ROOTS:
        for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
            try: txt = doc.read_text(encoding="utf-8", errors="ignore")
            except: continue
            for m in TEST_FENCE.finditer(txt or ""):
                try: spec = yaml.safe_load(m.group("body")) or {}
                except: continue
                if not isinstance(spec, dict) or spec.get("type")!="csv_threshold": continue
                nm = spec.get("name", doc.stem)
                items.append(dict(doc=str(doc), span=m.span(), text=txt, name=nm, spec=spec))
    # group by base key
    by={}
    for it in items:
        k=_base_key(it["name"])
        by.setdefault(k, []).append(it)
    # split into base/holdout
    pairs=[]
    for k, arr in by.items():
        base=[it for it in arr if not _is_holdout(it["name"])]
        hold=[it for it in arr if     _is_holdout(it["name"])]
        if base:
            pairs.append(dict(key=k, base=base, hold=hold))
    return pairs

def _pick_auc_column(df, wanted):
    nm = {_norm(c):c for c in df.columns}
    if wanted and _norm(wanted) in nm: return nm[_norm(wanted)]
    for w in ["auc","precision@theta*","precision_at_theta","delta_theta"]:
        if _norm(w) in nm: return nm[_norm(w)]
    # fallback: first numeric
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any(): return c
    return None

def _policy_value(mode, series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return None
    if mode=="strict_055":   return 0.55
    if mode=="above_chance": return 0.50
    if mode=="relative_90pct": return float(round(0.9*float(s.mean()), 6))
    return None

pairs = _get_pairs(TEST_FENCE, SOURCE_ROOTS)

rows=[]; rewrites=[]
for P in pairs:
    for b in P["base"]:
        spec_b = b["spec"]
        p_b = Path(spec_b.get("path",""))
        if not p_b.exists(): continue
        dfb = pd.read_csv(p_b)
        col_b = _pick_auc_column(dfb, spec_b.get("column"))
        if not col_b: continue
        mean_b = float(pd.to_numeric(dfb[col_b], errors="coerce").dropna().mean())

        # if no holdouts, annotate and continue
        if not P["hold"]:
            rows.append(dict(base=b["name"], hold=None, col_b=col_b, mean_b=mean_b, status="needs_holdout"))
            continue

        # choose the first compatible holdout
        h = None
        for cand in P["hold"]:
            spec_h = cand["spec"]; p_h = Path(spec_h.get("path",""))
            if not p_h.exists(): continue
            dfh = pd.read_csv(p_h)
            col_h = _pick_auc_column(dfh, spec_b.get("column"))
            if col_h: h=(cand, dfh, col_h); break
        if not h:
            rows.append(dict(base=b["name"], hold=None, col_b=col_b, mean_b=mean_b, status="holdout_missing_metric"))
            continue

        cand, dfh, col_h = h
        mean_h = float(pd.to_numeric(dfh[col_h], errors="coerce").dropna().mean())

        # compute thresholds by policy
        val_b = _policy_value(MODE, dfb[col_b])
        val_h = _policy_value(MODE, dfh[col_h])

        rows.append(dict(base=b["name"], hold=cand["name"], col_b=col_b, col_h=col_h,
                         mean_b=round(mean_b,6), mean_h=round(mean_h,6),
                         thr_b=val_b, thr_h=val_h, mode=MODE))

        if APPLY and val_b is not None and val_h is not None:
            # rewrite fenced blocks with updated values/op
            tb = b["text"]; sb=b["span"]
            th = cand["text"]; sh=cand["span"]
            spec_b2 = dict(spec_b); spec_b2["column"]=col_b; spec_b2["value"]=val_b; spec_b2["op"]=">"
            spec_h2 = dict(cand["spec"]); spec_h2["column"]=col_h; spec_h2["value"]=val_h; spec_h2["op"]=">"
            fb = "```cnt-test\n" + yaml.safe_dump(spec_b2, sort_keys=False) + "```"
            fh = "```cnt-test\n" + yaml.safe_dump(spec_h2, sort_keys=False) + "```"
            # apply to doc texts (replace spans; write back later)
            tb2 = tb[:sb[0]] + fb + tb[sb[1]:]
            th2 = th[:sh[0]] + fh + th[1:sh[1]] if False else th[:sh[0]] + fh + th[sh[1]:]
            Path(b["doc"]).write_text(tb2, encoding="utf-8")
            Path(cand["doc"]).write_text(th2, encoding="utf-8")
            rewrites.append((b["name"], cand["name"], col_b, col_h, val_b, val_h))

# show audit table
audit = pd.DataFrame(rows)
print("=== Pair Audit ===")
if not audit.empty:
    try:
        from IPython.display import display
        display(audit)
    except Exception:
        print(audit.to_string(index=False))
else:
    print("No pairs found.")

if APPLY and rewrites:
    print(f"\nRewrote thresholds for {len(rewrites)} base/holdout pairs under policy='{MODE}'.")

# re-run engine once
res = run_cycle()
print("\n=== Engine Result ===")
print(json.dumps(res, indent=2))

# show latest report
OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT_DIR/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep[["name","valid","passed","reason"]].tail(20))
    except Exception:
        print(dfrep.tail(20).to_string(index=False))


=== Pair Audit ===


,base,hold,col_b,mean_b,status
0,test_ewb_results,None,N_pairs,30.00000,needs_holdout
1,test_ewb_results,None,N_pairs,30.00000,needs_holdout
2,test_ewb_results,None,N_pairs,30.00000,needs_holdout
3,test_readme,None,N_pairs,30.00000,needs_holdout
4,test_readme,None,N_pairs,30.00000,needs_holdout
5,test_ewb_precision,None,AUC,0.60444,needs_holdout
6,test_ewb_auto,None,AUC,0.60444,needs_holdout
7,test_gold_verification_summary,None,N_pairs,30.00000,needs_holdout
8,test_cnt_correlates_report_20251015-163558,None,N_pairs,30.00000,needs_holdout
9,test_cnt_correlates_report_20251015-163558,None,N_pairs,30.00000,needs_holdout



=== Engine Result ===
{
  "score": {
    "clarity": 0.8011682926829268,
    "novelty": 1.0,
    "coherence": 0.9165754180460063,
    "falsifiability": 0.7292682926829268,
    "total": 0.861753000852965
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9553980941132267,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_ewb_theta2\\publish\\ewb_publish_v0_2\\ewb_results.md",
      "resid": 0.9123017500289156,
     

,name,valid,passed,reason
3,test_ewb_results,0,0,needs_holdout
4,test_ewb_results,0,0,needs_holdout
5,test_ewb_precision,0,0,needs_holdout
6,test_ewb_precision_holdout,0,0,needs_holdout
7,test_ewb_precision_holdout,0,0,needs_holdout
8,test_ewb_auto,0,0,needs_holdout
9,test_ewb_auto_holdout,0,0,needs_holdout
10,test_ewb_auto_holdout,0,0,needs_holdout
11,test_readme,0,0,needs_holdout
12,test_cnt_correlates_report_20251015-163558,0,0,needs_holdout


In [32]:
# === FORCE EWB BASE+HOLDOUT PAIR (GOLD, HASHED) + CALIBRATE + RE-RUN ===
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, hashlib, json, glob as _globmod

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# REQUIRE: TEST_FENCE and run_cycle from the mega cell already loaded
try:
    TEST_FENCE, run_cycle
except NameError:
    raise RuntimeError("Run your Mega Cell once (so TEST_FENCE/run_cycle exist), then re-run this cell.")

# ---- CONFIG (adjust if needed)
DOC     = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"  # keep both tests together
CSV_BASE = LAB / r"notebooks\archive\cnt_ewb_theta2\runs\ewb_cooling_segments_20251017-235217.csv"
CSV_HO   = LAB / r"notebooks\archive\cnt_ewb_theta2\runs\ewb_holdout_results.csv"
WANTED_METRIC = "AUC"   # we’ll auto-fallback if this column name varies

BASE_NAME = "test_ewb_pair"
HOLD_NAME = "test_ewb_pair_holdout"

def _sha256(p: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s).lower().strip())

def _pick_col(df: pd.DataFrame, wanted: str):
    nm = {_norm(c): c for c in df.columns}
    if _norm(wanted) in nm: return nm[_norm(wanted)]
    for alt in ["precision@theta*", "precision_at_theta", "auc", "delta_theta"]:
        if _norm(alt) in nm: return nm[_norm(alt)]
    # fallback: first numeric column
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any(): return c
    return None

# ---- load CSVs and choose metric column for BOTH files
if not CSV_BASE.exists():
    raise SystemExit(f"Base CSV not found: {CSV_BASE}")
if not CSV_HO.exists():
    raise SystemExit(f"Holdout CSV not found: {CSV_HO}")

dfb = pd.read_csv(CSV_BASE)
dfh = pd.read_csv(CSV_HO)

col_b = _pick_col(dfb, WANTED_METRIC)
col_h = _pick_col(dfh, col_b or WANTED_METRIC)
if not col_b or not col_h:
    raise SystemExit(f"Could not resolve a metric column. Base cols={list(dfb.columns)}, Holdout cols={list(dfh.columns)}")

# ---- set honest thresholds: 90% of each file’s mean on the chosen column
sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
if sb.empty or sh.empty:
    raise SystemExit("Chosen metric column(s) contain no numeric data.")
thr_b = float(round(0.9 * float(sb.mean()), 6))
thr_h = float(round(0.9 * float(sh.mean()), 6))

# ---- compose two GOLD cnt-test blocks (with hashes, rows, frozen_at)
spec_base = dict(
    name=BASE_NAME, type="csv_threshold",
    path=str(PureWindowsPath(CSV_BASE)), column=col_b, op=">", value=thr_b, agg="mean",
    gold=True, frozen_at=STAMP, hash=_sha256(CSV_BASE), rows=int(len(dfb))
)
spec_hold = dict(
    name=HOLD_NAME, type="csv_threshold",
    path=str(PureWindowsPath(CSV_HO)),   column=col_h, op=">", value=thr_h, agg="mean",
    gold=True, frozen_at=STAMP, hash=_sha256(CSV_HO), rows=int(len(dfh))
)

def _fence(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False) + "```"

DOC.parent.mkdir(parents=True, exist_ok=True)
txt = DOC.read_text(encoding="utf-8", errors="ignore") if DOC.exists() else "# EWB Results — Paired Evidence\n"

# upsert blocks idempotently by name
def upsert_block(text: str, name: str, block: str) -> str:
    out = text
    replaced = False
    for m in TEST_FENCE.finditer(text or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and str(spec.get("name","")).strip().lower() == name.lower():
            a,b = m.span()
            out = text[:a] + block + text[b:]
            replaced = True
            break
    if not replaced:
        out = (out.rstrip() + "\n\n" + block + "\n")
    return out

txt = upsert_block(txt, BASE_NAME, _fence(spec_base))
txt = upsert_block(txt, HOLD_NAME, _fence(spec_hold))
DOC.write_text(txt, encoding="utf-8")
print(f"[force-pair] wrote/updated in: {DOC.name}\n  base  : {BASE_NAME}  ({col_b}, thr={thr_b})\n  holdout: {HOLD_NAME} ({col_h}, thr={thr_h})")

# ---- (optional) ensure PAIR_MODE is soft and strict GOLD runner is active
# We assume your earlier patches set STRICT_GOLD_ONLY=True and the soft pair runner.
# If not, they can be re-applied; for now we just run.

# ---- re-run once and show the paired status
res = run_cycle()
print(json.dumps(res, indent=2))

OUT_DIR = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT_DIR/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    # show just our pair rows
    view = dfrep[dfrep["name"].str.contains(BASE_NAME, case=False, na=False) | dfrep["name"].str.contains(HOLD_NAME, case=False, na=False)]
    try:
        from IPython.display import display
        display(view[["name","valid","passed","reason","info"]])
    except Exception:
        print(view[["name","valid","passed","reason","info"]].to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


[force-pair] wrote/updated in: ewb_results.md
  base  : test_ewb_pair  (AUC, thr=0.543996)
  holdout: test_ewb_pair_holdout (AUC, thr=0.467013)
{
  "score": {
    "clarity": 0.7996292682926829,
    "novelty": 1.0,
    "coherence": 0.9165754180460063,
    "falsifiability": 0.7390243902439024,
    "total": 0.863807269145648
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9240678100081607,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\

,name,valid,passed,reason,info
11,test_ewb_pair,0,0,needs_holdout,mean(AUC)=0.604440 > 0.543996
12,test_ewb_pair_holdout,0,0,needs_holdout,mean(AUC)=0.518903 > 0.467013


In [33]:
# === Bind the robust pair runner INTO the module that run_cycle is using, then re-run ===
import sys, types, json, pandas as pd, re
from pathlib import Path
import glob as _globmod

# 0) Figure out where run_cycle lives
import inspect
rc_mod = inspect.getmodule(run_cycle)
print("[where]", rc_mod.__name__)

# 1) Robust helpers (normalized keys)
def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s or "").strip().lower())

def _is_holdout_name(name: str) -> bool:
    s = str(name or "").strip()
    return s.lower().endswith("_holdout")

def _base_key(name: str) -> str:
    s = str(name or "").strip()
    if s.lower().endswith("_holdout"):
        s = s[:-len("_holdout")]
    return _norm(s)

# 2) Robust paired-evidence runner (soft mode: singles -> needs_holdout; pairs must both pass)
def _paired_runner_soft(df: pd.DataFrame, out_csv: Path):
    rows = []
    for _, r in df.iterrows():
        path  = r["path"]
        tests = _parse_cnt_tests(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name)),
                base_key=_base_key(name)
            ))
    rep = pd.DataFrame(rows)
    if rep.empty:
        rep.to_csv(out_csv, index=False)
        return 0,0,0

    rep.loc[~rep["reason"].eq("ok"), ["valid","passed"]] = 0
    by_key = {k: list(idxs) for k, idxs in rep.groupby("base_key").indices.items()}

    to_pair_fail, to_needs = set(), set()
    for key, idxs in by_key.items():
        base_idxs = [i for i in idxs if rep.loc[i, "is_holdout"]==0]
        hold_idxs = [i for i in idxs if rep.loc[i, "is_holdout"]==1]
        base_pass = any(bool(rep.loc[i, "passed"]) for i in base_idxs)
        hold_pass = any(bool(rep.loc[i, "passed"]) for i in hold_idxs)

        if base_idxs and not hold_idxs:
            to_needs.update(base_idxs)
        elif hold_idxs and not base_idxs:
            to_needs.update(hold_idxs)
        else:
            if not (base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)

    if to_needs:
        rep.loc[list(to_needs), ["valid","passed"]] = 0
        rep.loc[list(to_needs), "reason"] = "needs_holdout"
    if to_pair_fail:
        rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
        rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    rep.to_csv(out_csv, index=False)
    total  = int(rep["valid"].sum())
    passed = int(rep["passed"].sum())
    runnable = int(len(rep))
    return passed, total, runnable

# 3) Monkey-patch the module that owns run_cycle
mod = rc_mod
# Make sure the module has the helpers the runner expects:
setattr(mod, "_is_holdout_name", _is_holdout_name)
setattr(mod, "_base_key", _base_key)
# STRICT gold should stay on
if not hasattr(mod, "STRICT_GOLD_ONLY"):
    mod.STRICT_GOLD_ONLY = True
# Use the strict tester already in the module namespace if present; else fall back to our global
if "_csv_threshold_test_strict" in globals() and not hasattr(mod, "_csv_threshold_test_strict"):
    setattr(mod, "_csv_threshold_test_strict", globals()["_csv_threshold_test_strict"])
if "_parse_cnt_tests" in globals() and not hasattr(mod, "_parse_cnt_tests"):
    setattr(mod, "_parse_cnt_tests", globals()["_parse_cnt_tests"])
if "TEST_FENCE" in globals() and not hasattr(mod, "TEST_FENCE"):
    setattr(mod, "TEST_FENCE", globals()["TEST_FENCE"])

# Patch the runner used by run_cycle
setattr(mod, "run_cnt_tests_on_df", _paired_runner_soft)

# 4) Re-run a cycle with the patched module function
res = run_cycle()
print(json.dumps(res, indent=2))

# 5) Show just our EWB pair rows to confirm they count as a pair
OUT_DIR = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT_DIR/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1])
    dfrep = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    view = dfrep[dfrep["name"].str.contains("test_ewb_pair", case=False, na=False)]
    try:
        from IPython.display import display
        display(view[["name","valid","passed","reason","info","base_key"]])
    except Exception:
        print(view[["name","valid","passed","reason","info","base_key"]].to_string(index=False))
else:
    print("\nNo tests_report_* yet.")


[where] __main__
{
  "score": {
    "clarity": 0.7996292682926829,
    "novelty": 1.0,
    "coherence": 0.9165754180460063,
    "falsifiability": 0.9890243902439024,
    "total": 0.926307269145648
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 2,
  "tests_passed": 2,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-163558.txt",
      "resid": 0.9240678100081607,
      "hint": "== CNT Correlates Audit == Timestamp: 20251015-163558 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add both fields to your CSVs. [GREY] EEG: L"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_ewb_theta2\\publish\\ewb_publish_v0_2\\ewb_results.md",
      "resid": 0.8990080476618345,
      "hint

,name,valid,passed,reason,info,base_key
11,test_ewb_pair,1,1,ok,mean(AUC)=0.604440 > 0.543996,testewbpair
12,test_ewb_pair_holdout,1,1,ok,mean(AUC)=0.518903 > 0.467013,testewbpair


In [34]:
# === EWB Pair Harvester (safe) + Re-run (single cell) ===
from pathlib import Path, PureWindowsPath
import pandas as pd, yaml, re, hashlib, glob as _globmod, json
from datetime import datetime, timezone

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
EWB = LAB / r"notebooks\archive\cnt_ewb_theta2"
DOC = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"

def _sha256(p: Path, chunk=1024*1024):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s): return re.sub(r"[^a-z0-9]+","", str(s).lower().strip())
def _pick_col(df, wanted):
    nm = {_norm(c):c for c in df.columns}
    for k in [wanted,"auc","precision@theta*","precision_at_theta","delta_theta"]:
        if k and _norm(k) in nm: return nm[_norm(k)]
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any(): return c
    return None

# find base csvs and a nearby holdout csv
bases = sorted(EWB.rglob("*.csv"))
holds = [p for p in bases if "holdout" in p.name.lower()]
hold_by_dir = {}
for h in holds:
    hold_by_dir.setdefault(h.parent, []).append(h)

wrote = 0
DOC.parent.mkdir(parents=True, exist_ok=True)
txt = DOC.read_text(encoding="utf-8", errors="ignore") if DOC.exists() else "# EWB Results — Paired Evidence\n"

for base in bases:
    if "holdout" in base.name.lower(): 
        continue
    # pick a holdout in same dir, else skip
    ho = None
    if base.parent in hold_by_dir:
        ho = hold_by_dir[base.parent][0]
    else:
        k = base.parent / "ewb_holdout_results.csv"
        if k.exists(): ho = k
    if not ho: 
        continue

    dfb = pd.read_csv(base)
    dfh = pd.read_csv(ho)
    # skip tables that don’t have meaningful metrics (e.g., pure counts)
    col_b = _pick_col(dfb, "auc")
    col_h = _pick_col(dfh, col_b)
    if not col_b or not col_h: 
        continue

    sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
    sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
    if sb.empty or sh.empty: 
        continue

    thr_b = float(round(0.9*float(sb.mean()), 6))
    thr_h = float(round(0.9*float(sh.mean()), 6))

    name_base = f"pair_{base.stem}"
    name_hold = f"{name_base}_holdout"

    spec_b = dict(name=name_base, type="csv_threshold", path=str(PureWindowsPath(base)),
                  column=col_b, op=">", value=thr_b, agg="mean",
                  gold=True, frozen_at=STAMP, hash=_sha256(base), rows=int(len(dfb)))
    spec_h = dict(name=name_hold, type="csv_threshold", path=str(PureWindowsPath(ho)),
                  column=col_h, op=">", value=thr_h, agg="mean",
                  gold=True, frozen_at=STAMP, hash=_sha256(ho), rows=int(len(dfh)))

    def _fence(d): return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False) + "```"
    # idempotent upsert
    def upsert(text, nm, block):
        replaced = False
        for m in TEST_FENCE.finditer(text or ""):
            try:
                s = yaml.safe_load(m.group("body")) or {}
            except Exception:
                continue
            if isinstance(s, dict) and str(s.get("name","")).lower()==nm.lower():
                a,b = m.span(); text = text[:a] + block + text[b:]; replaced=True; break
        if not replaced: text = text.rstrip()+"\n\n"+block+"\n"
        return text

    txt = upsert(txt, name_base, _fence(spec_b))
    txt = upsert(txt, name_hold, _fence(spec_h))
    wrote += 1

DOC.write_text(txt, encoding="utf-8")
print(f"[harvest] wrote/updated {wrote} base+holdout pairs into {DOC}")

# re-run once and print summary
res = run_cycle()
print(json.dumps(res, indent=2))

from glob import glob
reps = sorted(glob(str(OUT/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest report:", rep)
    print(dfrep["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(dfrep[dfrep["name"].str.contains("pair_", case=False, na=False)][["name","valid","passed","reason","info"]].tail(20))
    except:
        print(dfrep.tail(20).to_string(index=False))


[harvest] wrote/updated 5 base+holdout pairs into E:\CNT\notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md
{
  "score": {
    "clarity": 0.7945463414634146,
    "novelty": 1.0,
    "coherence": 0.9165754180460063,
    "falsifiability": 1.0,
    "total": 0.9277804398773553
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 12,
  "tests_passed": 12,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9557112452514159,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_o

NameError: name 'OUT' is not defined

In [35]:
# === Fix OUT path and show latest paired-evidence report ===
from pathlib import Path
import pandas as pd
import glob as globmod

# Resolve OUT even if ENGINE_ROOT isn't in scope
OUT = (Path(str(ENGINE_ROOT)) / "out") if "ENGINE_ROOT" in globals() else Path(r"E:\CNT\artifacts\cnt_engine_megacell\out")

rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
if not rep_paths:
    print("No tests_report_* found under", OUT)
else:
    rep = Path(rep_paths[-1])
    df  = pd.read_csv(rep)
    print("Latest report:", rep)
    print("\nReasons:\n", df["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(df.tail(20))
    except Exception:
        print(df.tail(20).to_string(index=False))


Latest report: E:\CNT\artifacts\cnt_engine_megacell\out\tests_report_20251026-183154.csv

Reasons:
 reason
needs_holdout    13
ok               12
pair_fail        10
Name: count, dtype: int64


,doc,name,type,valid,passed,info,reason,is_holdout,base_key
15,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_20251017-233201,csv_threshold,1,1,mean(AUC)=0.325391 > 0.292852,ok,0,pairewbsummary20251017233201
16,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_20251017-233201_holdout,csv_threshold,1,1,mean(AUC)=0.518903 > 0.467013,ok,1,pairewbsummary20251017233201
17,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_v2_1_20251017-234015,csv_threshold,1,1,mean(AUC)=0.234282 > 0.210854,ok,0,pairewbsummaryv2120251017234015
18,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_v2_1_20251017-234015_holdout,csv_threshold,1,1,mean(AUC)=0.518903 > 0.467013,ok,1,pairewbsummaryv2120251017234015
19,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_v2_2_20251017-234543,csv_threshold,1,1,mean(AUC)=0.418632 > 0.376769,ok,0,pairewbsummaryv2220251017234543
20,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_v2_2_20251017-234543_holdout,csv_threshold,1,1,mean(AUC)=0.518903 > 0.467013,ok,1,pairewbsummaryv2220251017234543
21,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_v2_3_20251017-234953,csv_threshold,1,1,mean(AUC)=0.458255 > 0.412429,ok,0,pairewbsummaryv2320251017234953
22,E:\CNT\notebooks\archive\cnt_ewb_theta2\publis...,pair_ewb_summary_v2_3_20251017-234953_holdout,csv_threshold,1,1,mean(AUC)=0.518903 > 0.467013,ok,1,pairewbsummaryv2320251017234953
23,E:\CNT\notebooks\archive\gra-v0.3\README.md,test_readme,csv_threshold,0,0,non_gold,needs_holdout,0,testreadme
24,E:\CNT\notebooks\archive\cnt_correlates_report...,test_cnt_correlates_report_20251015-163558,csv_threshold,0,0,mean(N_pairs)=30.000000 > 27.0,needs_holdout,0,testcntcorrelatesreport20251015163558


In [36]:
# === Quarantine placeholder tests (skip) + teach runner to ignore them ===
from pathlib import Path
import re, yaml, pandas as pd, json, glob as _globmod

LAB = Path(r"E:\CNT")
PLACEHOLDER_NAME_PAT = re.compile(r"^(test_readme|test_summary)$", re.IGNORECASE)
PLACEHOLDER_COL_KEYS = {"npairs","n_pairs","n-pairs"}

# require mega-cell globals
try:
    TEST_FENCE, SOURCE_ROOTS, run_cycle
except NameError:
    raise RuntimeError("Run your Mega Cell first, then this cell.")

# 1) mark placeholders with skip:true (idempotent)
updated = 0
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        changes = []
        for m in TEST_FENCE.finditer(txt or ""):
            body = m.group("body")
            try:
                spec = yaml.safe_load(body) or {}
            except Exception:
                continue
            if not isinstance(spec, dict) or spec.get("type") != "csv_threshold":
                continue
            name = str(spec.get("name",""))
            col  = str(spec.get("column",""))
            if PLACEHOLDER_NAME_PAT.match(name) or re.sub(r"[^a-z0-9]+","",col.lower()) in PLACEHOLDER_COL_KEYS:
                if not spec.get("skip", False):
                    spec["skip"] = True
                    fenced = "```cnt-test\n" + yaml.safe_dump(spec, sort_keys=False) + "```"
                    changes.append((m.span()[0], m.span()[1], fenced))
        if changes:
            new_txt = txt
            for a,b,rep in reversed(changes):
                new_txt = new_txt[:a] + rep + new_txt[b:]
            doc.write_text(new_txt, encoding="utf-8")
            updated += 1
print(f"[skip] updated_docs={updated}")

# 2) patch the strict tester to ignore skip:true
def _csv_threshold_test_strict(t: dict):
    if t.get("skip", False):
        return False, "skip", "skip"
    if not t.get("gold", False):
        return False, "non_gold", "non_gold"
    path = t.get("path")
    if not path or not Path(path).exists():
        return False, "missing_path", "missing_path"
    # hash check if provided
    if t.get("hash"):
        import hashlib
        h = hashlib.sha256()
        with open(path, "rb") as f:
            for b in iter(lambda: f.read(1<<20), b""):
                h.update(b)
        if h.hexdigest() != str(t["hash"]):
            return False, "hash_mismatch", "hash_mismatch"
    # evaluate via normalized tester
    ok, info, reason = _csv_threshold_test(t) if "_csv_threshold_test" in globals() else (False,"no_core_tester","invalid_type")
    return ok, info, reason

# 3) re-run and show a crisp summary
res = run_cycle()
print(json.dumps(res, indent=2))

OUT = LAB / r"artifacts\cnt_engine_megacell\out"
reps = sorted(_globmod.glob(str(OUT/"tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); df = pd.read_csv(rep)
    print("\nLatest:", rep)
    print(df["reason"].value_counts(dropna=False))
    try:
        from IPython.display import display
        display(df[df["reason"].isin(["ok","pair_fail","needs_holdout","skip"])][["name","valid","passed","reason","info"]].tail(20))
    except Exception:
        print(df.tail(20).to_string(index=False))


[skip] updated_docs=13
{
  "score": {
    "clarity": 0.7941439024390243,
    "novelty": 1.0,
    "coherence": 0.8991551091981508,
    "falsifiability": 1.0,
    "total": 0.9233247529092938
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 12,
  "tests_passed": 12,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_anomaly\\out\\CNT_Gold_Dossier_J135656-011722.md",
      "resid": 0.917654107784611,
      "hint": "# CNT Gold Dossier \u2014 WISE J135656.78\u2212011722.9 **Position (ICRS):** RA 209.236592, Dec -1.289716 ## AllWISE photometry & flags - W1=14.348999977111816, W2=14.0, W3=10.232000350952148, W4=8.480999946594238 - W1\u2212W2=0.349, W2\u2212W3=3.768 - ph_qual=, cc_flags=, ext_flg= - SNR: w1=37.4, w2=27.8 (W3/W4 from table if present) ## Catalog labels - SIMBAD: () - NED: WISEA J135656.78-011722.9 [G] z=0.148182 ## G"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_one_signal_20251017-215548\\summary.txt",
      "resid": 0.9175153485631594,
 

,name,valid,passed,reason,info
15,pair_ewb_summary_20251017-233201,1,1,ok,mean(AUC)=0.325391 > 0.292852
16,pair_ewb_summary_20251017-233201_holdout,1,1,ok,mean(AUC)=0.518903 > 0.467013
17,pair_ewb_summary_v2_1_20251017-234015,1,1,ok,mean(AUC)=0.234282 > 0.210854
18,pair_ewb_summary_v2_1_20251017-234015_holdout,1,1,ok,mean(AUC)=0.518903 > 0.467013
19,pair_ewb_summary_v2_2_20251017-234543,1,1,ok,mean(AUC)=0.418632 > 0.376769
20,pair_ewb_summary_v2_2_20251017-234543_holdout,1,1,ok,mean(AUC)=0.518903 > 0.467013
21,pair_ewb_summary_v2_3_20251017-234953,1,1,ok,mean(AUC)=0.458255 > 0.412429
22,pair_ewb_summary_v2_3_20251017-234953_holdout,1,1,ok,mean(AUC)=0.518903 > 0.467013
23,test_readme,0,0,needs_holdout,skip
24,test_cnt_correlates_report_20251015-163558,0,0,needs_holdout,skip


In [37]:
# ================================
# CNT — Paired Evidence Sweep v2
# ================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, hashlib, json, glob as _globmod

# ---------- CONFIG ----------
LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
STRICT_EWB_AUC055 = False   # set True to escalate EWB pairs to AUC > 0.55 after harvest
CALIBRATION_MODE  = "relative_90pct"  # threshold = 0.9 * mean for each CSV (base & holdout)

# ---------- REQUIRE MEGA-CELL PRIMITIVES ----------
if "run_cycle" not in globals():
    # try to import from the SSD script file
    import importlib.util
    mod_path = LAB / "cnt_engine_megacell.py"
    if not mod_path.exists():
        raise RuntimeError("run_cycle not found. Load your Mega Cell (or save it as E:\\CNT\\cnt_engine_megacell.py) and re-run.")
    spec = importlib.util.spec_from_file_location("cnt_engine", str(mod_path))
    cnt_engine = importlib.util.module_from_spec(spec); spec.loader.exec_module(cnt_engine)
    run_cycle = cnt_engine.run_cycle
    # adopt helpers into globals
    TEST_FENCE = getattr(cnt_engine, "TEST_FENCE", re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL))
    _csv_threshold_test_strict = getattr(cnt_engine, "_csv_threshold_test_strict", None)
    _parse_cnt_tests = getattr(cnt_engine, "_parse_cnt_tests", None)
    ENGINE_ROOT = getattr(cnt_engine, "ENGINE_ROOT", LAB / "artifacts" / "cnt_engine_megacell")
else:
    # use whatever is already loaded in this kernel
    TEST_FENCE = globals().get("TEST_FENCE", re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL))
    ENGINE_ROOT = globals().get("ENGINE_ROOT", LAB / "artifacts" / "cnt_engine_megacell")

# SOURCE_ROOTS from your mega cell
if "SOURCE_ROOTS" not in globals():
    SOURCE_ROOTS = [
        LAB / "notes",
        LAB / "artifacts" / "cnt_scroll",
        LAB / "artifacts" / "cnt_codex",
        LAB / "notebooks",
    ]

# ---------- UTILS ----------
def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s or "").strip().lower())

def _pick_metric_col(df: pd.DataFrame, prefer=None):
    nm = {_norm(c): c for c in df.columns}
    prefs = ([prefer] if prefer else []) + ["auc","precision@theta*","precision_at_theta","delta_theta","lead@hit","lead_at_hit"]
    for want in prefs:
        if want and _norm(want) in nm:
            return nm[_norm(want)]
    # fallback: first numeric column
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any():
            return c
    return None

def _policy_value(mode, series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return None
    if mode == "relative_90pct":
        return float(round(0.9*float(s.mean()), 6))
    if mode == "strict_055":
        return 0.55
    if mode == "above_chance":
        return 0.50
    return float(round(0.9*float(s.mean()), 6))

def _fence(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False, allow_unicode=True) + "```"

def _upsert_fenced(doc_text: str, name: str, block: str) -> str:
    out, replaced = doc_text, False
    for m in TEST_FENCE.finditer(doc_text or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and str(spec.get("name","")).strip().lower() == name.strip().lower():
            a,b = m.span(); out = doc_text[:a] + block + doc_text[b:]; replaced=True; break
    if not replaced:
        out = (out.rstrip() + "\n\n" + block + "\n")
    return out

# ---------- 0) Quarantine placeholders (skip:true) ----------
PLACEHOLDER_NAME_PAT = re.compile(r"^(test_readme|test_summary)$", re.IGNORECASE)
PLACEHOLDER_COL_KEYS = {"npairs","n_pairs","n-pairs"}
updated_skip = 0
for root in SOURCE_ROOTS:
    for doc in list(root.rglob("*.md")) + list(root.rglob("*.txt")):
        try:
            txt = doc.read_text(encoding="utf-8", errors="ignore")
        except Exception:
            continue
        changes=[]
        for m in TEST_FENCE.finditer(txt or ""):
            try:
                spec = yaml.safe_load(m.group("body")) or {}
            except Exception:
                continue
            if not isinstance(spec, dict) or spec.get("type")!="csv_threshold": 
                continue
            name = str(spec.get("name",""))
            col  = str(spec.get("column",""))
            if PLACEHOLDER_NAME_PAT.match(name) or _norm(col) in PLACEHOLDER_COL_KEYS:
                if not spec.get("skip", False):
                    spec["skip"] = True
                    changes.append((m.span(), _fence(spec)))
        if changes:
            new_txt = txt
            for (a,b), rep in reversed(changes):
                new_txt = new_txt[:a] + rep + new_txt[b:]
            doc.write_text(new_txt, encoding="utf-8"); updated_skip += 1
print(f"[skip] placeholders quarantined in docs: {updated_skip}")

# ---------- 1) Harvest EWB pairs (base + holdout) ----------
EWB_DIR = LAB / r"notebooks\archive\cnt_ewb_theta2"
DOC_EWB = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_EWB.parent.mkdir(parents=True, exist_ok=True)
txt_ewb = DOC_EWB.read_text(encoding="utf-8", errors="ignore") if DOC_EWB.exists() else "# EWB Results — Paired Evidence\n"

bases = sorted(EWB_DIR.rglob("*.csv"))
holds = [p for p in bases if "holdout" in p.name.lower()]
hold_by_dir = {}
for h in holds:
    hold_by_dir.setdefault(h.parent, []).append(h)

minted_ewb = 0
for base in bases:
    if "holdout" in base.name.lower():
        continue
    # pick a holdout neighbor
    ho = None
    if base.parent in hold_by_dir:
        ho = hold_by_dir[base.parent][0]
    else:
        cand = base.parent / "ewb_holdout_results.csv"
        if cand.exists(): ho = cand
    if not ho: 
        continue

    dfb = pd.read_csv(base); dfh = pd.read_csv(ho)
    col_b = _pick_metric_col(dfb, "auc"); col_h = _pick_metric_col(dfh, col_b)
    if not col_b or not col_h: 
        continue
    sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
    sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
    if sb.empty or sh.empty:
        continue

    thr_b = _policy_value("relative_90pct", sb)
    thr_h = _policy_value("relative_90pct", sh)

    name_base = f"pair_{base.stem}"
    name_hold = f"{name_base}_holdout"

    spec_b = dict(name=name_base, type="csv_threshold", path=str(PureWindowsPath(base)),
                  column=col_b, op=">", value=thr_b, agg="mean",
                  gold=True, frozen_at=STAMP, hash=_sha256(base), rows=int(len(dfb)))
    spec_h = dict(name=name_hold, type="csv_threshold", path=str(PureWindowsPath(ho)),
                  column=col_h, op=">", value=thr_h, agg="mean",
                  gold=True, frozen_at=STAMP, hash=_sha256(ho), rows=int(len(dfh)))

    txt_ewb = _upsert_fenced(txt_ewb, name_base, _fence(spec_b))
    txt_ewb = _upsert_fenced(txt_ewb, name_hold, _fence(spec_h))
    minted_ewb += 1

# optional escalation for EWB pairs to strict AUC > 0.55
if STRICT_EWB_AUC055:
    new_txt = txt_ewb
    for m in TEST_FENCE.finditer(txt_ewb or ""):
        try: spec = yaml.safe_load(m.group("body")) or {}
        except Exception: continue
        if not isinstance(spec, dict) or spec.get("type")!="csv_threshold": continue
        nm = str(spec.get("name",""))
        if _norm(nm).startswith("pair_") or _norm(nm).startswith("test_ewb_pair"):
            col = str(spec.get("column",""))
            if "auc" in _norm(col) or "precision" in _norm(col):
                spec["op"] = ">"
                spec["value"] = 0.55
                a,b = m.span()
                new_txt = new_txt[:a] + _fence(spec) + new_txt[b:]
    txt_ewb = new_txt

DOC_EWB.write_text(txt_ewb, encoding="utf-8")
print(f"[ewb] harvested pairs written: {minted_ewb} → {DOC_EWB}")

# ---------- 2) Harvest Cooling pairs (*_labeled as holdout) ----------
# Looks for files like cnt_cooling_log_...csv and cnt_cooling_log_..._labeled.csv in same dir
minted_cooling = 0
COOL_GLOB = list(EWB_DIR.rglob("ewb_cooling_segments*.csv")) + list(LAB.rglob("cnt_cooling_log_*.csv"))
by_key = {}
for p in COOL_GLOB:
    stem = p.stem
    key = stem.replace("_labeled", "")
    by_key.setdefault((p.parent, key), []).append(p)

for (parent, key), files in by_key.items():
    base = None; hold = None
    for p in files:
        if p.stem.endswith("_labeled"):
            hold = p
        else:
            base = p
    if not (base and hold and base.exists() and hold.exists()):
        continue
    dfb = pd.read_csv(base); dfh = pd.read_csv(hold)
    col_b = _pick_metric_col(dfb, "auc"); col_h = _pick_metric_col(dfh, col_b)
    if not col_b or not col_h:
        continue
    sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
    sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
    if sb.empty or sh.empty:
        continue

    thr_b = _policy_value(CALIBRATION_MODE, sb)
    thr_h = _policy_value(CALIBRATION_MODE, sh)

    name_base = f"pair_cooling_{key}"
    name_hold = f"{name_base}_holdout"

    spec_b = dict(name=name_base, type="csv_threshold", path=str(PureWindowsPath(base)),
                  column=col_b, op=">", value=thr_b, agg="mean",
                  gold=True, frozen_at=STAMP, hash=_sha256(base), rows=int(len(dfb)))
    spec_h = dict(name=name_hold, type="csv_threshold", path=str(PureWindowsPath(hold)),
                  column=col_h, op=">", value=thr_h, agg="mean",
                  gold=True, frozen_at=STAMP, hash=_sha256(hold), rows=int(len(dfh)))

    # write to a Cooling doc under notes/
    DOC_COOL = LAB / r"notes\cooling_pairs.md"
    txt = DOC_COOL.read_text(encoding="utf-8", errors="ignore") if DOC_COOL.exists() else "# Cooling — Paired Evidence\n"
    txt = _upsert_fenced(txt, name_base, _fence(spec_b))
    txt = _upsert_fenced(txt, name_hold, _fence(spec_h))
    DOC_COOL.write_text(txt, encoding="utf-8")
    minted_cooling += 1

print(f"[cooling] minted pairs: {minted_cooling}")

# ---------- 3) Patch the runner (robust paired-evidence, soft mode) ----------
import inspect
rc_mod = inspect.getmodule(run_cycle)

def _is_holdout_name(name: str) -> bool:
    s = str(name or "").strip().lower()
    return s.endswith("_holdout")

def _base_key(name: str) -> str:
    s = str(name or "").strip()
    return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

def _csv_threshold_test_strict_patch(t: dict):
    # respect skip + gold + hash verification; delegate to core tester available in the module/globals
    if t.get("skip", False):
        return False, "skip", "skip"
    if not t.get("gold", False):
        return False, "non_gold", "non_gold"
    p = t.get("path"); 
    if not p or not Path(p).exists():
        return False, "missing_path", "missing_path"
    if t.get("hash"):
        h_now = _sha256(Path(p))
        if str(h_now) != str(t["hash"]):
            return False, "hash_mismatch", "hash_mismatch"
    # call whichever normalized tester exists (module or global)
    core = getattr(rc_mod, "_csv_threshold_test", globals().get("_csv_threshold_test", None))
    if core is None:
        return False, "no_core_tester", "invalid_type"
    ok, info, reason = core(t)
    return ok, info, reason

def _paired_runner_soft(df: pd.DataFrame, out_csv: Path):
    rows = []
    parse = getattr(rc_mod, "_parse_cnt_tests", globals().get("_parse_cnt_tests"))
    if parse is None:
        parse = lambda s: []
    for _, r in df.iterrows():
        path = r["path"]
        tests = parse(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict_patch(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name)),
                base_key=_base_key(name)
            ))
    rep = pd.DataFrame(rows)
    if rep.empty:
        rep.to_csv(out_csv, index=False); return 0,0,0

    # consider only rows that reached 'ok'
    rep.loc[~rep["reason"].eq("ok"), ["valid","passed"]] = 0

    # pair gating
    by_key = {k: list(idxs) for k, idxs in rep.groupby("base_key").indices.items()}
    to_pair_fail, to_needs = set(), set()
    for key, idxs in by_key.items():
        base_idxs = [i for i in idxs if rep.loc[i,"is_holdout"]==0]
        hold_idxs = [i for i in idxs if rep.loc[i,"is_holdout"]==1]
        base_pass = any(bool(rep.loc[i,"passed"]) for i in base_idxs)
        hold_pass = any(bool(rep.loc[i,"passed"]) for i in hold_idxs)
        if base_idxs and not hold_idxs:
            to_needs.update(base_idxs)
        elif hold_idxs and not base_idxs:
            to_needs.update(hold_idxs)
        else:
            if not (base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)

    if to_needs:
        rep.loc[list(to_needs), ["valid","passed"]] = 0
        rep.loc[list(to_needs), "reason"] = "needs_holdout"
    if to_pair_fail:
        rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
        rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    rep.to_csv(out_csv, index=False)
    total  = int(rep["valid"].sum())
    passed = int(rep["passed"].sum())
    runnable = int(len(rep))
    return passed, total, runnable

# bind into the module used by run_cycle
setattr(rc_mod, "_csv_threshold_test_strict", _csv_threshold_test_strict_patch)
setattr(rc_mod, "run_cnt_tests_on_df", _paired_runner_soft)

# ---------- 4) Re-run and report ----------
res = run_cycle()
print(json.dumps(res, indent=2))

OUT = Path(str(ENGINE_ROOT)) / "out"
reps = sorted(_globmod.glob(str(OUT / "tests_report_*.csv")))
if reps:
    rep = Path(reps[-1]); dfrep = pd.read_csv(rep)
    print("\nLatest report:", rep)
    print(dfrep["reason"].value_counts(dropna=False))

    # Pair dashboard (compact)
    def _is_hold(name): return str(name).lower().strip().endswith("_holdout")
    dash = dfrep.copy()
    dash["is_hold"] = dash["name"].apply(_is_hold).astype(int)
    dash["base_key"] = dash["name"].apply(lambda s: (_norm(s[:-9]) if _is_hold(s) else _norm(s)))
    grp = dash.groupby("base_key").agg(
        base_ok=("passed", lambda s: int(any((s[dash.loc[s.index,"is_hold"]==0)]))),
        hold_ok=("passed", lambda s: int(any((s[dash.loc[s.index,"is_hold"]==1)]))),
        counted=("valid","sum"),
        total=("name","count"),
    )
    grp["pair_ok"] = (grp["base_ok"] & grp["hold_ok"]).astype(int)
    try:
        from IPython.display import display
        display(grp.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).tail(24))
    except Exception:
        print(grp.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).tail(24).to_string(index=False))
else:
    print("No tests_report_* yet.")


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (139954510.py, line 358)

In [38]:
# === Pair dashboard FIX (no fragile lambdas) ===
from pathlib import Path
import pandas as pd, re, os
import glob as globmod

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
OUT = (Path(str(ENGINE_ROOT)) / "out") if "ENGINE_ROOT" in globals() else LAB / r"artifacts\cnt_engine_megacell\out"

rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
rep = Path(rep_paths[-1])
df  = pd.read_csv(rep)

def _norm(s): return re.sub(r"[^a-z0-9]+","", str(s).lower().strip())
def _is_hold(name): return str(name).lower().strip().endswith("_holdout")

# Normalize keys and flags
df["is_hold"] = df["name"].apply(_is_hold).astype(int)
df["base_key"] = df["name"].apply(lambda s: _norm(s[:-9]) if _is_hold(s) else _norm(s))
df["ok_row"] = (df["reason"].eq("ok")).astype(int)

# Components: a row contributes to base_ok/hold_ok only if it actually PASSED and is ok
df["base_pass"] = ((df["is_hold"]==0) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)
df["hold_pass"] = ((df["is_hold"]==1) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)

# Aggregate cleanly
grp = df.groupby("base_key").agg(
    base_ok=("base_pass", "max"),
    hold_ok=("hold_pass", "max"),
    counted=("valid", "sum"),
    total=("name", "count"),
)
grp["pair_ok"] = (grp["base_ok"] & grp["hold_ok"]).astype(int)

print("=== Pair Summary ===")
try:
    from IPython.display import display
    display(grp.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(40))
except Exception:
    print(grp.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(40).to_string())


=== Pair Summary ===


,base_ok,hold_ok,counted,total,pair_ok
base_key,,,,,
pairewbcoolingsegments2025101723521,0,1,1,1,0
pairewbcoolingsegments20251017235217,1,0,1,1,0
pairewbsummary2025101723320,0,1,1,1,0
pairewbsummary20251017233201,1,0,1,1,0
pairewbsummaryv212025101723401,0,1,1,1,0
pairewbsummaryv2120251017234015,1,0,1,1,0
pairewbsummaryv222025101723454,0,1,1,1,0
pairewbsummaryv2220251017234543,1,0,1,1,0
pairewbsummaryv232025101723495,0,1,1,1,0


In [39]:
# ===========================================
# CNT — Paired Evidence: Lock & Export (One Cell)
# ===========================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, json, zipfile, hashlib
import pandas as pd
import glob as globmod

# ---------- CONFIG ----------
LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART = LAB / r"artifacts\cnt_engine_megacell"
OUT = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

DOC_EWB   = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL  = LAB / r"notes\cooling_pairs.md"

# Toggle: escalate EWB paired thresholds to AUC > 0.55, then re-run
ESCALATE_EWB_TO_055 = False

# ---------- REQUIRE mega-cell CORE ----------
try:
    run_cycle
    TEST_FENCE
except NameError:
    raise RuntimeError("Load your Mega Cell first so run_cycle() and TEST_FENCE exist in this kernel.")

# ---------- MONKEY-PATCH the paired-evidence runner (SOFT mode) ----------
import inspect
rc_mod = inspect.getmodule(run_cycle)

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _is_holdout_name(name: str) -> bool:
    s = str(name or "").strip().lower()
    return s.endswith("_holdout")

def _base_key(name: str) -> str:
    s = str(name or "").strip()
    if s.lower().endswith("_holdout"):
        s = s[:-len("_holdout")]
    return _norm(s)

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

# strict tester that respects skip + gold + hash, delegates to core normalized tester
def _csv_threshold_test_strict_patch(t: dict):
    if t.get("skip", False):
        return False, "skip", "skip"
    if not t.get("gold", False):
        return False, "non_gold", "non_gold"
    p = t.get("path")
    if not p or not Path(p).exists():
        return False, "missing_path", "missing_path"
    if t.get("hash"):
        if _sha256(Path(p)) != str(t["hash"]):
            return False, "hash_mismatch", "hash_mismatch"
    core = getattr(rc_mod, "_csv_threshold_test", globals().get("_csv_threshold_test", None))
    if core is None:
        return False, "no_core_tester", "invalid_type"
    ok, info, reason = core(t)
    return ok, info, reason

def _paired_runner_soft(df: pd.DataFrame, out_csv: Path):
    rows = []
    parse = getattr(rc_mod, "_parse_cnt_tests", globals().get("_parse_cnt_tests", lambda s: []))
    for _, r in df.iterrows():
        path  = r["path"]
        tests = parse(r.get("text",""))
        for t in tests:
            name  = t.get("name", Path(path).stem)
            ttype = t.get("type","unknown")
            if ttype == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict_patch(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttype}", "invalid_type"
            rows.append(dict(
                doc=path, name=name, type=ttype,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_holdout_name(name)),
                base_key=_base_key(name),
            ))
    rep = pd.DataFrame(rows)
    if rep.empty:
        rep.to_csv(out_csv, index=False); return 0,0,0

    # keep only strict-ok rows as candidates
    rep.loc[~rep["reason"].eq("ok"), ["valid","passed"]] = 0

    # pair gating
    by_key = {k: list(idxs) for k, idxs in rep.groupby("base_key").indices.items()}
    to_pair_fail, to_needs = set(), set()
    for key, idxs in by_key.items():
        base_idxs = [i for i in idxs if rep.loc[i,"is_holdout"]==0]
        hold_idxs = [i for i in idxs if rep.loc[i,"is_holdout"]==1]
        base_pass = any(bool(rep.loc[i,"passed"]) for i in base_idxs)
        hold_pass = any(bool(rep.loc[i,"passed"]) for i in hold_idxs)
        if base_idxs and not hold_idxs:
            to_needs.update(base_idxs)
        elif hold_idxs and not base_idxs:
            to_needs.update(hold_idxs)
        else:
            if not (base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)

    if to_needs:
        rep.loc[list(to_needs), ["valid","passed"]] = 0
        rep.loc[list(to_needs), "reason"] = "needs_holdout"
    if to_pair_fail:
        rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
        rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    rep.to_csv(out_csv, index=False)
    total  = int(rep["valid"].sum())
    passed = int(rep["passed"].sum())
    runnable = int(len(rep))
    return passed, total, runnable

# Bind into the module used by run_cycle
setattr(rc_mod, "_csv_threshold_test_strict", _csv_threshold_test_strict_patch)
setattr(rc_mod, "run_cnt_tests_on_df", _paired_runner_soft)

# ---------- (Optional) Escalate EWB pairs to AUC > 0.55 ----------
if ESCALATE_EWB_TO_055 and DOC_EWB.exists():
    txt = DOC_EWB.read_text(encoding="utf-8", errors="ignore")
    changed = 0
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if not isinstance(spec, dict) or spec.get("type")!="csv_threshold":
            continue
        nm  = str(spec.get("name",""))
        col = str(spec.get("column",""))
        if (_norm(nm).startswith("pair_") or _norm(nm).startswith("test_ewb_pair")) and ("auc" in _norm(col) or "precision" in _norm(col)):
            spec["op"] = ">"
            spec["value"] = 0.55
            a,b = m.span()
            txt = txt[:a] + ("```cnt-test\n"+yaml.safe_dump(spec, sort_keys=False)+"```") + txt[b:]
            changed += 1
    if changed:
        DOC_EWB.write_text(txt, encoding="utf-8")
        print(f"[strict] escalated {changed} EWB tests to AUC>0.55")

# ---------- RUN once ----------
res = run_cycle()
print(json.dumps(res, indent=2))

# ---------- PAIR DASHBOARD (robust, no fragile lambdas) ----------
rep_paths = sorted(globmod.glob(str(OUT/"tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
rep = Path(rep_paths[-1])
df  = pd.read_csv(rep)

def _is_hold(name): return str(name).lower().strip().endswith("_holdout")

# Normalize flags (do NOT recompute base_key unless missing)
if "base_key" not in df.columns:
    df["base_key"] = df["name"].apply(lambda s: _norm(s[:-9]) if _is_hold(s) else _norm(s))
df["is_hold"]   = df["name"].apply(_is_hold).astype(int)
df["ok_row"]    = (df["reason"].eq("ok")).astype(int)
df["base_pass"] = ((df["is_hold"]==0) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)
df["hold_pass"] = ((df["is_hold"]==1) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)

dash = df.groupby("base_key").agg(
    base_ok=("base_pass","max"),
    hold_ok=("hold_pass","max"),
    counted=("valid","sum"),
    total=("name","count"),
)
dash["pair_ok"] = (dash["base_ok"] & dash["hold_ok"]).astype(int)

print("\n=== Pair Summary ===")
try:
    from IPython.display import display
    display(dash.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(50))
except Exception:
    print(dash.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(50).to_string())

# ---------- PROOF BUNDLE (zip) ----------
bundle_dir = ART / "bundles"
bundle_dir.mkdir(parents=True, exist_ok=True)
STAMP = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
bundle = bundle_dir / f"cnt_proof_bundle_{STAMP}.zip"

with zipfile.ZipFile(bundle, "w", zipfile.ZIP_DEFLATED) as z:
    # core artifacts
    if Path(rep).exists():          z.write(rep, arcname=Path(rep).name)
    if STATE.exists():              z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():       z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists():      z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    # key docs that carry the tests
    if DOC_EWB.exists():            z.write(DOC_EWB, arcname=DOC_EWB.name)
    if DOC_COOL.exists():           z.write(DOC_COOL, arcname=DOC_COOL.name)

print(f"\nProof bundle → {bundle}")


{
  "score": {
    "clarity": 0.7941439024390243,
    "novelty": 1.0,
    "coherence": 0.8991551091981508,
    "falsifiability": 1.0,
    "total": 0.9233247529092938
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 12,
  "tests_passed": 12,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_anomaly\\out\\CNT_Gold_Dossier_J135656-011722.md",
      "resid": 0.917654107784611,
      "hint": "# CNT Gold Dossier \u2014 WISE J135656.78\u2212011722.9 **Position (ICRS):** RA 209.236592, Dec -1.289716 ## AllWISE photometry & flags - W1=14.348999977111816, W2=14.0, W3=10.232000350952148, W4=8.480999946594238 - W1\u2212W2=0.349, W2\u2212W3=3.768 - ph_qual=, cc_flags=, ext_flg= - SNR: w1=37.4, w2=27.8 (W3/W4 from table if present) ## Catalog labels - SIMBAD: () - NED: WISEA J135656.78-011722.9 [G] z=0.148182 ## G"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_one_signal_20251017-215548\\summary.txt",
      "resid": 0.9175153485631594,
      "hint": "== CNT On

,base_ok,hold_ok,counted,total,pair_ok
base_key,,,,,
pairewbcoolingsegments20251017235217,1,1,2,2,1
pairewbsummary20251017233201,1,1,2,2,1
pairewbsummaryv2120251017234015,1,1,2,2,1
pairewbsummaryv2220251017234543,1,1,2,2,1
pairewbsummaryv2320251017234953,1,1,2,2,1
testewbpair,1,1,2,2,1
testgoldverificationsummary,0,0,0,1,0
testcntcorrelatesreport20251015163558,0,0,0,2,0
testcntcorrelatesreport20251015164130,0,0,0,3,0



Proof bundle → E:\CNT\artifacts\cnt_engine_megacell\bundles\cnt_proof_bundle_20251026-225233.zip


In [41]:
# ===========================================
# CNT — Proof Seal v1 (Pairs-Only) — ONE CELL
# ===========================================
from pathlib import Path
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, json, zipfile, glob as globmod, hashlib

# ---------- Paths ----------
LAB   = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART   = LAB / r"artifacts\cnt_engine_megacell"
OUT   = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

DOC_EWB  = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL = LAB / r"notes\cooling_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# Require engine bits
try:
    TEST_FENCE
except NameError:
    import re
    TEST_FENCE = re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL)

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _is_hold(name: str) -> bool:
    return str(name or "").lower().strip().endswith("_holdout")

# ---------- 1) Load latest paired-evidence report ----------
rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1])
df  = pd.read_csv(REP)

# ensure expected columns
for col in ["base_key","reason","passed","valid","name","doc"]:
    assert col in df.columns, f"Report missing column '{col}'"

# keep only rows that are strict-ok AND counted valid
df_ok = df[(df["reason"]=="ok") & (df["valid"]==1) & (df["passed"]==1)].copy()
assert not df_ok.empty, "No passing rows found in latest report. Run the engine first."

# group rows by normalized base key; a true pair has a base and a holdout
df_ok["is_holdout"] = df_ok["name"].apply(_is_hold).astype(int)
pairs = []
for k, g in df_ok.groupby("base_key"):
    has_base = any(g["is_holdout"]==0)
    has_hold = any(g["is_holdout"]==1)
    if has_base and has_hold:
        pairs.append((k, g.copy()))

assert pairs, "No true base↔holdout pairs in ok set."

# ---------- 2) Parse all docs to map name -> full YAML spec ----------
def _scan_doc(doc_path: Path):
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return {}
    specs = {}
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "type" in spec and "name" in spec:
            specs[str(spec["name"]).strip()] = spec
    return specs

# Build a unified spec index from manifest docs we care about
name2spec = {}
doc_candidates = set(df_ok["doc"].tolist())
# add canonical docs
for extra in [DOC_EWB, DOC_COOL]:
    if extra.exists():
        doc_candidates.add(str(extra))
for d in doc_candidates:
    name2spec.update(_scan_doc(Path(d)))

# ---------- 3) Assemble the SEAL payload (pairs-only) ----------
def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

pairs_payload = []
for base_key, g in pairs:
    # pick one base and one holdout row
    gb = g[g["is_holdout"]==0].iloc[0]
    gh = g[g["is_holdout"]==1].iloc[0]
    spec_b = name2spec.get(gb["name"], {})
    spec_h = name2spec.get(gh["name"], {})
    # sanity: ensure gold + hash present; if not, compute hashes now
    def _enrich(spec):
        out = dict(spec)
        p = Path(out.get("path",""))
        if p.exists() and not out.get("hash"):
            out["hash"] = _sha256(p)
        if p.exists() and not out.get("rows"):
            try:
                out["rows"] = int(len(pd.read_csv(p)))
            except Exception:
                pass
        return out
    spec_b = _enrich(spec_b)
    spec_h = _enrich(spec_h)
    pairs_payload.append(dict(
        base_key   = base_key,
        doc_base   = str(gb["doc"]),
        doc_hold   = str(gh["doc"]),
        spec_base  = spec_b,
        spec_hold  = spec_h,
        report_rows= [gb.to_dict(), gh.to_dict()],
    ))

SEAL = dict(
    seal_version = "CNT-Proof-1",
    created_at   = STAMP,
    lab_root     = str(LAB),
    engine_state = (yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file  = REP.name,
    pairs_ok     = len(pairs_payload),
    pairs        = pairs_payload,
)

# ---------- 4) Write CNT_SEAL.{yaml,md} ----------
SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"

with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

# Simple human-readable seal
lines = []
lines.append(f"# CNT Proof Seal — Pairs-Only")
lines.append(f"- Created: {STAMP}")
lines.append(f"- Lab   : {LAB}")
lines.append(f"- Pairs : {len(pairs_payload)}\n")
for i, P in enumerate(pairs_payload, 1):
    sb = P["spec_base"]; sh = P["spec_hold"]
    lines.append(f"## Pair {i}: `{P['base_key']}`")
    lines.append(f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`")
    lines.append(f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}")
    lines.append(f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`")
    lines.append(f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')} {sh.get('value','?')}`  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}\n")
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")

print(f"[seal] wrote {SEAL_YAML.name} and {SEAL_MD.name}")

# ---------- 5) Build Proof Bundle (with seal) ----------
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"

with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    # core artifacts
    if Path(REP).exists():     z.write(REP, arcname=Path(REP).name)
    if STATE.exists():         z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():  z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists(): z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    # seal
    z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    z.write(SEAL_MD,   arcname=SEAL_MD.name)
    # key docs
    if DOC_EWB.exists():  z.write(DOC_EWB,  arcname=DOC_EWB.name)
    if DOC_COOL.exists(): z.write(DOC_COOL, arcname=DOC_COOL.name)

print(f"[bundle] {BUNDLE}")


[seal] wrote CNT_SEAL.yaml and CNT_SEAL.md
[bundle] E:\CNT\artifacts\cnt_engine_megacell\bundles\cnt_proof_bundle_20251026-225844_SEAL.zip


In [42]:
# ===========================================
# CNT — 3I Atlas Paired Harvester + Seal Refresh (ONE CELL)
# ===========================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, json, zipfile, hashlib
import pandas as pd
import glob as globmod

# ---------- ROOTS ----------
LAB   = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART   = LAB / r"artifacts\cnt_engine_megacell"
OUT   = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

ATLAS_ROOT = LAB / r"notebooks\archive\cnt_3i_atlas_all8"
DOC_ATLAS  = LAB / r"notes\atlas_pairs.md"  # where we’ll write the tests
DOC_EWB    = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL   = LAB / r"notes\cooling_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---------- REQUIRE MEGA-CELL CORE ----------
try:
    run_cycle
    TEST_FENCE
except NameError:
    raise RuntimeError("Load your Mega Cell first so run_cycle() and TEST_FENCE exist.")

# ---------- HELPERS ----------
def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _pick_metric_col(df: pd.DataFrame, prefer=None):
    nm = {_norm(c): c for c in df.columns}
    prefs = ([prefer] if prefer else []) + [
        "auc","roc_auc","auroc","ap","aupr","f1","f1_score","precision","recall","accuracy"
    ]
    for want in prefs:
        if want and _norm(want) in nm:
            return nm[_norm(want)]
    # fallback: first numeric column
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any():
            return c
    return None

def _policy_value(series, mode="relative_90pct"):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return None
    if mode == "relative_90pct":
        return float(round(0.9*float(s.mean()), 6))
    if mode == "above_chance":
        return 0.50
    if mode == "strict_055":
        return 0.55
    return float(round(0.9*float(s.mean()), 6))

def _fence(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False, allow_unicode=True) + "```"

def _upsert_fenced(doc_text: str, name: str, block: str) -> str:
    out, replaced = doc_text, False
    for m in TEST_FENCE.finditer(doc_text or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and str(spec.get("name","")).strip().lower() == name.strip().lower():
            a,b = m.span(); out = doc_text[:a] + block + doc_text[b:]; replaced=True; break
    if not replaced:
        out = (out.rstrip() + "\n\n" + block + "\n")
    return out

def _atlas_basekey(stem: str):
    # remove typical holdout/val/test tokens to group base↔holdout
    s = stem
    s = re.sub(r"(?i)([_\-](holdout|val|validation|test))+$", "", s)
    return s

# ---------- 1) SCAN ATLAS TABLES AND MINT PAIRS ----------
pairs_minted = 0
DOC_ATLAS.parent.mkdir(parents=True, exist_ok=True)
txt_atlas = DOC_ATLAS.read_text(encoding="utf-8", errors="ignore") if DOC_ATLAS.exists() else "# 3I Atlas — Paired Evidence\n"

# Find all CSVs under .../out/tables/
table_csvs = [Path(p) for p in globmod.glob(str(ATLAS_ROOT / "**" / "out" / "tables" / "*.csv"), recursive=True)]

# Group by directory; within each dir, group by base key (stem without holdout/test tokens)
from collections import defaultdict
by_dirkey = defaultdict(lambda: {"base": None, "hold": None, "all": []})

for p in table_csvs:
    stem = p.stem
    key  = _atlas_basekey(stem)
    is_hold = bool(re.search(r"(?i)(holdout|val|validation|test)", stem))
    rec = by_dirkey[(p.parent, key)]
    rec["all"].append(p)
    if is_hold:
        rec["hold"] = p if (rec["hold"] is None) else rec["hold"]
    else:
        rec["base"] = p if (rec["base"] is None) else rec["base"]

for (parent, key), rec in by_dirkey.items():
    base, hold = rec["base"], rec["hold"]
    if not (base and hold and base.exists() and hold.exists()):
        continue
    try:
        dfb = pd.read_csv(base); dfh = pd.read_csv(hold)
    except Exception:
        continue

    col_b = _pick_metric_col(dfb, None)
    col_h = _pick_metric_col(dfh, col_b)
    if not col_b or not col_h:
        continue

    sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
    sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
    if sb.empty or sh.empty:
        continue

    thr_b = _policy_value(sb, "relative_90pct")
    thr_h = _policy_value(sh, "relative_90pct")

    name_base = f"pair_atlas_{key}"
    name_hold = f"{name_base}_holdout"

    spec_b = dict(
        name=name_base, type="csv_threshold",
        path=str(PureWindowsPath(base)), column=col_b, op=">", value=thr_b, agg="mean",
        gold=True, frozen_at=STAMP, hash=_sha256(base), rows=int(len(dfb))
    )
    spec_h = dict(
        name=name_hold, type="csv_threshold",
        path=str(PureWindowsPath(hold)), column=col_h, op=">", value=thr_h, agg="mean",
        gold=True, frozen_at=STAMP, hash=_sha256(hold), rows=int(len(dfh))
    )

    txt_atlas = _upsert_fenced(txt_atlas, name_base, _fence(spec_b))
    txt_atlas = _upsert_fenced(txt_atlas, name_hold, _fence(spec_h))
    pairs_minted += 1

DOC_ATLAS.write_text(txt_atlas, encoding="utf-8")
print(f"[atlas] minted/updated pairs: {pairs_minted} → {DOC_ATLAS}")

# ---------- 2) RE-RUN ENGINE (paired-evidence runner assumed patched already) ----------
res = run_cycle()
print(json.dumps(res, indent=2))

# ---------- 3) BUILD/REFRESH SEAL (pairs-only) AND BUNDLE ----------
# Reuse the latest tests report, extract true base↔holdout ok pairs, write SEAL + bundle

def _load_specs_from(doc_path: Path):
    out = {}
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1])
dfrep = pd.read_csv(REP)

def _is_hold(nm): return str(nm).lower().strip().endswith("_holdout")
def _base_key(nm):
    s = str(nm).strip()
    return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

ok = dfrep[(dfrep["reason"]=="ok") & (dfrep["valid"]==1) & (dfrep["passed"]==1)].copy()
ok["is_holdout"] = ok["name"].apply(_is_hold).astype(int)
pairs = []
for k, g in ok.groupby(dfrep["name"].apply(_base_key)):
    gb = g[g["is_holdout"]==0]
    gh = g[g["is_holdout"]==1]
    if not gb.empty and not gh.empty:
        pairs.append((k, gb.iloc[0], gh.iloc[0]))

# Build name->spec index from key docs + any doc referenced by ok report rows
name2spec = {}
doc_candidates = set(ok["doc"].tolist())
for extra in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
    if extra.exists():
        doc_candidates.add(str(extra))
for d in doc_candidates:
    name2spec.update(_load_specs_from(Path(d)))

def _enrich(spec):
    out = dict(spec or {})
    p = Path(out.get("path",""))
    if p.exists() and not out.get("hash"): out["hash"]=_sha256(p)
    if p.exists() and not out.get("rows"):
        try: out["rows"]=int(len(pd.read_csv(p)))
        except Exception: pass
    return out

pairs_payload=[]
for key, row_b, row_h in pairs:
    sb = _enrich(name2spec.get(row_b["name"], {}))
    sh = _enrich(name2spec.get(row_h["name"], {}))
    pairs_payload.append(dict(
        base_key=key,
        doc_base=str(row_b["doc"]),
        doc_hold=str(row_h["doc"]),
        spec_base=sb, spec_hold=sh,
        report_rows=[row_b.to_dict(), row_h.to_dict()],
    ))

SEAL = dict(
    seal_version="CNT-Proof-1",
    created_at=STAMP,
    lab_root=str(LAB),
    engine_state=(yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file=Path(REP).name,
    pairs_ok=len(pairs_payload),
    pairs=pairs_payload,
)

SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"
with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

lines = [f"# CNT Proof Seal — Pairs-Only",
         f"- Created: {STAMP}",
         f"- Lab   : {LAB}",
         f"- Pairs : {len(pairs_payload)}",
         ""]
for i,P in enumerate(pairs_payload,1):
    sb,sh=P["spec_base"],P["spec_hold"]
    lines += [
        f"## Pair {i}: `{P['base_key']}`",
        f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`",
        f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}",
        f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`",
        f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')} {sh.get('value','?')}`  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}",
        ""
    ]
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")
print(f"[seal] refreshed → {SEAL_YAML.name}, {SEAL_MD.name}")

# Bundle everything (report + state + manifests + seal + docs)
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"
with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    if Path(REP).exists():     z.write(REP, arcname=Path(REP).name)
    if STATE.exists():         z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():  z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists(): z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    if SEAL_YAML.exists():     z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    if SEAL_MD.exists():       z.write(SEAL_MD, arcname=SEAL_MD.name)
    for doc in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
        if doc.exists(): z.write(doc, arcname=doc.name)

print(f"[bundle] {BUNDLE}")


[atlas] minted/updated pairs: 0 → E:\CNT\notes\atlas_pairs.md
{
  "score": {
    "clarity": 0.7989761904761905,
    "novelty": 1.0,
    "coherence": 0.8896968888565527,
    "falsifiability": 1.0,
    "total": 0.9221682698331858
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 12,
  "tests_passed": 12,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9505677925712496,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.7949045085606743,
      "hint": "#

In [43]:
# ===========================================
# CNT — Atlas Deep Sweep (pairs + seal) — ONE CELL
# ===========================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, json, zipfile, hashlib, glob as globmod

# ---------- ROOTS ----------
LAB   = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART   = LAB / r"artifacts\cnt_engine_megacell"
OUT   = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

# sweep ALL timestamped atlas dirs
ATLAS_GLOB = str(LAB / r"notebooks\archive\cnt_3i_atlas_all8*")
DOC_ATLAS  = LAB / r"notes\atlas_pairs.md"  # where we’ll write the tests
DOC_EWB    = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL   = LAB / r"notes\cooling_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---------- REQUIRE MEGA-CELL CORE ----------
try:
    run_cycle
    TEST_FENCE
except NameError:
    raise RuntimeError("Load your Mega Cell first so run_cycle() and TEST_FENCE exist.")

# ---------- HELPERS ----------
def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

# pick meaningful metric columns if present
METRIC_PREFS = ["auc","roc_auc","auroc","ap","aupr","f1","f1_score","precision","recall","accuracy"]

def _pick_metric_col(df: pd.DataFrame, prefer=None):
    nm = {_norm(c): c for c in df.columns}
    prefs = ([prefer] if prefer else []) + METRIC_PREFS
    for want in prefs:
        if want and _norm(want) in nm:
            return nm[_norm(want)]
    # fallback: first numeric column
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any():
            return c
    return None

def _policy_value(series, mode="relative_90pct"):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return None
    if mode == "relative_90pct": return float(round(0.9*float(s.mean()), 6))
    if mode == "above_chance":   return 0.50
    if mode == "strict_055":     return 0.55
    return float(round(0.9*float(s.mean()), 6))

def _fence(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False, allow_unicode=True) + "```"

def _upsert_fenced(doc_text: str, name: str, block: str) -> str:
    out, replaced = doc_text, False
    for m in TEST_FENCE.finditer(doc_text or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and str(spec.get("name","")).strip().lower() == name.strip().lower():
            a,b = m.span(); out = doc_text[:a] + block + doc_text[b:]; replaced=True; break
    if not replaced:
        out = (out.rstrip() + "\n\n" + block + "\n")
    return out

def _is_holdout_stem(stem: str) -> bool:
    return bool(re.search(r"(?i)(holdout|val|validation|test)", stem))

def _atlas_basekey(stem: str):
    # strip common holdout/test tokens and trailing run indices
    s = re.sub(r"(?i)([_\-](holdout|val|validation|test))+$", "", stem)
    s = re.sub(r"[_\-](v\d+|\d{6,})$", "", s)
    return s

def _load_specs_from(doc_path: Path):
    out = {}
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

# ---------- 1) SCAN ALL ATLAS CSVs (not just out/tables) ----------
atlas_dirs = [Path(p) for p in globmod.glob(ATLAS_GLOB) if Path(p).is_dir()]
csv_paths  = []
for d in atlas_dirs:
    csv_paths += [Path(p) for p in globmod.glob(str(d / "**" / "*.csv"), recursive=True)]

# Group by (nearest 'out' dir if present else parent) + base key
from collections import defaultdict
by_bucket = defaultdict(lambda: {"base": None, "hold": None, "all": []})

for p in csv_paths:
    stem = p.stem
    # bucket dir: prefer .../out/... if present
    parts = p.parts
    if "out" in [s.lower() for s in parts]:
        # slice up to 'out'
        idx = [s.lower() for s in parts].index("out")
        bucket_dir = Path(*parts[:idx+1])  # e.g., .../out
    else:
        bucket_dir = p.parent
    key = _atlas_basekey(stem)
    rec = by_bucket[(bucket_dir, key)]
    rec["all"].append(p)
    if _is_holdout_stem(stem):
        rec["hold"] = rec["hold"] or p
    else:
        rec["base"] = rec["base"] or p

# ---------- 2) MINT GOLD base↔holdout tests into atlas_pairs.md ----------
DOC_ATLAS.parent.mkdir(parents=True, exist_ok=True)
txt_atlas = DOC_ATLAS.read_text(encoding="utf-8", errors="ignore") if DOC_ATLAS.exists() else "# 3I Atlas — Paired Evidence\n"

minted_pairs = 0
for (bucket, key), rec in by_bucket.items():
    base, hold = rec["base"], rec["hold"]
    if not (base and hold and base.exists() and hold.exists()):
        continue
    try:
        dfb = pd.read_csv(base); dfh = pd.read_csv(hold)
    except Exception:
        continue

    col_b = _pick_metric_col(dfb, None)
    col_h = _pick_metric_col(dfh, col_b)
    if not col_b or not col_h:
        continue

    sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
    sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
    if sb.empty or sh.empty:
        continue

    thr_b = _policy_value(sb, "relative_90pct")
    thr_h = _policy_value(sh, "relative_90pct")

    # unique, stable names per bucket+key
    bslug = _norm(str(bucket).replace("\\","/").split("/")[-1] + "_" + key)[:48]
    name_base = f"pair_atlas_{bslug}"
    name_hold = f"{name_base}_holdout"

    spec_b = dict(
        name=name_base, type="csv_threshold",
        path=str(PureWindowsPath(base)), column=col_b, op=">", value=thr_b, agg="mean",
        gold=True, frozen_at=STAMP, hash=_sha256(base), rows=int(len(dfb))
    )
    spec_h = dict(
        name=name_hold, type="csv_threshold",
        path=str(PureWindowsPath(hold)), column=col_h, op=">", value=thr_h, agg="mean",
        gold=True, frozen_at=STAMP, hash=_sha256(hold), rows=int(len(dfh))
    )

    txt_atlas = _upsert_fenced(txt_atlas, name_base, _fence(spec_b))
    txt_atlas = _upsert_fenced(txt_atlas, name_hold, _fence(spec_h))
    minted_pairs += 1

DOC_ATLAS.write_text(txt_atlas, encoding="utf-8")
print(f"[atlas] scanned_dirs={len(atlas_dirs)} csvs={len(csv_paths)} minted_pairs={minted_pairs} → {DOC_ATLAS}")

# ---------- 3) RUN ENGINE (paired-evidence runner assumed already patched) ----------
res = run_cycle()
print(json.dumps(res, indent=2))

# ---------- 4) REFRESH SEAL (pairs-only) + BUNDLE ----------
# Use latest report to gather true pairs
rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1])
dfrep = pd.read_csv(REP)

def _is_hold(nm): return str(nm).lower().strip().endswith("_holdout")
def _base_key(nm):
    s = str(nm).strip()
    return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

ok = dfrep[(dfrep["reason"]=="ok") & (dfrep["valid"]==1) & (dfrep["passed"]==1)].copy()
ok["is_holdout"] = ok["name"].apply(_is_hold).astype(int)

# name -> spec across docs we care about
def _load_specs_from(doc_path: Path):
    out = {}
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

name2spec = {}
doc_candidates = set(ok["doc"].tolist())
for extra in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
    if extra.exists(): doc_candidates.add(str(extra))
for d in doc_candidates:
    name2spec.update(_load_specs_from(Path(d)))

def _enrich(spec):
    out = dict(spec or {})
    p = Path(out.get("path",""))
    if p.exists() and not out.get("hash"):
        out["hash"] = _sha256(p)
    if p.exists() and not out.get("rows"):
        try: out["rows"] = int(len(pd.read_csv(p)))
        except Exception: pass
    return out

pairs_payload=[]
for k, g in ok.groupby(ok["name"].apply(_base_key)):
    gb = g[g["is_holdout"]==0]
    gh = g[g["is_holdout"]==1]
    if gb.empty or gh.empty: 
        continue
    rb, rh = gb.iloc[0], gh.iloc[0]
    sb = _enrich(name2spec.get(rb["name"], {}))
    sh = _enrich(name2spec.get(rh["name"], {}))
    pairs_payload.append(dict(
        base_key=k,
        doc_base=str(rb["doc"]), doc_hold=str(rh["doc"]),
        spec_base=sb, spec_hold=sh,
        report_rows=[rb.to_dict(), rh.to_dict()],
    ))

SEAL = dict(
    seal_version="CNT-Proof-1",
    created_at=STAMP,
    lab_root=str(LAB),
    engine_state=(yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file=Path(REP).name,
    pairs_ok=len(pairs_payload),
    pairs=pairs_payload,
)

SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"
with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

lines = [f"# CNT Proof Seal — Pairs-Only",
         f"- Created: {STAMP}",
         f"- Lab   : {LAB}",
         f"- Pairs : {len(pairs_payload)}",
         ""]
for i,P in enumerate(pairs_payload,1):
    sb,sh=P["spec_base"],P["spec_hold"]
    lines += [
        f"## Pair {i}: `{P['base_key']}`",
        f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`",
        f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}",
        f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`",
        f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')}` {sh.get('value','?')}  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}",
        ""
    ]
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")
print(f"[seal] refreshed → {SEAL_YAML.name}, {SEAL_MD.name}")

# ---------- 5) BUNDLE everything ----------
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"
with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    if Path(REP).exists():     z.write(REP, arcname=Path(REP).name)
    if STATE.exists():         z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():  z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists(): z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    if SEAL_YAML.exists():     z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    if SEAL_MD.exists():       z.write(SEAL_MD, arcname=SEAL_MD.name)
    for doc in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
        if doc.exists(): z.write(doc, arcname=doc.name)

print(f"[bundle] {BUNDLE}")


[atlas] scanned_dirs=12 csvs=131 minted_pairs=0 → E:\CNT\notes\atlas_pairs.md
{
  "score": {
    "clarity": 0.7989761904761905,
    "novelty": 1.0,
    "coherence": 0.8896968888565527,
    "falsifiability": 0.7571428571428571,
    "total": 0.8614539841189001
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9505677925712496,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.79490

In [44]:
# ===========================================
# CNT — Atlas Deep Sweep (pairs + seal) — ONE CELL
# ===========================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, json, zipfile, hashlib, glob as globmod

# ---------- ROOTS ----------
LAB   = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART   = LAB / r"artifacts\cnt_engine_megacell"
OUT   = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

# sweep ALL timestamped atlas dirs
ATLAS_GLOB = str(LAB / r"notebooks\archive\cnt_3i_atlas_all8*")
DOC_ATLAS  = LAB / r"notes\atlas_pairs.md"  # where we’ll write the tests
DOC_EWB    = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL   = LAB / r"notes\cooling_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---------- REQUIRE MEGA-CELL CORE ----------
try:
    run_cycle
    TEST_FENCE
except NameError:
    raise RuntimeError("Load your Mega Cell first so run_cycle() and TEST_FENCE exist.")

# ---------- HELPERS ----------
def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

# pick meaningful metric columns if present
METRIC_PREFS = ["auc","roc_auc","auroc","ap","aupr","f1","f1_score","precision","recall","accuracy"]

def _pick_metric_col(df: pd.DataFrame, prefer=None):
    nm = {_norm(c): c for c in df.columns}
    prefs = ([prefer] if prefer else []) + METRIC_PREFS
    for want in prefs:
        if want and _norm(want) in nm:
            return nm[_norm(want)]
    # fallback: first numeric column
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any():
            return c
    return None

def _policy_value(series, mode="relative_90pct"):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return None
    if mode == "relative_90pct": return float(round(0.9*float(s.mean()), 6))
    if mode == "above_chance":   return 0.50
    if mode == "strict_055":     return 0.55
    return float(round(0.9*float(s.mean()), 6))

def _fence(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False, allow_unicode=True) + "```"

def _upsert_fenced(doc_text: str, name: str, block: str) -> str:
    out, replaced = doc_text, False
    for m in TEST_FENCE.finditer(doc_text or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and str(spec.get("name","")).strip().lower() == name.strip().lower():
            a,b = m.span(); out = doc_text[:a] + block + doc_text[b:]; replaced=True; break
    if not replaced:
        out = (out.rstrip() + "\n\n" + block + "\n")
    return out

def _is_holdout_stem(stem: str) -> bool:
    return bool(re.search(r"(?i)(holdout|val|validation|test)", stem))

def _atlas_basekey(stem: str):
    # strip common holdout/test tokens and trailing run indices
    s = re.sub(r"(?i)([_\-](holdout|val|validation|test))+$", "", stem)
    s = re.sub(r"[_\-](v\d+|\d{6,})$", "", s)
    return s

def _load_specs_from(doc_path: Path):
    out = {}
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

# ---------- 1) SCAN ALL ATLAS CSVs (not just out/tables) ----------
atlas_dirs = [Path(p) for p in globmod.glob(ATLAS_GLOB) if Path(p).is_dir()]
csv_paths  = []
for d in atlas_dirs:
    csv_paths += [Path(p) for p in globmod.glob(str(d / "**" / "*.csv"), recursive=True)]

# Group by (nearest 'out' dir if present else parent) + base key
from collections import defaultdict
by_bucket = defaultdict(lambda: {"base": None, "hold": None, "all": []})

for p in csv_paths:
    stem = p.stem
    # bucket dir: prefer .../out/... if present
    parts = p.parts
    if "out" in [s.lower() for s in parts]:
        # slice up to 'out'
        idx = [s.lower() for s in parts].index("out")
        bucket_dir = Path(*parts[:idx+1])  # e.g., .../out
    else:
        bucket_dir = p.parent
    key = _atlas_basekey(stem)
    rec = by_bucket[(bucket_dir, key)]
    rec["all"].append(p)
    if _is_holdout_stem(stem):
        rec["hold"] = rec["hold"] or p
    else:
        rec["base"] = rec["base"] or p

# ---------- 2) MINT GOLD base↔holdout tests into atlas_pairs.md ----------
DOC_ATLAS.parent.mkdir(parents=True, exist_ok=True)
txt_atlas = DOC_ATLAS.read_text(encoding="utf-8", errors="ignore") if DOC_ATLAS.exists() else "# 3I Atlas — Paired Evidence\n"

minted_pairs = 0
for (bucket, key), rec in by_bucket.items():
    base, hold = rec["base"], rec["hold"]
    if not (base and hold and base.exists() and hold.exists()):
        continue
    try:
        dfb = pd.read_csv(base); dfh = pd.read_csv(hold)
    except Exception:
        continue

    col_b = _pick_metric_col(dfb, None)
    col_h = _pick_metric_col(dfh, col_b)
    if not col_b or not col_h:
        continue

    sb = pd.to_numeric(dfb[col_b], errors="coerce").dropna()
    sh = pd.to_numeric(dfh[col_h], errors="coerce").dropna()
    if sb.empty or sh.empty:
        continue

    thr_b = _policy_value(sb, "relative_90pct")
    thr_h = _policy_value(sh, "relative_90pct")

    # unique, stable names per bucket+key
    bslug = _norm(str(bucket).replace("\\","/").split("/")[-1] + "_" + key)[:48]
    name_base = f"pair_atlas_{bslug}"
    name_hold = f"{name_base}_holdout"

    spec_b = dict(
        name=name_base, type="csv_threshold",
        path=str(PureWindowsPath(base)), column=col_b, op=">", value=thr_b, agg="mean",
        gold=True, frozen_at=STAMP, hash=_sha256(base), rows=int(len(dfb))
    )
    spec_h = dict(
        name=name_hold, type="csv_threshold",
        path=str(PureWindowsPath(hold)), column=col_h, op=">", value=thr_h, agg="mean",
        gold=True, frozen_at=STAMP, hash=_sha256(hold), rows=int(len(dfh))
    )

    txt_atlas = _upsert_fenced(txt_atlas, name_base, _fence(spec_b))
    txt_atlas = _upsert_fenced(txt_atlas, name_hold, _fence(spec_h))
    minted_pairs += 1

DOC_ATLAS.write_text(txt_atlas, encoding="utf-8")
print(f"[atlas] scanned_dirs={len(atlas_dirs)} csvs={len(csv_paths)} minted_pairs={minted_pairs} → {DOC_ATLAS}")

# ---------- 3) RUN ENGINE (paired-evidence runner assumed already patched) ----------
res = run_cycle()
print(json.dumps(res, indent=2))

# ---------- 4) REFRESH SEAL (pairs-only) + BUNDLE ----------
# Use latest report to gather true pairs
rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1])
dfrep = pd.read_csv(REP)

def _is_hold(nm): return str(nm).lower().strip().endswith("_holdout")
def _base_key(nm):
    s = str(nm).strip()
    return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

ok = dfrep[(dfrep["reason"]=="ok") & (dfrep["valid"]==1) & (dfrep["passed"]==1)].copy()
ok["is_holdout"] = ok["name"].apply(_is_hold).astype(int)

# name -> spec across docs we care about
def _load_specs_from(doc_path: Path):
    out = {}
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

name2spec = {}
doc_candidates = set(ok["doc"].tolist())
for extra in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
    if extra.exists(): doc_candidates.add(str(extra))
for d in doc_candidates:
    name2spec.update(_load_specs_from(Path(d)))

def _enrich(spec):
    out = dict(spec or {})
    p = Path(out.get("path",""))
    if p.exists() and not out.get("hash"):
        out["hash"] = _sha256(p)
    if p.exists() and not out.get("rows"):
        try: out["rows"] = int(len(pd.read_csv(p)))
        except Exception: pass
    return out

pairs_payload=[]
for k, g in ok.groupby(ok["name"].apply(_base_key)):
    gb = g[g["is_holdout"]==0]
    gh = g[g["is_holdout"]==1]
    if gb.empty or gh.empty: 
        continue
    rb, rh = gb.iloc[0], gh.iloc[0]
    sb = _enrich(name2spec.get(rb["name"], {}))
    sh = _enrich(name2spec.get(rh["name"], {}))
    pairs_payload.append(dict(
        base_key=k,
        doc_base=str(rb["doc"]), doc_hold=str(rh["doc"]),
        spec_base=sb, spec_hold=sh,
        report_rows=[rb.to_dict(), rh.to_dict()],
    ))

SEAL = dict(
    seal_version="CNT-Proof-1",
    created_at=STAMP,
    lab_root=str(LAB),
    engine_state=(yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file=Path(REP).name,
    pairs_ok=len(pairs_payload),
    pairs=pairs_payload,
)

SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"
with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

lines = [f"# CNT Proof Seal — Pairs-Only",
         f"- Created: {STAMP}",
         f"- Lab   : {LAB}",
         f"- Pairs : {len(pairs_payload)}",
         ""]
for i,P in enumerate(pairs_payload,1):
    sb,sh=P["spec_base"],P["spec_hold"]
    lines += [
        f"## Pair {i}: `{P['base_key']}`",
        f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`",
        f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}",
        f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`",
        f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')}` {sh.get('value','?')}  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}",
        ""
    ]
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")
print(f"[seal] refreshed → {SEAL_YAML.name}, {SEAL_MD.name}")

# ---------- 5) BUNDLE everything ----------
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"
with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    if Path(REP).exists():     z.write(REP, arcname=Path(REP).name)
    if STATE.exists():         z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():  z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists(): z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    if SEAL_YAML.exists():     z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    if SEAL_MD.exists():       z.write(SEAL_MD, arcname=SEAL_MD.name)
    for doc in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
        if doc.exists(): z.write(doc, arcname=doc.name)

print(f"[bundle] {BUNDLE}")


[atlas] scanned_dirs=12 csvs=131 minted_pairs=0 → E:\CNT\notes\atlas_pairs.md
{
  "score": {
    "clarity": 0.7989761904761905,
    "novelty": 1.0,
    "coherence": 0.8896968888565527,
    "falsifiability": 0.7571428571428571,
    "total": 0.8614539841189001
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9505677925712496,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.79490

In [45]:
# ===========================================
# CNT — Atlas Pair Inference v3 (schema-based)
# ===========================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, json, zipfile, hashlib, itertools, math
import numpy as np
import glob as globmod

# ---------- ROOTS ----------
LAB   = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART   = LAB / r"artifacts\cnt_engine_megacell"
OUT   = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

ATLAS_GLOB = str(LAB / r"notebooks\archive\cnt_3i_atlas_all8*")
DOC_ATLAS  = LAB / r"notes\atlas_pairs.md"   # where Atlas tests will live
DOC_EWB    = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL   = LAB / r"notes\cooling_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---------- REQUIRE ENGINE CORE ----------
try:
    run_cycle
    TEST_FENCE
except NameError:
    raise RuntimeError("Load your Mega Cell first so run_cycle() and TEST_FENCE exist in this kernel.")

# ---------- TUNABLES ----------
MAX_PAIRS_PER_BUCKET   = 6      # cap new Atlas pairs per bucket so we don't spam
MIN_COL_JACCARD        = 0.60   # require at least 60% overlap of normalized column names
ROW_RATIO_MIN, ROW_RATIO_MAX = 0.5, 2.0  # similar table size guard
CALIBRATION_MODE       = "relative_90pct"  # 'relative_90pct' | 'above_chance' | 'strict_055'
METRIC_PREFS = ["auc","roc_auc","auroc","ap","aupr","f1","f1_score","precision","recall","accuracy"]

# ---------- HELPERS ----------
def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _metric_col(df: pd.DataFrame, prefer=None):
    nm = {_norm(c): c for c in df.columns}
    prefs = ([prefer] if prefer else []) + METRIC_PREFS
    for want in prefs:
        if want and _norm(want) in nm:
            return nm[_norm(want)]
    # fallback: first numeric column
    for c in df.columns:
        if pd.to_numeric(df[c], errors="coerce").notna().any():
            return c
    return None

def _policy_value(series, mode="relative_90pct"):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return None
    if mode == "relative_90pct": return float(round(0.9*float(s.mean()), 6))
    if mode == "above_chance":   return 0.50
    if mode == "strict_055":     return 0.55
    return float(round(0.9*float(s.mean()), 6))

def _fence(d: dict) -> str:
    return "```cnt-test\n" + yaml.safe_dump(d, sort_keys=False, allow_unicode=True) + "```"

def _upsert_fenced(doc_text: str, name: str, block: str) -> str:
    out, replaced = doc_text, False
    for m in TEST_FENCE.finditer(doc_text or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and str(spec.get("name","")).strip().lower() == name.strip().lower():
            a,b = m.span(); out = doc_text[:a] + block + doc_text[b:]; replaced=True; break
    if not replaced:
        out = (out.rstrip() + "\n\n" + block + "\n")
    return out

def _col_sig(df: pd.DataFrame):
    cols = [c for c in df.columns if pd.to_numeric(df[c], errors="coerce").notna().any() or df[c].dtype.kind in "biufcOS"]
    return set(_norm(c) for c in cols)

def _jaccard(a:set, b:set):
    if not a and not b: return 1.0
    return len(a & b) / max(1, len(a | b))

def _bucket_for(p: Path):
    # prefer .../out/... ancestor; else parent folder as bucket
    parts = [s.lower() for s in p.parts]
    if "out" in parts:
        idx = parts.index("out")
        return Path(*p.parts[:idx+1])
    return p.parent

# ---------- 1) COLLECT all Atlas CSVs across timestamped dirs ----------
atlas_dirs = [Path(p) for p in globmod.glob(ATLAS_GLOB) if Path(p).is_dir()]
csv_paths  = []
for d in atlas_dirs:
    csv_paths += [Path(p) for p in globmod.glob(str(d / "**" / "*.csv"), recursive=True)]

if not csv_paths:
    print("[atlas] no CSVs found under", ATLAS_GLOB)

# ---------- 2) BUILD per-file descriptors (row count, col signature, preferred metric) ----------
files = []
for p in csv_paths:
    try:
        df = pd.read_csv(p, nrows=500)  # cheap header/sample read for sig/metric
        sig = _col_sig(df)
        nrows_full = None
        try:
            # fast row count without loading entire file
            nrows_full = sum(1 for _ in open(p, "r", encoding="utf-8", errors="ignore")) - 1
        except Exception:
            pass
        metric = _metric_col(df, None)
        files.append(dict(path=p, bucket=_bucket_for(p), rows=nrows_full, sig=sig, metric=metric))
    except Exception:
        continue

# group by bucket
from collections import defaultdict
by_bucket = defaultdict(list)
for f in files:
    by_bucket[f["bucket"]].append(f)

# ---------- 3) INFER pairs inside each bucket by schema similarity ----------
def pair_candidates(infos):
    # compute scores for all pairs; greedy select non-overlapping best
    scored = []
    for i, j in itertools.combinations(range(len(infos)), 2):
        A, B = infos[i], infos[j]
        # basic filters
        if not A["sig"] or not B["sig"]:
            continue
        jac = _jaccard(A["sig"], B["sig"])
        if jac < MIN_COL_JACCARD:
            continue
        # row ratio close-ish
        ra = A["rows"] or 0; rb = B["rows"] or 0
        ok_rows = (ra==0 or rb==0) or (ROW_RATIO_MIN <= (ra / rb if rb else 1) <= ROW_RATIO_MAX)
        if not ok_rows:
            continue
        # metric that exists in BOTH (prefer A.metric/B.metric; otherwise any intersecting numeric col)
        common_cols = list(A["sig"] & B["sig"])
        preferred = None
        for pref in ([A["metric"], B["metric"]] + METRIC_PREFS):
            if pref and _norm(pref) in common_cols:
                preferred = pref; break
        has_metric = preferred is not None
        score = jac + (0.25 if has_metric else 0.0) + (0.05 if ("holdout" in str(B["path"]).lower() or "val" in str(B["path"]).lower() or "test" in str(B["path"]).lower()) else 0.0)
        scored.append((score, i, j, preferred))
    scored.sort(reverse=True, key=lambda t: t[0])
    # greedy non-overlap
    used = set()
    pairs = []
    for score, i, j, pref in scored:
        if i in used or j in used:
            continue
        pairs.append((infos[i], infos[j], pref, score))
        used.add(i); used.add(j)
        if len(pairs) >= MAX_PAIRS_PER_BUCKET:
            break
    return pairs

# ---------- 4) MINT tests for accepted pairs ----------
DOC_ATLAS.parent.mkdir(parents=True, exist_ok=True)
txt_atlas = DOC_ATLAS.read_text(encoding="utf-8", errors="ignore") if DOC_ATLAS.exists() else "# 3I Atlas — Paired Evidence\n"

minted_pairs = 0
mint_log = []

for bucket, infos in by_bucket.items():
    pairs = pair_candidates(infos)
    for A, B, pref, score in pairs:
        try:
            dfA = pd.read_csv(A["path"])
            dfB = pd.read_csv(B["path"])
        except Exception:
            continue
        colA = _metric_col(dfA, pref)
        colB = _metric_col(dfB, colA)
        if not colA or not colB:
            continue
        sA = pd.to_numeric(dfA[colA], errors="coerce").dropna()
        sB = pd.to_numeric(dfB[colB], errors="coerce").dropna()
        if sA.empty or sB.empty:
            continue

        thrA = _policy_value(sA, CALIBRATION_MODE)
        thrB = _policy_value(sB, CALIBRATION_MODE)

        # stable slug for names (bucket last dir + common stem-ish)
        bslug = _norm(str(bucket).replace("\\","/").split("/")[-1])
        stem_common = _norm(Path(A["path"]).stem.split("_")[0] + "_" + Path(B["path"]).stem.split("_")[0])[:32]
        slug = (bslug + "_" + stem_common)[:48]

        name_base = f"pair_atlas_{slug}_a"
        name_hold = f"pair_atlas_{slug}_b"

        specA = dict(
            name=name_base, type="csv_threshold",
            path=str(PureWindowsPath(A["path"])), column=colA, op=">", value=thrA, agg="mean",
            gold=True, frozen_at=STAMP, hash=_sha256(A["path"]), rows=int(len(dfA))
        )
        specB = dict(
            name=name_hold, type="csv_threshold",
            path=str(PureWindowsPath(B["path"])), column=colB, op=">", value=thrB, agg="mean",
            gold=True, frozen_at=STAMP, hash=_sha256(B["path"]), rows=int(len(dfB))
        )

        txt_atlas = _upsert_fenced(txt_atlas, name_base, _fence(specA))
        txt_atlas = _upsert_fenced(txt_atlas, name_hold, _fence(specB))
        minted_pairs += 1
        mint_log.append(dict(bucket=str(bucket), base=str(A["path"]), hold=str(B["path"]), colA=colA, colB=colB, score=round(score,3)))

DOC_ATLAS.write_text(txt_atlas, encoding="utf-8")
print(f"[atlas] schema-inferred pairs minted: {minted_pairs} → {DOC_ATLAS}")
if mint_log:
    df_minted = pd.DataFrame(mint_log)
    try:
        from IPython.display import display
        display(df_minted.head(12))
    except Exception:
        print(df_minted.head(12).to_string(index=False))

# ---------- 5) RUN ENGINE (paired-evidence runner assumed already patched) ----------
res = run_cycle()
print(json.dumps(res, indent=2))

# ---------- 6) REFRESH SEAL + BUNDLE ----------
def _load_specs_from(doc_path: Path):
    out = {}
    try:
        txt = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(txt or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

# Gather true counted pairs from latest report
rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1])
dfrep = pd.read_csv(REP)

def _is_hold(nm): return str(nm).lower().strip().endswith("_holdout")
def _base_key(nm):
    s = str(nm).strip()
    return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

ok = dfrep[(dfrep["reason"]=="ok") & (dfrep["valid"]==1) & (dfrep["passed"]==1)].copy()
ok["is_holdout"] = ok["name"].apply(_is_hold).astype(int)

# name -> spec from docs
name2spec = {}
doc_candidates = set(ok["doc"].tolist())
for extra in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
    if extra.exists(): doc_candidates.add(str(extra))
for d in doc_candidates:
    name2spec.update(_load_specs_from(Path(d)))

def _enrich(spec):
    out = dict(spec or {})
    p = Path(out.get("path",""))
    if p.exists() and not out.get("hash"):
        out["hash"] = _sha256(p)
    if p.exists() and not out.get("rows"):
        try: out["rows"] = int(len(pd.read_csv(p)))
        except Exception: pass
    return out

pairs_payload=[]
for k, g in ok.groupby(ok["name"].apply(_base_key)):
    gb = g[g["is_holdout"]==0]
    gh = g[g["is_holdout"]==1]
    if gb.empty or gh.empty: 
        continue
    rb, rh = gb.iloc[0], gh.iloc[0]
    sb = _enrich(name2spec.get(rb["name"], {}))
    sh = _enrich(name2spec.get(rh["name"], {}))
    pairs_payload.append(dict(
        base_key=k,
        doc_base=str(rb["doc"]), doc_hold=str(rh["doc"]),
        spec_base=sb, spec_hold=sh,
        report_rows=[rb.to_dict(), rh.to_dict()],
    ))

SEAL = dict(
    seal_version="CNT-Proof-1",
    created_at=STAMP,
    lab_root=str(LAB),
    engine_state=(yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file=Path(REP).name,
    pairs_ok=len(pairs_payload),
    pairs=pairs_payload,
)

SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"
with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

lines = [f"# CNT Proof Seal — Pairs-Only",
         f"- Created: {STAMP}",
         f"- Lab   : {LAB}",
         f"- Pairs : {len(pairs_payload)}",
         ""]
for i,P in enumerate(pairs_payload,1):
    sb,sh=P["spec_base"],P["spec_hold"]
    lines += [
        f"## Pair {i}: `{P['base_key']}`",
        f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`",
        f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}",
        f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`",
        f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')} {sh.get('value','?')}`  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}",
        ""
    ]
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")
print(f"[seal] refreshed → {SEAL_YAML.name}, {SEAL_MD.name}")

# Bundle everything
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"
with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    if Path(REP).exists():     z.write(REP, arcname=Path(REP).name)
    if STATE.exists():         z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():  z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists(): z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    if SEAL_YAML.exists():     z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    if SEAL_MD.exists():       z.write(SEAL_MD, arcname=SEAL_MD.name)
    for doc in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
        if doc.exists(): z.write(doc, arcname=doc.name)

print(f"[bundle] {BUNDLE}")


[atlas] schema-inferred pairs minted: 19 → E:\CNT\notes\atlas_pairs.md


,bucket,base,hold,colA,colB,score
0,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,wavelength_nm,wavelength_nm,1.25
1,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,EW_389.0nm,EW_389.0nm,1.25
2,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,wavelength_nm,wavelength_nm,1.25
3,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,EW_389.0nm,EW_389.0nm,1.25
4,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,wavelength_nm,wavelength_nm,1.25
5,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,EW_389.0nm,EW_389.0nm,1.25
6,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,wavelength_nm,wavelength_nm,1.25
7,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,wavelength_nm,wavelength_nm,1.25
8,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,EW_388.3nm,EW_388.3nm,1.25
9,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,E:\CNT\notebooks\archive\cnt_3i_atlas_all8_202...,wavelength_nm,wavelength_nm,1.25


{
  "score": {
    "clarity": 0.7956333333333334,
    "novelty": 1.0,
    "coherence": 0.8834342625939264,
    "falsifiability": 0.7666666666666666,
    "total": 0.8614335656484816
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9987418722285857,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.9341605828009674,
      "hint": "# Gauge-Restored Agents (GRA) v0.3 **Contract:** I

In [46]:
# ================================
# CNT — Atlas Pair Renamer (a/b → base/_holdout) + Re-Run + Seal
# ================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, json, glob as globmod, hashlib, zipfile

LAB = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART = LAB / r"artifacts\cnt_engine_megacell"
OUT = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"

DOC_ATLAS = LAB / r"notes\atlas_pairs.md"
DOC_EWB   = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL  = LAB / r"notes\cooling_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# Need the mega-cell’s parser/runner
try:
    TEST_FENCE, run_cycle
except NameError:
    TEST_FENCE = re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL)
    raise RuntimeError("Load your main Mega Cell first so TEST_FENCE and run_cycle are defined.")

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p,"rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

# --- 1) Read atlas_pairs.md and collect pair_atlas_*_(a|b) tests
if not DOC_ATLAS.exists():
    raise SystemExit(f"Atlas doc not found: {DOC_ATLAS}")

txt = DOC_ATLAS.read_text(encoding="utf-8", errors="ignore")
pat_name = re.compile(r"^(pair_atlas_.+?)_(a|b)$", re.IGNORECASE)
hold_tokens = re.compile(r"(?i)\b(holdout|oos|oot|out[-_]?of[-_]?sample|val|validation|test|dev|eval)\b")

blocks = []
for m in TEST_FENCE.finditer(txt or ""):
    body = m.group("body")
    try:
        spec = yaml.safe_load(body) or {}
    except Exception:
        continue
    if not isinstance(spec, dict) or spec.get("type") != "csv_threshold":
        continue
    nm = str(spec.get("name","")).strip()
    mm = pat_name.match(nm)
    if mm:
        slug, tag = mm.group(1), mm.group(2).lower()
        blocks.append(dict(slug=slug, tag=tag, name=nm, span=m.span(), spec=spec))

# Group by slug, expect 2 per group
from collections import defaultdict
by_slug = defaultdict(list)
for b in blocks:
    by_slug[b["slug"]].append(b)

# --- 2) Decide base vs holdout for each slug, rename accordingly, keep GOLD metadata
changes = []
renamed_pairs = 0

def pick_hold(a, b):
    # Prefer file/name with holdout tokens
    def score(item):
        s = 0
        p = str(item["spec"].get("path",""))
        n = item["name"]
        if hold_tokens.search(p): s += 2
        if hold_tokens.search(n): s += 1
        # Light heuristic: smaller mean metric might be holdout (optional)
        col = item["spec"].get("column")
        if p and col and Path(p).exists():
            try:
                v = pd.to_numeric(pd.read_csv(p)[col], errors="coerce").dropna().mean()
                item["_mean"] = float(v)
            except Exception:
                pass
        return s
    sa, sb = score(a), score(b)
    if sa > sb: return a, b  # (hold, base)
    if sb > sa: return b, a
    # Fallback: tag 'b' as holdout
    return (b, a) if b["tag"] == "b" else (a, b)

for slug, items in by_slug.items():
    if len(items) < 2:
        continue
    # two items: choose hold/base
    hold, base = pick_hold(items[0], items[1])
    # new names
    base_name = slug
    hold_name = f"{slug}_holdout"
    # Build fenced blocks
    base_spec = dict(base["spec"]); base_spec["name"] = base_name
    hold_spec = dict(hold["spec"]); hold_spec["name"] = hold_name
    # Ensure GOLD metadata completeness (hash/rows)
    for sp in (base_spec, hold_spec):
        p = Path(sp.get("path",""))
        if p.exists() and not sp.get("hash"): sp["hash"] = _sha256(p)
        if p.exists() and not sp.get("rows"):
            try: sp["rows"] = int(len(pd.read_csv(p)))
            except Exception: pass
        sp["gold"] = True
    # Fenced strings
    fb = "```cnt-test\n" + yaml.safe_dump(base_spec, sort_keys=False, allow_unicode=True) + "```"
    fh = "```cnt-test\n" + yaml.safe_dump(hold_spec, sort_keys=False, allow_unicode=True) + "```"
    # Queue replacements at their original spans
    # Replace the exact matched blocks (order reversed later)
    for it, rep in [(base, fb), (hold, fh)]:
        changes.append((it["span"][0], it["span"][1], rep))
    renamed_pairs += 1

# Apply changes (reverse spans to keep indices stable)
if changes:
    new_txt = txt
    for a,b,rep in sorted(changes, key=lambda t: t[0], reverse=True):
        new_txt = new_txt[:a] + rep + new_txt[b:]
    DOC_ATLAS.write_text(new_txt, encoding="utf-8")
print(f"[atlas-rename] slugs_seen={len(by_slug)} renamed_pairs={renamed_pairs}")

# --- 3) Re-run engine once (paired-evidence runner already patched earlier)
res = run_cycle()
print(json.dumps(res, indent=2))

# --- 4) Pair dashboard (robust)
rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1])
df  = pd.read_csv(REP)

def _is_hold(name): return str(name).lower().strip().endswith("_holdout")
if "base_key" not in df.columns:
    df["base_key"] = df["name"].apply(lambda s: _norm(s[:-9]) if _is_hold(s) else _norm(s))
df["is_hold"]   = df["name"].apply(_is_hold).astype(int)
df["ok_row"]    = (df["reason"].eq("ok")).astype(int)
df["base_pass"] = ((df["is_hold"]==0) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)
df["hold_pass"] = ((df["is_hold"]==1) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)

dash = df.groupby("base_key").agg(
    base_ok=("base_pass","max"),
    hold_ok=("hold_pass","max"),
    counted=("valid","sum"),
    total=("name","count"),
)
dash["pair_ok"] = (dash["base_ok"] & dash["hold_ok"]).astype(int)

print("\n=== Pair Summary (after rename) ===")
try:
    from IPython.display import display
    display(dash.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(50))
except Exception:
    print(dash.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(50).to_string())

# --- 5) Refresh SEAL (pairs-only) + bundle
def _load_specs_from(doc_path: Path):
    out = {}
    try:
        t = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return out
    for m in TEST_FENCE.finditer(t or ""):
        try:
            spec = yaml.safe_load(m.group("body")) or {}
        except Exception:
            continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

def _enrich(spec):
    out = dict(spec or {})
    p = Path(out.get("path",""))
    if p.exists() and not out.get("hash"): out["hash"] = _sha256(p)
    if p.exists() and not out.get("rows"):
        try: out["rows"] = int(len(pd.read_csv(p)))
        except Exception: pass
    return out

ok = df[(df["reason"]=="ok") & (df["valid"]==1) & (df["passed"]==1)].copy()
ok["is_holdout"] = ok["name"].apply(_is_hold).astype(int)

# name->spec index from relevant docs
name2spec = {}
docs_use = set(ok["doc"].tolist())
for extra in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
    if extra.exists(): docs_use.add(str(extra))
for d in docs_use:
    name2spec.update(_load_specs_from(Path(d)))

pairs_payload=[]
for k, g in ok.groupby(df["name"].apply(lambda s: _norm(s[:-9]) if _is_hold(s) else _norm(s))):
    gb = g[g["is_holdout"]==0]
    gh = g[g["is_holdout"]==1]
    if gb.empty or gh.empty: 
        continue
    rb, rh = gb.iloc[0], gh.iloc[0]
    sb = _enrich(name2spec.get(rb["name"], {}))
    sh = _enrich(name2spec.get(rh["name"], {}))
    pairs_payload.append(dict(
        base_key=k,
        doc_base=str(rb["doc"]), doc_hold=str(rh["doc"]),
        spec_base=sb, spec_hold=sh,
        report_rows=[rb.to_dict(), rh.to_dict()],
    ))

SEAL = dict(
    seal_version="CNT-Proof-1",
    created_at=STAMP,
    lab_root=str(LAB),
    engine_state=(yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file=Path(REP).name,
    pairs_ok=len(pairs_payload),
    pairs=pairs_payload,
)

SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"
with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

lines = [f"# CNT Proof Seal — Pairs-Only",
         f"- Created: {STAMP}",
         f"- Lab   : {LAB}",
         f"- Pairs : {len(pairs_payload)}",
         ""]
for i,P in enumerate(pairs_payload,1):
    sb,sh=P["spec_base"],P["spec_hold"]
    lines += [
        f"## Pair {i}: `{P['base_key']}`",
        f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`",
        f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}",
        f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`",
        f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')} {sh.get('value','?')}`  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}",
        ""
    ]
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")
print(f"[seal] refreshed → {SEAL_YAML.name}, {SEAL_MD.name}")

# Bundle
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"
with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    if Path(REP).exists():     z.write(REP, arcname=Path(REP).name)
    if STATE.exists():         z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists():  z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists(): z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    if SEAL_YAML.exists():     z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    if SEAL_MD.exists():       z.write(SEAL_MD, arcname=SEAL_MD.name)
    for doc in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
        if doc.exists(): z.write(doc, arcname=doc.name)
print(f"[bundle] {BUNDLE}")


[atlas-rename] slugs_seen=2 renamed_pairs=2
{
  "score": {
    "clarity": 0.7956142857142857,
    "novelty": 1.0,
    "coherence": 0.8834342625939264,
    "falsifiability": 0.7666666666666666,
    "total": 0.8614288037437197
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9987418722285857,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.9341605828009674,
      "hint": "# Gaug

,base_ok,hold_ok,counted,total,pair_ok
base_key,,,,,
pairatlasdataspectrumspectru,0,0,0,1,0
pairatlasdataspectrumspectrum,0,0,0,1,0
pairatlasoutgragr,0,0,0,1,0
pairatlasoutgragra,0,0,0,1,0
pairewbcoolingsegments2025101723521,0,0,0,1,0
pairewbcoolingsegments20251017235217,0,0,0,1,0
pairewbsummary2025101723320,0,0,0,1,0
pairewbsummary20251017233201,0,0,0,1,0
pairewbsummaryv212025101723401,0,0,0,1,0


[seal] refreshed → CNT_SEAL.yaml, CNT_SEAL.md
[bundle] E:\CNT\artifacts\cnt_engine_megacell\bundles\cnt_proof_bundle_20251026-231719_SEAL.zip


In [47]:
# ===========================================
# CNT — Recovery & Pair Count Fix (ONE CELL)
# ===========================================
from pathlib import Path, PureWindowsPath
from datetime import datetime, timezone
import os, re, yaml, pandas as pd, json, hashlib, glob as globmod, zipfile

LAB   = Path(os.getenv("CNT_LAB_DIR", r"E:\CNT")).resolve()
ART   = LAB / r"artifacts\cnt_engine_megacell"
OUT   = ART / "out"
STATE = ART / "CNT_STATE.yaml"
MANIFEST_CSV  = ART / "gold_manifest.csv"
MANIFEST_YAML = ART / "gold_manifest.yaml"
DOC_EWB  = LAB / r"notebooks\archive\cnt_ewb_theta2\publish\ewb_publish_v0_2\ewb_results.md"
DOC_COOL = LAB / r"notes\cooling_pairs.md"
DOC_ATLAS= LAB / r"notes\atlas_pairs.md"

STAMP = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# ---- EXPECT MEGA-CELL CORE ----
try:
    run_cycle
except NameError:
    raise RuntimeError("Load your main Mega Cell first so run_cycle() exists in this kernel.")
TEST_FENCE = globals().get("TEST_FENCE", re.compile(r"```cnt-test\s*(?P<body>.*?)```", re.DOTALL))

# ---------- Helpers ----------
def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", str(s or "").strip().lower())

def _sha256(p: Path, chunk=1<<20):
    h = hashlib.sha256()
    with open(p, "rb") as f:
        for b in iter(lambda: f.read(chunk), b""):
            h.update(b)
    return h.hexdigest()

def _match_col(df: pd.DataFrame, requested: str):
    want = _norm(requested or "")
    if not want: return None
    nm = {_norm(c): c for c in df.columns}
    return nm.get(want)

# ---------- Robust core CSV tester (fallback) ----------
def _csv_threshold_test_core(t: dict):
    """
    Evaluates a csv_threshold spec with tolerant column matching.
    Returns (ok, info, reason) with reason in {"ok","skip","non_gold","missing_path","hash_mismatch","missing_column","no_numeric","invalid_type"}.
    """
    if t.get("skip", False):
        return False, "skip", "skip"

    path = t.get("path")
    col  = t.get("column")
    op   = str(t.get("op", ">"))
    try:  val = float(t.get("value", 0.0))
    except Exception: val = 0.0
    agg  = str(t.get("agg", "mean")).lower()

    if not path or not Path(path).exists():
        return False, "missing_path", "missing_path"

    # hash check if present
    if t.get("hash"):
        if _sha256(Path(path)) != str(t["hash"]):
            return False, "hash_mismatch", "hash_mismatch"

    df = pd.read_csv(path)
    col_actual = _match_col(df, col)
    if col_actual is None:
        # try a few common AUC/precision aliases
        for want in ["auc","roc_auc","auroc","precision@theta*","precision_at_theta","delta_theta","f1","accuracy"]:
            col_actual = _match_col(df, want)
            if col_actual: break
    if col_actual is None:
        return False, "missing_column", "missing_column"

    s = pd.to_numeric(df[col_actual], errors="coerce").dropna()
    if s.empty:
        return False, "no_numeric", "no_numeric"

    if   agg == "mean":   metric = float(s.mean())
    elif agg == "median": metric = float(s.median())
    elif agg == "any":    metric = bool((s > val).any()) if op in [">", ">="] else bool((s < val).any())
    elif agg == "all":    metric = bool((s > val).all()) if op in [">", ">="] else bool((s < val).all())
    else:                 metric = float(s.mean())

    def _cmp(a,b,op): return {">":a>b, ">=":a>=b, "<":a<b, "<=":a<=b}.get(op, a>b)

    if isinstance(metric, bool):
        ok, info = metric, f"{agg}({col_actual})→{ok}"
    else:
        ok, info = _cmp(metric, val, op), f"{agg}({col_actual})={metric:.6f} {op} {val}"

    return bool(ok), info, "ok"

# ---------- Patch into the live module that owns run_cycle ----------
import inspect
rc_mod = inspect.getmodule(run_cycle)

# Ensure the module exposes the core tester callable
setattr(rc_mod, "_csv_threshold_test", _csv_threshold_test_core)

# Strict wrapper: honors skip + gold + hash, delegates to core
def _csv_threshold_test_strict_patch(t: dict):
    if t.get("skip", False):     return False, "skip", "skip"
    if not t.get("gold", False): return False, "non_gold", "non_gold"
    # core tester also verifies path/hash/column
    return _csv_threshold_test_core(t)

# Soft paired runner: singles -> needs_holdout; base+hold must both pass
def _paired_runner_soft(df: pd.DataFrame, out_csv: Path):
    rows = []
    parse = getattr(rc_mod, "_parse_cnt_tests", globals().get("_parse_cnt_tests", lambda s: []))

    def _is_hold(name: str) -> bool:
        return str(name or "").strip().lower().endswith("_holdout")
    def _base_key(name: str) -> str:
        s = str(name or "").strip()
        return _norm(s[:-9] if s.lower().endswith("_holdout") else s)

    for _, r in df.iterrows():
        path  = r["path"]
        tests = parse(r.get("text",""))
        for t in tests:
            nm   = t.get("name", Path(path).stem)
            ttyp = t.get("type","unknown")
            if ttyp == "csv_threshold":
                ok, info, reason = _csv_threshold_test_strict_patch(t)
            else:
                ok, info, reason = False, f"unknown_type:{ttyp}", "invalid_type"
            rows.append(dict(
                doc=path, name=nm, type=ttyp,
                valid=int(reason=="ok"), passed=int(bool(ok and reason=="ok")),
                info=info, reason=reason,
                is_holdout=int(_is_hold(nm)), base_key=_base_key(nm),
            ))

    rep = pd.DataFrame(rows)
    if rep.empty:
        rep.to_csv(out_csv, index=False); return 0,0,0

    # Only strict-ok rows can be counted
    rep.loc[~rep["reason"].eq("ok"), ["valid","passed"]] = 0

    # Pair gating
    by_key = {k: list(idxs) for k, idxs in rep.groupby("base_key").indices.items()}
    to_pair_fail, to_needs = set(), set()

    for key, idxs in by_key.items():
        base_idxs = [i for i in idxs if rep.loc[i,"is_holdout"]==0]
        hold_idxs = [i for i in idxs if rep.loc[i,"is_holdout"]==1]
        base_pass = any(bool(rep.loc[i,"passed"]) for i in base_idxs)
        hold_pass = any(bool(rep.loc[i,"passed"]) for i in hold_idxs)

        if base_idxs and not hold_idxs:
            to_needs.update(base_idxs)
        elif hold_idxs and not base_idxs:
            to_needs.update(hold_idxs)
        else:
            if not (base_pass and hold_pass):
                to_pair_fail.update(base_idxs + hold_idxs)

    if to_needs:
        rep.loc[list(to_needs), ["valid","passed"]] = 0
        rep.loc[list(to_needs), "reason"] = "needs_holdout"
    if to_pair_fail:
        rep.loc[list(to_pair_fail), ["valid","passed"]] = 0
        rep.loc[list(to_pair_fail), "reason"] = "pair_fail"

    rep.to_csv(out_csv, index=False)
    total  = int(rep["valid"].sum())
    passed = int(rep["passed"].sum())
    runnable = int(len(rep))
    return passed, total, runnable

# Bind patched functions
setattr(rc_mod, "_csv_threshold_test_strict", _csv_threshold_test_strict_patch)
setattr(rc_mod, "run_cnt_tests_on_df", _paired_runner_soft)

# ---------- RUN once ----------
res = run_cycle()
print(json.dumps(res, indent=2))

# ---------- Reasons breakdown + compact dashboard ----------
rep_paths = sorted(globmod.glob(str(OUT / "tests_report_*.csv")))
assert rep_paths, f"No tests_report_* in {OUT}"
REP = Path(rep_paths[-1]); df = pd.read_csv(REP)

print("\nReasons breakdown:")
print(df["reason"].value_counts(dropna=False))

def _is_hold(nm): return str(nm).lower().strip().endswith("_holdout")
if "base_key" not in df.columns:
    df["base_key"] = df["name"].apply(lambda s: _norm(s[:-9]) if _is_hold(s) else _norm(s))
df["is_hold"]   = df["name"].apply(_is_hold).astype(int)
df["base_pass"] = ((df["is_hold"]==0) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)
df["hold_pass"] = ((df["is_hold"]==1) & (df["passed"]==1) & df["reason"].eq("ok")).astype(int)

dash = df.groupby("base_key").agg(
    base_ok=("base_pass","max"),
    hold_ok=("hold_pass","max"),
    counted=("valid","sum"),
    total=("name","count"),
)
dash["pair_ok"] = (dash["base_ok"] & dash["hold_ok"]).astype(int)
try:
    from IPython.display import display
    display(dash.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(40))
except Exception:
    print("\n=== Pair Summary ===")
    print(dash.sort_values(["pair_ok","counted","total"], ascending=[False,False,True]).head(40).to_string())

# ---------- Refresh SEAL (pairs-only) + bundle ----------
def _load_specs_from(doc_path: Path):
    out = {}
    try: t = doc_path.read_text(encoding="utf-8", errors="ignore")
    except Exception: return out
    for m in TEST_FENCE.finditer(t or ""):
        try: spec = yaml.safe_load(m.group("body")) or {}
        except Exception: continue
        if isinstance(spec, dict) and "name" in spec and "type" in spec:
            out[str(spec["name"]).strip()] = spec
    return out

# keep only counted ok rows
ok = df[(df["reason"]=="ok") & (df["valid"]==1) & (df["passed"]==1)].copy()
ok["is_holdout"] = ok["name"].apply(_is_hold).astype(int)

name2spec = {}
doc_candidates = set(ok["doc"].tolist())
for extra in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
    if extra.exists(): doc_candidates.add(str(extra))
for d in doc_candidates:
    name2spec.update(_load_specs_from(Path(d)))

def _enrich(spec):
    out = dict(spec or {})
    p = Path(out.get("path",""))
    if p.exists() and not out.get("hash"):
        out["hash"] = _sha256(p)
    if p.exists() and not out.get("rows"):
        try: out["rows"] = int(len(pd.read_csv(p)))
        except Exception: pass
    return out

pairs_payload=[]
for k, g in ok.groupby(df["base_key"] if "base_key" in df.columns else df["name"].apply(lambda s: _norm(s[:-9]) if _is_hold(s) else _norm(s))):
    gb = g[g["is_holdout"]==0]
    gh = g[g["is_holdout"]==1]
    if gb.empty or gh.empty: 
        continue
    rb, rh = gb.iloc[0], gh.iloc[0]
    sb = _enrich(name2spec.get(rb["name"], {}))
    sh = _enrich(name2spec.get(rh["name"], {}))
    pairs_payload.append(dict(
        base_key=k,
        doc_base=str(rb["doc"]), doc_hold=str(rh["doc"]),
        spec_base=sb, spec_hold=sh,
        report_rows=[rb.to_dict(), rh.to_dict()],
    ))

SEAL = dict(
    seal_version="CNT-Proof-1",
    created_at=STAMP,
    lab_root=str(LAB),
    engine_state=(yaml.safe_load(STATE.read_text(encoding="utf-8")) if STATE.exists() else {}),
    report_file=Path(REP).name,
    pairs_ok=len(pairs_payload),
    pairs=pairs_payload,
)

SEAL_YAML = ART / "CNT_SEAL.yaml"
SEAL_MD   = ART / "CNT_SEAL.md"
with open(SEAL_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(SEAL, f, sort_keys=False, allow_unicode=True)

lines = [f"# CNT Proof Seal — Pairs-Only",
         f"- Created: {STAMP}",
         f"- Lab   : {LAB}",
         f"- Pairs : {len(pairs_payload)}",
         ""]
for i,P in enumerate(pairs_payload,1):
    sb,sh=P["spec_base"],P["spec_hold"]
    lines += [
        f"## Pair {i}: `{P['base_key']}`",
        f"- Base   : **{sb.get('name','?')}**  \n  doc: `{P['doc_base']}`",
        f"  - csv: `{sb.get('path','?')}`  \n    col: `{sb.get('column','?')}`  \n    rule: `{sb.get('op','?')} {sb.get('value','?')}`  \n    hash: `{sb.get('hash','—')}`  \n    rows: {sb.get('rows','?')}",
        f"- Holdout: **{sh.get('name','?')}**  \n  doc: `{P['doc_hold']}`",
        f"  - csv: `{sh.get('path','?')}`  \n    col: `{sh.get('column','?')}`  \n    rule: `{sh.get('op','?')} {sh.get('value','?')}`  \n    hash: `{sh.get('hash','—')}`  \n    rows: {sh.get('rows','?')}",
        ""
    ]
SEAL_MD.write_text("\n".join(lines), encoding="utf-8")
print(f"[seal] refreshed → {SEAL_YAML.name}, {SEAL_MD.name}")

# Bundle
BUNDLES = ART / "bundles"; BUNDLES.mkdir(parents=True, exist_ok=True)
BUNDLE  = BUNDLES / f"cnt_proof_bundle_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}_SEAL.zip"
with zipfile.ZipFile(BUNDLE, "w", zipfile.ZIP_DEFLATED) as z:
    rep_path = Path(REP)
    if rep_path.exists():     z.write(rep_path, arcname=rep_path.name)
    if STATE.exists():        z.write(STATE, arcname=STATE.name)
    if MANIFEST_CSV.exists(): z.write(MANIFEST_CSV, arcname=MANIFEST_CSV.name)
    if MANIFEST_YAML.exists():z.write(MANIFEST_YAML, arcname=MANIFEST_YAML.name)
    if SEAL_YAML.exists():    z.write(SEAL_YAML, arcname=SEAL_YAML.name)
    if SEAL_MD.exists():      z.write(SEAL_MD,   arcname=SEAL_MD.name)
    for doc in [DOC_EWB, DOC_COOL, DOC_ATLAS]:
        if doc.exists(): z.write(doc, arcname=doc.name)
print(f"[bundle] {BUNDLE}")


{
  "score": {
    "clarity": 0.7956142857142857,
    "novelty": 1.0,
    "coherence": 0.8834342625939264,
    "falsifiability": 0.7666666666666666,
    "total": 0.8614288037437197
  },
  "proposed": 5,
  "accepted": 0,
  "tests_total": 0,
  "tests_passed": 0,
  "hidden": [
    {
      "path": "E:\\CNT\\notebooks\\archive\\cnt_correlates_report_20251015-164130.txt",
      "resid": 0.9987418722285857,
      "hint": "== CNT Correlates Audit (Fused) == Timestamp: 20251015-164130 - Cooling logs: 8 - Cosmo triage tables: 1 - EEG laterality tables: 2 - Kuramoto/Ising summaries: 0 - Kuramoto dispersion CSVs: 0 - Gray-Scott edge images: 1 - Scanned logs/text: 50 - Glyph label tables: 0 == Results == [GREY] Cooling: Temp oscillation amplitude predicts clock stability Need logs with GPU temp + GPU clock columns. Add b"
    },
    {
      "path": "E:\\CNT\\notebooks\\archive\\gra-v0.3\\README.md",
      "resid": 0.9341605828009674,
      "hint": "# Gauge-Restored Agents (GRA) v0.3 **Contract:** I

,base_ok,hold_ok,counted,total,pair_ok
base_key,,,,,
pairatlasdataspectrumspectru,0,0,0,1,0
pairatlasdataspectrumspectrum,0,0,0,1,0
pairatlasoutgragr,0,0,0,1,0
pairatlasoutgragra,0,0,0,1,0
pairewbcoolingsegments2025101723521,0,0,0,1,0
pairewbcoolingsegments20251017235217,0,0,0,1,0
pairewbsummary2025101723320,0,0,0,1,0
pairewbsummary20251017233201,0,0,0,1,0
pairewbsummaryv212025101723401,0,0,0,1,0


[seal] refreshed → CNT_SEAL.yaml, CNT_SEAL.md
[bundle] E:\CNT\artifacts\cnt_engine_megacell\bundles\cnt_proof_bundle_20251026-232425_SEAL.zip


In [ ]:
# MEGA CELL: safe backup -> sanity checks -> small canary run
# Drop this into your JupyterLab and run. It tries to be defensive and informative.

import os, sys, shutil, json, time, traceback
from pathlib import Path
from datetime import datetime
import subprocess

# --- CONFIG (edit if you want different targets) ---
LAB_ROOT = Path(os.getenv("CNT_LAB_DIR") or Path.cwd())  # auto-detect or current dir
BACKUP_DIR = LAB_ROOT.parent / f"{LAB_ROOT.name}_backups"
BACKUP_KEEP = 5   # how many previous backups to keep
ENGINE_CHECK_CMD = None  # optional shell command to run engine tests, e.g. "pytest -q"
CANARY_RUN_CELLS = 1      # how many tiny canary cycles to run (keeps short)
CANARY_NOTE = "canary_small_run"

# --- HELPERS ---
def now_tag():
    return datetime.utcnow().strftime("%Y%m%d-%H%M%S")

def make_zip_backup(src: Path, dest_dir: Path):
    dest_dir.mkdir(parents=True, exist_ok=True)
    stamp = now_tag()
    zip_name = dest_dir / f"{src.name}_backup_{stamp}.zip"
    print(f"Creating backup: {zip_name}")
    shutil.make_archive(str(zip_name.with_suffix('')), 'zip', root_dir=str(src))
    # rotate
    zips = sorted(dest_dir.glob(f"{src.name}_backup_*.zip"), key=lambda p: p.stat().st_mtime, reverse=True)
    for old in zips[BACKUP_KEEP:]:
        try:
            old.unlink()
            print(f"Pruned old backup: {old.name}")
        except Exception:
            pass
    return zip_name

def safe_call(func, *a, **kw):
    try:
        return ("ok", func(*a, **kw))
    except Exception as e:
        tb = traceback.format_exc()
        return ("err", (e, tb))

# --- STEP 0: quick readiness info ---
print("CNT Lab root:", LAB_ROOT)
print("Python:", sys.version.splitlines()[0])
try:
    import torch, importlib
    has_torch = True
except Exception:
    has_torch = False
print("Torch available:", has_torch)

# --- STEP 1: backup (zip snapshot) ---
try:
    backup_zip = make_zip_backup(LAB_ROOT, BACKUP_DIR)
except Exception as e:
    print("Backup failed (aborting). Reason:", e)
    raise

# --- STEP 2: run engine sanity checks (non-destructive) ---
sanity = {}
# 2a: run a known 'run_cycle' if present
try:
    # If your engine is a module exposed in the notebook, try to call run_cycle
    if 'run_cycle' in globals():
        print("Found run_cycle() in globals — executing small call...")
        status, res = safe_call(globals()['run_cycle'])
        sanity['run_cycle'] = status
        sanity['run_cycle_res'] = res if status=='ok' else str(res[0]) + "\n" + res[1]
    else:
        # try to import cnt_engine from expected path (best-effort)
        try:
            import cnt_engine
            run_cycle = getattr(cnt_engine, 'run_cycle', None)
            if callable(run_cycle):
                print("Imported cnt_engine.run_cycle — executing small call...")
                status, res = safe_call(run_cycle)
                sanity['run_cycle'] = status
                sanity['run_cycle_res'] = res if status=='ok' else str(res[0]) + "\n" + res[1]
            else:
                sanity['run_cycle'] = "missing"
                print("cnt_engine.run_cycle not found.")
        except Exception as imp_e:
            sanity['run_cycle'] = "import_fail"
            sanity['run_cycle_err'] = traceback.format_exc()
            print("Could not import cnt_engine:", imp_e)
except Exception as e:
    sanity['run_cycle'] = "err"
    sanity['run_cycle_err'] = traceback.format_exc()

# 2b: discover latest tests_report CSVs in artifacts/out
out_dir = LAB_ROOT / "artifacts" / "cnt_engine_megacell" / "out"
reports = []
if out_dir.exists():
    reports = sorted(out_dir.glob("tests_report_*.csv"), key=lambda p: p.stat().st_mtime)
    if reports:
        print("Found test reports; latest:", reports[-1])
        sanity['latest_report'] = str(reports[-1])
    else:
        print("No tests_report_*.csv found in", out_dir)
        sanity['latest_report'] = None
else:
    print("Expected out dir not found:", out_dir)
    sanity['latest_report'] = None

# 2c: optional engine test command (pytest or bespoke)
if ENGINE_CHECK_CMD:
    print("Running engine test command:", ENGINE_CHECK_CMD)
    try:
        cproc = subprocess.run(ENGINE_CHECK_CMD, shell=True, check=False, capture_output=True, text=True, cwd=str(LAB_ROOT))
        sanity['engine_check_cmd'] = dict(returncode=cproc.returncode, stdout=cproc.stdout[:2000], stderr=cproc.stderr[:2000])
        print("Engine check exit", cproc.returncode)
    except Exception as e:
        sanity['engine_check_cmd'] = ("err", str(e))
        print("Engine check command failed:", e)

# --- STEP 3: small CANARY run (non-destructive, short) ---
canary = {}
try:
    # Strategy: run a single lightweight cycle or a "dry" run, prefer run_cycle(dry=True) if available
    if 'run_cycle' in globals():
        rc = globals()['run_cycle']
        try:
            print("Trying run_cycle(dry=True) ...")
            status, r = safe_call(rc)  # call plain first; many of your failures are in sub-functions so plain call is safest
            canary['run_cycle_call'] = status
            canary['run_cycle_res'] = r if status=='ok' else str(r[0])+"\n"+r[1]
        except TypeError:
            # fallback: no-arg call attempted above; if it failed earlier, capture
            canary['run_cycle_call'] = "typeerror"
    else:
        # fallback: try a tiny file-read and parsing to emulate a canary
        print("No run_cycle in globals; doing a lightweight file-read canary.")
        sample_paths = list((LAB_ROOT / "notebooks" / "archive").glob("**/*.txt"))[:3]
        sample_info = []
        for p in sample_paths:
            try:
                txt = p.read_text(encoding="utf-8", errors="ignore")[:2000]
                sample_info.append((str(p), len(txt)))
            except Exception as e:
                sample_info.append((str(p), f"err:{e}"))
        canary['sample_reads'] = sample_info
except Exception as e:
    canary['err'] = traceback.format_exc()

# --- STEP 4: results & recommended decision ---
print("\n--- SNAPSHOT & SANITY SUMMARY ---")
print("Backup:", backup_zip)
print(json.dumps(sanity, indent=2)[:4000])
print("\nCANARY:", json.dumps(canary, indent=2)[:4000])

# Decision heuristic (suggested thresholds, edit if you like)
# If run_cycle succeeded or we have at least one recent tests_report with OK count > 0, can proceed with canaries.
proceed_reasons = []
if sanity.get('run_cycle') == 'ok':
    proceed_reasons.append("run_cycle succeeded")
if sanity.get('latest_report'):
    try:
        import pandas as pd
        rep = pd.read_csv(sanity['latest_report'])
        ok_count = rep['reason'].eq('ok').sum() if 'reason' in rep.columns else None
        if ok_count and ok_count > 0:
            proceed_reasons.append(f"latest tests_report OK count = {ok_count}")
    except Exception:
        pass

if proceed_reasons:
    print("\nRecommendation: OK to begin limited 'attacks' (canary experiments). Reasons:", proceed_reasons)
    print("Suggested play: run 1-3 small canary experiments, monitor logs, then scale.")
    print("Suggested safety steps (do before heavy runs):")
    print("  1) git commit / tag current state or rely on created backup zip.")
    print("  2) run a single canary on a small dataset/segment.")
    print("  3) inspect tests_report_* CSV and engine logs; if any 'pair_fail' or 'needs_holdout' appear, pause and fix.")
else:
    print("\nRecommendation: do NOT run large experiments yet. Fix the failing/`needs_holdout` tests first.")
    print("Common quick fixes based on your recent errors:")
    print("  * ensure `glob` is imported from the stdlib: `from glob import glob` (your traceback showed `glob` was a function object).")
    print("  * watch for pandas Index vs list: replace `.index.index` mistaken calls with `.to_list()` or `.tolist()`.")
    print("  * rerun the mega cell tests after fixing the two errors (glob and Index.index).")
    print("  * re-run this mega cell after fixes to get a fresh backup + sanity check.")
    print("\nIf you want I can also produce a targeted one-cell patch to fix the two specific tracebacks you saw (the `glob` misuse and the `Index.index` misuse).")

# --- final metadata write (helpful ledger) ---
ledger = {
    "stamp": now_tag(),
    "lab_root": str(LAB_ROOT),
    "backup": str(backup_zip),
    "sanity": sanity,
    "canary": canary,
}
ledger_file = BACKUP_DIR / f"sanity_ledger_{now_tag()}.json"
try:
    ledger_file.write_text(json.dumps(ledger, indent=2))
    print("Wrote ledger:", ledger_file)
except Exception as e:
    print("Failed to write ledger:", e)

# Return summary object for notebook consumption
ledger


CNT Lab root: E:\CNT
Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
Torch available: True
Creating backup: E:\CNT_backups\CNT_backup_20251028-031155.zip


C:\Users\caleb\AppData\Local\Temp\ipykernel_26692\1132815659.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().strftime("%Y%m%d-%H%M%S")
